<a href="https://colab.research.google.com/github/profsandromesquita/doutorado/blob/main/Reordena%C3%A7%C3%A3o_Coordenadas_CAR_T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AUDITORIA DE COORDENADAS REPETIDAS

Lê o PDB

- Identifica automaticamente o primeiro e o último átomo
- Lê ID, X, Y, Z de cada átomo
- Compara coordenadas numericamente (tratando 76.160 e 76.16 como iguais)
- Lista todos os conjuntos de átomos que compartilham exatamente a mesma coordenada (em 3D)

In [ ]:
# -*- coding: utf-8 -*-
# Script para Google Colab: detectar coordenadas repetidas em um PDB

from google.colab import files

def carregar_arquivo_pdb():
    """
    Abre um seletor de arquivo no Colab e retorna o nome do arquivo PDB escolhido.
    """
    print("Faça upload do arquivo PDB (por exemplo: md1amd12-frame-unico-576-583.pdb)")
    uploaded = files.upload()
    if not uploaded:
        raise RuntimeError("Nenhum arquivo foi enviado.")
    # Pega o primeiro arquivo enviado
    filename = next(iter(uploaded.keys()))
    print(f"Arquivo carregado: {filename}")
    return filename


def ler_atomos_pdb(filename):
    """
    Lê o arquivo PDB e retorna uma lista de átomos, cada um como um dicionário:
        {'id': int, 'x': float, 'y': float, 'z': float}
    Considera linhas que começam com 'ATOM' ou 'HETATM'.
    """
    atoms = []
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith(("ATOM", "HETATM")):
                # Exemplo:
                # ATOM   8641  N   ILE B 576      74.770  70.720 169.110  1.00  0.00           N
                parts = line.split()
                # parts[0] = 'ATOM'
                # parts[1] = ID
                # parts[6], parts[7], parts[8] = X, Y, Z
                try:
                    atom_id = int(parts[1])
                    x = float(parts[6])
                    y = float(parts[7])
                    z = float(parts[8])
                except (IndexError, ValueError) as e:
                    # Se alguma linha estiver fora do padrão, apenas avisa e ignora
                    print("Linha PDB inválida ou inesperada, ignorando:")
                    print(line.strip())
                    continue

                atoms.append({
                    "id": atom_id,
                    "x": x,
                    "y": y,
                    "z": z
                })

    if not atoms:
        raise RuntimeError("Nenhum átomo ATOM/HETATM encontrado no arquivo.")

    # Primeiro e último átomo (na ordem em que aparecem no arquivo)
    first_atom = atoms[0]
    last_atom = atoms[-1]

    print("Primeiro átomo encontrado:")
    print(f"  ID: {first_atom['id']}, coords: ({first_atom['x']:.3f}, "
          f"{first_atom['y']:.3f}, {first_atom['z']:.3f})")

    print("Último átomo encontrado:")
    print(f"  ID: {last_atom['id']}, coords: ({last_atom['x']:.3f}, "
          f"{last_atom['y']:.3f}, {last_atom['z']:.3f})")

    return atoms


def encontrar_coordenadas_repetidas(atoms, casas_decimais=3):
    """
    Compara coordenadas de todos os átomos e detecta coordenadas repetidas.

    - Converte X, Y, Z para float (para tratar '76.160' e '76.16' como iguais).
    - Arredonda para 'casas_decimais' (por padrão 3, como em PDB).
    - Usa um dicionário coordenada -> lista de IDs.
    - Retorna um dicionário com apenas as coordenadas que aparecem em >= 2 átomos.
    """
    from collections import defaultdict

    coord_map = defaultdict(list)

    # Construir o mapa de coordenadas
    # Cada chave é (x, y, z) arredondado; o valor é a lista de IDs que têm essa coordenada
    for atom in atoms:
        key = (
            round(atom["x"], casas_decimais),
            round(atom["y"], casas_decimais),
            round(atom["z"], casas_decimais),
        )
        coord_map[key].append(atom["id"])

    # Filtrar apenas coordenadas repetidas
    repetidas = {
        coord: ids for coord, ids in coord_map.items() if len(ids) > 1
    }

    return repetidas


def comparar_por_fluxo_sequencial(atoms, casas_decimais=3):
    """
    Implementa exatamente o fluxo descrito:
      - Fixa um átomo de referência (por ordem no arquivo)
      - Compara com todos os átomos seguintes (id+1, id+2, ..., último)
      - Se coordenadas forem iguais (após conversão para float e arredondamento),
        registra a combinação de IDs em uma lista.

    Retorna uma lista de tuplas: (id_ref, id_igual, (x, y, z)).
    """
    resultados = []
    n = len(atoms)

    # Pré-normaliza coordenadas para evitar fazer round a cada comparação
    coords_norm = []
    for atom in atoms:
        coords_norm.append((
            round(atom["x"], casas_decimais),
            round(atom["y"], casas_decimais),
            round(atom["z"], casas_decimais),
        ))

    for i in range(n - 1):
        id_ref = atoms[i]["id"]
        coord_ref = coords_norm[i]

        # Compara com todos os átomos "depois" do átomo de referência
        for j in range(i + 1, n):
            id_comp = atoms[j]["id"]
            coord_comp = coords_norm[j]

            if coord_comp == coord_ref:
                resultados.append((id_ref, id_comp, coord_ref))

    return resultados


# === Execução principal no Colab ===
filename = carregar_arquivo_pdb()
atoms = ler_atomos_pdb(filename)

print("\n=== Método 1: dicionário coordenada -> lista de IDs ===")
repetidas = encontrar_coordenadas_repetidas(atoms, casas_decimais=3)

if not repetidas:
    print("Nenhuma coordenada repetida encontrada (com 3 casas decimais).")
else:
    print(f"Encontradas {len(repetidas)} coordenadas distintas repetidas:\n")
    for coord, ids in sorted(repetidas.items()):
        x, y, z = coord
        print(f"Coordenada ({x:.3f}, {y:.3f}, {z:.3f}) -> átomos {ids}")

print("\n=== Método 2: fluxo sequencial (referência ID + comparação até o fim) ===")
pares_repetidos = comparar_por_fluxo_sequencial(atoms, casas_decimais=3)

if not pares_repetidos:
    print("Nenhum par de átomos com coordenadas idênticas encontrado.")
else:
    print(f"Foram encontrados {len(pares_repetidos)} pares de átomos com coordenadas idênticas:\n")
    for (id_ref, id_igual, coord) in pares_repetidos:
        x, y, z = coord
        print(f"Átomo ref {id_ref} e átomo {id_igual} -> "
              f"({x:.3f}, {y:.3f}, {z:.3f})")


Faça upload do arquivo PDB (por exemplo: md1amd12-frame-unico-576-583.pdb)


Saving backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581_ARG582.pdb to backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581_ARG582 (1).pdb
Arquivo carregado: backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581_ARG582 (1).pdb
Primeiro átomo encontrado:
  ID: 8641, coords: (74.770, 70.720, 169.110)
Último átomo encontrado:
  ID: 8777, coords: (76.990, 74.880, 184.850)

=== Método 1: dicionário coordenada -> lista de IDs ===
Nenhuma coordenada repetida encontrada (com 3 casas decimais).

=== Método 2: fluxo sequencial (referência ID + comparação até o fim) ===
Nenhum par de átomos com coordenadas idênticas encontrado.


## VALIDANDO COORDENADAS

Boa, agora é hora de fazer o “checksum geométrico” da estrutura. 😄
A ideia: comparar o multiconjunto de coordenadas (x,y,z) do PDB original desordenado com o PDB reestruturado, garantindo:

- Mesmo número de coordenadas;
- Mesma multiplicidade de cada tripla (x, y, z);
- Nenhuma coordenada nova, nenhuma perdida, nenhuma alterada.

Abaixo vai um script completo para Google Colab usando files.upload() e Counter:

In [ ]:
# -*- coding: utf-8 -*-
"""
Validação global de coordenadas entre:
- PDB original desordenado
- PDB reestruturado

Verifica se o multiconjunto de coordenadas (x, y, z) é idêntico:
- nenhuma coordenada foi alterada,
- nenhuma foi deletada,
- nenhuma foi duplicada ou criada.

Comparação feita em float com arredondamento (padrão 3 casas decimais),
para tratar 76.160 vs 76.16 como a mesma coordenada.
"""

from collections import Counter
import math

def coletar_coordenadas_pdb(pdb_path, ndigits=3):
    """
    Lê um arquivo PDB e retorna uma lista de tuplas (x, y, z) arredondadas.
    Considera apenas linhas ATOM / HETATM.
    """
    coords = []
    with open(pdb_path, 'r') as f:
        for line in f:
            if line.startswith(("ATOM", "HETATM")):
                try:
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                except ValueError:
                    # Linha mal formatada ou sem coordenadas numéricas
                    continue
                coords.append((
                    round(x, ndigits),
                    round(y, ndigits),
                    round(z, ndigits),
                ))
    return coords

def comparar_pdbs_coords(pdb_original, pdb_reestruturado, ndigits=3, max_print=30):
    """
    Compara as coordenadas (x,y,z) dos dois PDBs.

    - pdb_original: caminho do PDB desordenado (original)
    - pdb_reestruturado: caminho do PDB reestruturado
    - ndigits: casas decimais para arredondamento
    - max_print: máximo de discrepâncias para listar no console
    """
    coords_orig = coletar_coordenadas_pdb(pdb_original, ndigits=ndigits)
    coords_rest = coletar_coordenadas_pdb(pdb_reestruturado, ndigits=ndigits)

    print(f"Arquivo original      : {pdb_original}")
    print(f"Arquivo reestruturado : {pdb_reestruturado}")
    print(f"Casas decimais usadas para comparação: {ndigits}")
    print()
    print(f"Nº de coordenadas (ATOM/HETATM) no original     : {len(coords_orig)}")
    print(f"Nº de coordenadas (ATOM/HETATM) no reestruturado: {len(coords_rest)}")

    c_orig = Counter(coords_orig)
    c_rest = Counter(coords_rest)

    if c_orig == c_rest:
        print("\n✅ MULTICONJUNTO DE COORDENADAS IDÊNTICO!")
        print("→ Nenhuma coordenada foi alterada, perdida, duplicada ou criada.")
        return

    print("\n⚠️ Foram encontradas diferenças no multiconjunto de coordenadas.")
    print("Analisando discrepâncias por coordenada (x, y, z):\n")

    # Coordenadas que aparecem em um ou outro (união dos conjuntos)
    todas = set(c_orig.keys()) | set(c_rest.keys())

    only_in_orig = []
    only_in_rest = []
    diff_mult = []

    for coord in todas:
        n_orig = c_orig.get(coord, 0)
        n_rest = c_rest.get(coord, 0)
        if n_orig != n_rest:
            if n_orig > n_rest:
                only_in_orig.append((coord, n_orig - n_rest))
            elif n_rest > n_orig:
                only_in_rest.append((coord, n_rest - n_orig))
            diff_mult.append((coord, n_orig, n_rest))

    if not diff_mult:
        print("Diferenças sutis não rastreáveis por contagem (algo muito estranho).")
        return

    print("Resumo das diferenças (coordenada : multiplicidade_no_original -> multiplicidade_no_reestruturado):")
    for coord, n_orig, n_rest in diff_mult[:max_print]:
        print(f"  {coord} : {n_orig} -> {n_rest}")
    if len(diff_mult) > max_print:
        print(f"... (+ {len(diff_mult) - max_print} coordenadas com diferenças não listadas)")

    print("\nCoordenadas presentes MAIS VEZES no ORIGINAL (perdidas / não reaproveitadas no reestruturado):")
    if only_in_orig:
        for coord, delta in only_in_orig[:max_print]:
            print(f"  {coord} : +{delta} ocorrência(s) no original")
        if len(only_in_orig) > max_print:
            print(f"... (+ {len(only_in_orig) - max_print} não listadas)")
    else:
        print("  Nenhuma.")

    print("\nCoordenadas presentes MAIS VEZES no REESTRUTURADO (duplicadas / novas):")
    if only_in_rest:
        for coord, delta in only_in_rest[:max_print]:
            print(f"  {coord} : +{delta} ocorrência(s) no reestruturado")
        if len(only_in_rest) > max_print:
            print(f"... (+ {len(only_in_rest) - max_print} não listadas)")
    else:
        print("  Nenhuma.")

    print("\nConclusão:")
    print("→ Os arquivos NÃO têm exatamente o mesmo multiconjunto de coordenadas.")
    print("  Verifique as listas acima para entender quais coordenadas divergiram.")


# ===========================
# Bloco para uso no Google Colab
# ===========================

if __name__ == "__main__":
    try:
        from google.colab import files  # type: ignore

        print("1) Faça upload do PDB ORIGINAL DESORDENADO (ex.: md1amd12-frame-unico-desordenado.pdb):")
        up1 = files.upload()
        if not up1:
            raise RuntimeError("Nenhum arquivo original foi enviado.")
        pdb_original = list(up1.keys())[0]

        print("\n2) Faça upload do PDB REESTRUTURADO (ex.: md1amd12-frame-unico_reestruturado.pdb):")
        up2 = files.upload()
        if not up2:
            raise RuntimeError("Nenhum arquivo reestruturado foi enviado.")
        pdb_reestruturado = list(up2.keys())[0]

        print("\n=== Iniciando comparação geométrica global ===\n")
        comparar_pdbs_coords(pdb_original, pdb_reestruturado,
                             ndigits=3,  # 76.160 == 76.16
                             max_print=50)

    except ImportError:
        # Modo local (fora do Colab): ajuste os nomes dos arquivos manualmente
        pdb_original = "md1amd12-frame-unico-desordenado.pdb"
        pdb_reestruturado = "md1amd12-frame-unico_reestruturado.pdb"
        print("\nRodando em modo local, usando nomes hardcoded:")
        comparar_pdbs_coords(pdb_original, pdb_reestruturado,
                             ndigits=3,
                             max_print=50)


1) Faça upload do PDB ORIGINAL DESORDENADO (ex.: md1amd12-frame-unico-desordenado.pdb):


Saving md1amd12-frame-unico-desordenado.pdb to md1amd12-frame-unico-desordenado.pdb

2) Faça upload do PDB REESTRUTURADO (ex.: md1amd12-frame-unico_reestruturado.pdb):


Saving md1amd12-frame-unico_reestruturado.txt to md1amd12-frame-unico_reestruturado.txt

=== Iniciando comparação geométrica global ===

Arquivo original      : md1amd12-frame-unico-desordenado.pdb
Arquivo reestruturado : md1amd12-frame-unico_reestruturado.txt
Casas decimais usadas para comparação: 3

Nº de coordenadas (ATOM/HETATM) no original     : 11200
Nº de coordenadas (ATOM/HETATM) no reestruturado: 11200

✅ MULTICONJUNTO DE COORDENADAS IDÊNTICO!
→ Nenhuma coordenada foi alterada, perdida, duplicada ou criada.


## BUSCA POR ÁTOMOS PRÓXIMOS AO BACKBONE

In [ ]:
# -*- coding: utf-8 -*-
# Script para Google Colab: vizinhos mais próximos de N, CA e C em um PDB

from google.colab import files
import math

def carregar_arquivo_pdb():
    """
    Abre um seletor de arquivo no Colab e retorna o nome do arquivo PDB escolhido.
    """
    print("Faça upload do arquivo PDB (por exemplo: md1amd12-frame-unico-576-583.pdb)")
    uploaded = files.upload()
    if not uploaded:
        raise RuntimeError("Nenhum arquivo foi enviado.")
    # Pega o primeiro arquivo enviado
    filename = next(iter(uploaded.keys()))
    print(f"Arquivo carregado: {filename}")
    return filename


def ler_atomos_pdb(filename):
    """
    Lê o arquivo PDB e retorna uma lista de átomos.
    Cada átomo é um dicionário com:
        {
          'id': int,          # serial
          'name': str,        # nome do átomo (N, CA, C, HN, etc.)
          'resname': str,     # nome do resíduo (ILE, THR, etc.)
          'chain': str,       # ID da cadeia (B, A, etc.) se existir
          'resid': int,       # número do resíduo
          'x': float,
          'y': float,
          'z': float,
          'line': str         # linha PDB original
        }

    Usa o formato de colunas padrão PDB para máxima robustez.
    """
    atoms = []
    with open(filename, 'r') as f:
        for line in f:
            record = line[0:6].strip()
            if record not in ("ATOM", "HETATM"):
                continue

            try:
                atom_id  = int(line[6:11])
                name     = line[12:16].strip()
                resname  = line[17:20].strip()
                chain    = line[21].strip()
                resid    = int(line[22:26])
                x        = float(line[30:38])
                y        = float(line[38:46])
                z        = float(line[46:54])
            except (ValueError, IndexError):
                print("Linha PDB inválida ou inesperada, ignorando:")
                print(line.rstrip("\n"))
                continue

            atoms.append({
                "id": atom_id,
                "name": name,
                "resname": resname,
                "chain": chain,
                "resid": resid,
                "x": x,
                "y": y,
                "z": z,
                "line": line.rstrip("\n")
            })

    if not atoms:
        raise RuntimeError("Nenhum átomo ATOM/HETATM encontrado no arquivo.")

    first_atom = atoms[0]
    last_atom = atoms[-1]

    print("Primeiro átomo encontrado:")
    print(f"  {first_atom['line']}")
    print("Último átomo encontrado:")
    print(f"  {last_atom['line']}")

    return atoms


def distancia_euclidiana(a, b):
    """
    Distância euclidiana entre dois átomos (dicionários com x, y, z).
    """
    dx = a["x"] - b["x"]
    dy = a["y"] - b["y"]
    dz = a["z"] - b["z"]
    return math.sqrt(dx*dx + dy*dy + dz*dz)


def encontrar_vizinhos_proximos(atoms, n_vizinhos=4):
    """
    Para cada átomo de referência (N, CA, C), procura os n_vizinhos mais próximos
    em termos de distância euclidiana.

    A busca segue o fluxo solicitado:
      - Para um átomo de referência com índice i na lista:
          1) percorre átomos posteriores (i+1 até o fim)
          2) percorre átomos anteriores (i-1 até o início)
      - Não compara com ele mesmo (mesmo ID).

    Retorna uma lista de entradas:
      [
        {
          'ref_atom': <dict do átomo de referência>,
          'neighbors': [ lista de até n_vizinhos dicts com info e distância ]
        },
        ...
      ]
    """
    resultados = []

    # Índices dos átomos de referência (N, CA, C)
    ref_indices = [
        i for i, atom in enumerate(atoms)
        if atom["name"] in ("N", "CA", "C")
    ]

    if not ref_indices:
        raise RuntimeError("Nenhum átomo de referência (N, CA, C) encontrado na estrutura.")

    n = len(atoms)

    for idx_count, i in enumerate(ref_indices, start=1):
        ref_atom = atoms[i]
        ref_id = ref_atom["id"]

        vizinhos = []

        # 1) IDs posteriores (i+1 -> n-1)
        for j in range(i + 1, n):
            atom_j = atoms[j]
            if atom_j["id"] == ref_id:
                continue  # não compara com ele mesmo (seguro, mas aqui já não aconteceria)
            dist = distancia_euclidiana(ref_atom, atom_j)
            vizinhos.append({
                "id": atom_j["id"],
                "name": atom_j["name"],
                "resname": atom_j["resname"],
                "resid": atom_j["resid"],
                "chain": atom_j["chain"],
                "x": atom_j["x"],
                "y": atom_j["y"],
                "z": atom_j["z"],
                "distance": dist
            })

        # 2) IDs anteriores (i-1 -> 0)
        for j in range(i - 1, -1, -1):
            atom_j = atoms[j]
            if atom_j["id"] == ref_id:
                continue
            dist = distancia_euclidiana(ref_atom, atom_j)
            vizinhos.append({
                "id": atom_j["id"],
                "name": atom_j["name"],
                "resname": atom_j["resname"],
                "resid": atom_j["resid"],
                "chain": atom_j["chain"],
                "x": atom_j["x"],
                "y": atom_j["y"],
                "z": atom_j["z"],
                "distance": dist
            })

        # Ordena por distância crescente e pega os n_vizinhos mais próximos
        vizinhos_ordenados = sorted(vizinhos, key=lambda v: v["distance"])
        vizinhos_top = vizinhos_ordenados[:n_vizinhos]

        resultados.append({
            "ref_atom": ref_atom,
            "neighbors": vizinhos_top
        })

    return resultados


def imprimir_relatorio(resultados):
    """
    Imprime o relatório no formato solicitado:

    REF 1: <linha original do átomo de referência>
    <nome_ref> - <id>  <nome>  <resname>  <resid>  (x  y  z) - distancia
    ...
    """
    for k, entry in enumerate(resultados, start=1):
        ref_atom = entry["ref_atom"]
        neighbors = entry["neighbors"]

        print(f"REF {k}: {ref_atom['line']}")
        ref_name = ref_atom["name"]

        for v in neighbors:
            x = v["x"]
            y = v["y"]
            z = v["z"]
            dist = v["distance"]
            print(
                f"{ref_name} - {v['id']:5d}  {v['name']:<4s} {v['resname']:>3s} {v['resid']:4d}  "
                f"({x:7.3f}  {y:7.3f}  {z:7.3f}) - {dist:6.3f}"
            )
        print()  # linha em branco entre referências


# === Execução principal no Colab ===
filename = carregar_arquivo_pdb()
atoms = ler_atomos_pdb(filename)

print("\nCalculando vizinhos mais próximos (N, CA, C)...\n")
resultados = encontrar_vizinhos_proximos(atoms, n_vizinhos=4)

print("=== RELATÓRIO FINAL ===\n")
imprimir_relatorio(resultados)


Faça upload do arquivo PDB (por exemplo: md1amd12-frame-unico-576-583.pdb)


Saving md1amd12-frame-unico-576-583.pdb to md1amd12-frame-unico-576-583 (1).pdb
Arquivo carregado: md1amd12-frame-unico-576-583 (1).pdb
Primeiro átomo encontrado:
  ATOM   8641  N   ILE B 576      74.770  70.720 169.110  1.00  0.00           N
Último átomo encontrado:
  ATOM   8777  O   GLY B 583      76.990  74.880 184.850  1.00  0.00           O

Calculando vizinhos mais próximos (N, CA, C)...

=== RELATÓRIO FINAL ===

REF 1: ATOM   8641  N   ILE B 576      74.770  70.720 169.110  1.00  0.00           N
N -  8642  HN   ILE  576  ( 74.480   71.330  168.380) -  0.995
N -  8643  CA   ILE  576  ( 75.490   71.380  170.140) -  1.419
N -  8644  HA   ILE  576  ( 76.260   70.740  170.540) -  2.065
N -  8653  2HG1 ILE  576  ( 76.770   71.430  167.970) -  2.409

REF 2: ATOM   8643  CA  ILE B 576      75.490  71.380 170.140  1.00  0.00           C
CA -  8644  HA   ILE  576  ( 76.260   70.740  170.540) -  1.078
CA -  8641  N    ILE  576  ( 74.770   70.720  169.110) -  1.419
CA -  8658  C    ILE  

## POSSÍVEIS ROTAS

In [ ]:
# -*- coding: utf-8 -*-
"""
Exploração de todos os caminhos possíveis de backbone N–CA–C
entre N(576) [8641] e C(583) [8776] com troca de coordenadas por caminho.

- Lê um PDB via upload (no Colab).
- Reconstrói a ordem canônica do backbone (N, CA, C) dos resíduos 576 a 583.
- Explora todos os caminhos N–CA–C–N–CA–C... com:
    * restrições geométricas por tipo de ligação;
    * até 23 ligações (24 vértices);
    * comprimento total entre 30 e 40 Å para ser aceito.
- Em cada ramo, faz swaps de coordenadas locais e mantém uma matriz
  de coordenadas independente por caminho.
"""

import math
import copy

# -------------------------------------------------------------
# Utilitário para upload no Google Colab
# -------------------------------------------------------------
def carregar_pdb_colab():
    """
    Tenta usar o botão de upload do Colab.
    Se não estiver no Colab, cai para input de nome de arquivo.
    """
    try:
        from google.colab import files  # type: ignore
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo enviado.")
        # Pega o primeiro arquivo enviado
        filename = next(iter(uploaded.keys()))
        print(f"PDB carregado: {filename}")
        return filename
    except ImportError:
        # Fallback para ambiente local / outro ambiente
        filename = input("Informe o caminho do arquivo PDB: ").strip()
        return filename


# -------------------------------------------------------------
# Parsing de PDB
# -------------------------------------------------------------
def parse_pdb_coords(filename):
    """
    Lê um arquivo PDB e retorna:
    - atoms: lista de dicionários com info geométrica
    - atom_by_serial: dict serial -> dict do átomo
    """
    atoms = []
    with open(filename, "r") as f:
        for line in f:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                continue
            # PDB format (colunas padrão)
            serial = int(line[6:11])
            name = line[12:16].strip()
            resname = line[17:20].strip()
            chain = line[21].strip()
            resseq = int(line[22:26])
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])

            atom = {
                "serial": serial,
                "name": name,
                "resname": resname,
                "chain": chain,
                "resseq": resseq,
                "x": x,
                "y": y,
                "z": z,
                "line": line.rstrip("\n")
            }
            atoms.append(atom)

    # Ordena por serial para garantir consistência
    atoms.sort(key=lambda a: a["serial"])
    atom_by_serial = {a["serial"]: a for a in atoms}
    print(f"Total de átomos lidos: {len(atoms)}")
    return atoms, atom_by_serial


# -------------------------------------------------------------
# Definição da ordem canônica do backbone
# -------------------------------------------------------------
def build_backbone_order(atoms, atom_by_serial,
                         start_serial=8641, end_serial=8776):
    """
    Constrói a ordem canônica do backbone (N, CA, C) do resíduo inicial
    ao final, com base nos próprios registros do PDB.

    Usa:
    - start_serial: ID do N inicial (ex.: 8641, N ILE B 576)
    - end_serial:   ID do C final   (ex.: 8776, C GLY B 583)

    Retorna:
    - backbone_ids: lista de seriais [N576, CA576, C576, N577, CA577, ... C583]
    """
    if start_serial not in atom_by_serial:
        raise ValueError(f"Start_serial {start_serial} não encontrado no PDB.")
    if end_serial not in atom_by_serial:
        raise ValueError(f"End_serial {end_serial} não encontrado no PDB.")

    start_atom = atom_by_serial[start_serial]
    end_atom = atom_by_serial[end_serial]

    chain = start_atom["chain"]
    res_start = start_atom["resseq"]
    res_end = end_atom["resseq"]

    if chain != end_atom["chain"]:
        raise ValueError("Start e end estão em cadeias diferentes, revise.")

    backbone_ids = []

    for res in range(res_start, res_end + 1):
        for aname in ("N", "CA", "C"):
            candidates = [
                a for a in atoms
                if a["chain"] == chain
                and a["resseq"] == res
                and a["name"] == aname
            ]
            if not candidates:
                raise ValueError(
                    f"Não encontrei átomo {aname} no resíduo {res} cadeia {chain}."
                )
            # Assume único
            backbone_ids.append(candidates[0]["serial"])

    # Checagens de consistência
    if backbone_ids[0] != start_serial:
        raise ValueError(
            f"O primeiro átomo do backbone ({backbone_ids[0]}) "
            f"não é o start_serial ({start_serial})."
        )
    if backbone_ids[-1] != end_serial:
        raise ValueError(
            f"O último átomo do backbone ({backbone_ids[-1]}) "
            f"não é o end_serial ({end_serial})."
        )

    print("Backbone canônico (serial, nome, res, resseq):")
    for s in backbone_ids:
        a = atom_by_serial[s]
        print(f"  {s:5d}  {a['name']:>3s}  {a['resname']:>3s}  {a['resseq']:4d}")

    return backbone_ids


# -------------------------------------------------------------
# Geometria: distâncias e janelas permitidas
# -------------------------------------------------------------
def dist(p1, p2):
    """Distância euclidiana entre duas coordenadas 3D."""
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    dz = p1[2] - p2[2]
    return math.sqrt(dx*dx + dy*dy + dz*dz)


def bond_limits(name1, name2):
    """
    Retorna (d_min, d_max) para o tipo de ligação entre name1 e name2
    (backbone N/CA/C).

    Janelas (em Å):
      N–CA:  1.40 – 1.60
      CA–C:  1.40 – 1.70
      C–N:   1.25 – 1.45
    """
    pair = (name1, name2)

    if pair in (("N", "CA"), ("CA", "N")):
        return (1.40, 1.60)
    if pair in (("CA", "C"), ("C", "CA")):
        return (1.40, 1.70)
    if pair in (("C", "N"), ("N", "C")):
        return (1.25, 1.45)

    # Se aparecer algo estranho, retornamos None para marcar erro
    return None


# -------------------------------------------------------------
# Busca de todos os caminhos com árvore de estados
# -------------------------------------------------------------
def find_all_backbone_paths(atoms, atom_by_serial, backbone_ids,
                            max_steps=23,
                            min_total=30.0,
                            max_total=40.0):
    """
    Explora todos os caminhos possíveis entre:
      backbone_ids[0] (N 576) e backbone_ids[-1] (C 583)
    seguindo a ordem canônica de backbone, mas permitindo que as
    COORDENADAS venham de quaisquer átomos candidatos (via swap).

    - Cada estado tem:
        * step: índice do último vértice canônico definido (0..23)
        * coords: dict serial -> (x, y, z) para ESTE caminho
        * locked: conjunto de seriais cuja coordenada já é vértice do backbone
        * total: soma das distâncias entre vértices do backbone
        * edges: lista de passos com metadados

    Retorna:
    - solutions: lista de estados finais válidos (23 passos, soma entre 30 e 40 Å)
    - stats: dicionário com contagens pedidas
    """
    # Matriz de coordenadas inicial (compartilhada apenas no início)
    coords0 = {a["serial"]: (a["x"], a["y"], a["z"]) for a in atoms}
    all_serials = [a["serial"] for a in atoms]

    L = len(backbone_ids)
    if L - 1 != max_steps:
        raise ValueError(
            f"Número de passos (backbone_ids - 1 = {L-1}) "
            f"não bate com max_steps={max_steps}."
        )

    start_id = backbone_ids[0]
    end_id = backbone_ids[-1]

    # Estatísticas
    stats = {
        "caminhos_estouraram_23_passos": 0,      # aqui tenderá a ficar 0 (ordem canônica fixa)
        "caminhos_passaram_40A_antes_23": 0,
        "caminhos_chegaram_alvo_23_menor_30A": 0,
        "caminhos_chegaram_alvo_23_entre_30e40A": 0,
        "caminhos_mortos_sem_candidato": 0       # extra: sem candidato em algum passo
    }

    # Soluções finais
    solutions = []

    # Cada estado é um dicionário conforme descrito acima
    inicial = {
        "step": 0,                    # já estamos no vértice 0 (N 576)
        "coords": coords0,
        "locked": {start_id},         # N(576) já é vértice e nunca mais troca
        "total": 0.0,
        "edges": []                   # nenhum passo ainda
    }

    # Vamos usar DFS (stack) para explorar caminhos
    stack = [inicial]

    while stack:
        state = stack.pop()
        k = state["step"]
        total = state["total"]

        # Se já chegamos ao último vértice canônico (C 583)
        if k == max_steps:
            # Já chegamos no átomo final (por construção é backbone_ids[-1])
            # Verifica faixa de distância total
            if total < min_total:
                stats["caminhos_chegaram_alvo_23_menor_30A"] += 1
            elif total <= max_total:
                stats["caminhos_chegaram_alvo_23_entre_30e40A"] += 1
                solutions.append(state)
            else:
                # Caminho final com >40 Å (não foi pedido explicitamente,
                # mas é claramente inválido; poderíamos ter podado antes)
                stats["caminhos_passaram_40A_antes_23"] += 1
            continue

        # Se ainda não completou o backbone, expandimos mais um passo
        curr_id = backbone_ids[k]
        next_id = backbone_ids[k + 1]

        curr_atom = atom_by_serial[curr_id]
        next_atom = atom_by_serial[next_id]

        limits = bond_limits(curr_atom["name"], next_atom["name"])
        if limits is None:
            raise ValueError(
                f"Par de backbone inesperado: {curr_atom['name']}-{next_atom['name']}"
            )
        d_min, d_max = limits

        curr_coord = state["coords"][curr_id]

        found_candidate = False

        # Loop em TODOS os átomos como candidatos de coordenadas para o próximo vértice
        for cand_id in all_serials:
            # Não podemos usar um átomo cuja coordenada já é vértice fixo
            if cand_id in state["locked"]:
                continue

            cand_coord = state["coords"][cand_id]
            d = dist(curr_coord, cand_coord)

            if d < d_min or d > d_max:
                continue

            # Candidato aceito: geramos novo estado (novo caminho)
            found_candidate = True

            # Copia da matriz de coordenadas para ESTE caminho filho
            new_coords = state["coords"].copy()

            # Swap de coordenadas entre next_id (vértice canônico do backbone)
            # e o cand_id (átomo candidato), se forem diferentes
            if cand_id != next_id:
                tmp = new_coords[next_id]
                new_coords[next_id] = new_coords[cand_id]
                new_coords[cand_id] = tmp

            new_total = total + d
            new_step = k + 1

            # Poda por comprimento > 40 Å antes de completar os 23 passos
            if new_total > max_total and new_step < max_steps:
                stats["caminhos_passaram_40A_antes_23"] += 1
                continue

            # Atualiza conjunto de vértices fixos (backbone já definido)
            new_locked = set(state["locked"])
            new_locked.add(next_id)

            # Acrescenta aresta ao histórico
            new_edges = list(state["edges"])
            new_edges.append({
                "from": curr_id,
                "to": next_id,
                "donor": cand_id,      # átomo que forneceu a coordenada
                "distance": d
            })

            new_state = {
                "step": new_step,
                "coords": new_coords,
                "locked": new_locked,
                "total": new_total,
                "edges": new_edges
            }

            stack.append(new_state)

        # Se não houve nenhum candidato neste passo, o caminho morre aqui
        if not found_candidate:
            stats["caminhos_mortos_sem_candidato"] += 1

    # Por construção, não existem caminhos que "estouram" 23 passos
    # sem cair exatamente no último vértice, então esta estatística
    # tende a ficar em 0. Mantemos só pela especificação.
    return solutions, stats


# -------------------------------------------------------------
# Impressão legível dos caminhos
# -------------------------------------------------------------
def print_solution(sol, backbone_ids, atom_by_serial, idx=1):
    """
    Imprime um caminho em formato legível:

    REF 1: ATOM ... (N inicial)
    N  -> CA ...
    CA -> C  ...
    ...
    """
    print("=" * 80)
    print(f"Caminho #{idx}  |  passos = {len(sol['edges'])}, "
          f"soma distâncias = {sol['total']:.3f} Å")
    print("-" * 80)

    coords = sol["coords"]

    # REF 1
    first_id = backbone_ids[0]
    a0 = atom_by_serial[first_id]
    x0, y0, z0 = coords[first_id]
    print(
        f"REF 1: {a0['line']}\n"
        f"       coords usadas neste caminho: "
        f"({x0:8.3f} {y0:8.3f} {z0:8.3f})"
    )
    print()

    # Cada aresta (ligação backbone)
    for i, edge in enumerate(sol["edges"], start=1):
        from_id = edge["from"]
        to_id = edge["to"]
        donor_id = edge["donor"]
        d = edge["distance"]

        af = atom_by_serial[from_id]
        at = atom_by_serial[to_id]
        xt, yt, zt = coords[to_id]

        print(
            f"PASSO {i:2d}: "
            f"{af['name']:>2s}({from_id:5d}) {af['resname']:>3s} {af['resseq']:4d}  ->  "
            f"{at['name']:>2s}({to_id:5d}) {at['resname']:>3s} {at['resseq']:4d}  "
            f"coord_to=({xt:8.3f} {yt:8.3f} {zt:8.3f})  "
            f"dist={d:6.3f}  donor_id={donor_id:5d}"
        )

    print("=" * 80)
    print()


# -------------------------------------------------------------
# Função principal
# -------------------------------------------------------------
def main():
    # 1) Carrega PDB
    filename = carregar_pdb_colab()

    # 2) Faz parsing
    atoms, atom_by_serial = parse_pdb_coords(filename)

    # 3) Define backbone canônico entre N(576) [8641] e C(583) [8776]
    #    Se quiser generalizar, basta mudar esses IDs.
    START_SERIAL = 8641
    END_SERIAL = 8776

    backbone_ids = build_backbone_order(
        atoms,
        atom_by_serial,
        start_serial=START_SERIAL,
        end_serial=END_SERIAL
    )

    # 4) Explora todos os caminhos com restrições geométricas
    solutions, stats = find_all_backbone_paths(
        atoms,
        atom_by_serial,
        backbone_ids,
        max_steps=23,
        min_total=30.0,
        max_total=40.0
    )

    # 5) Estatísticas globais
    print("\n===== ESTATÍSTICAS GERAIS =====")
    print(f"Total de caminhos válidos (23 passos, 30–40 Å): "
          f"{stats['caminhos_chegaram_alvo_23_entre_30e40A']}")
    print(f"Caminhos que passaram de 40 Å com menos de 23 passos: "
          f"{stats['caminhos_passaram_40A_antes_23']}")
    print(f"Caminhos que chegaram ao alvo com 23 passos mas < 30 Å: "
          f"{stats['caminhos_chegaram_alvo_23_menor_30A']}")
    print(f"Caminhos que estouraram 23 passos (teoricamente 0 aqui): "
          f"{stats['caminhos_estouraram_23_passos']}")
    print(f"Caminhos mortos por falta de candidato em algum passo: "
          f"{stats['caminhos_mortos_sem_candidato']}")

    # 6) Imprime caminhos válidos (se houver)
    if solutions:
        print("\n===== CAMINHOS VÁLIDOS ENCONTRADOS =====\n")
        for i, sol in enumerate(solutions, start=1):
            print_solution(sol, backbone_ids, atom_by_serial, idx=i)
    else:
        print("\nNenhum caminho válido encontrado com as restrições atuais.")


# -----------------------------------------------------------------
# Execução
# -----------------------------------------------------------------
if __name__ == "__main__":
    main()


Saving md1amd12-frame-unico-576-583.pdb to md1amd12-frame-unico-576-583.pdb
PDB carregado: md1amd12-frame-unico-576-583.pdb
Total de átomos lidos: 137
Backbone canônico (serial, nome, res, resseq):
   8641    N  ILE   576
   8643   CA  ILE   576
   8658    C  ILE   576
   8660    N  THR   577
   8662   CA  THR   577
   8672    C  THR   577
   8674    N  LEU   578
   8676   CA  LEU   578
   8691    C  LEU   578
   8693    N  TYR   579
   8695   CA  TYR   579
   8712    C  TYR   579
   8714    N  CYS   580
   8716   CA  CYS   580
   8723    C  CYS   580
   8725    N  LYS   581
   8727   CA  LYS   581
   8745    C  LYS   581
   8747    N  ARG   582
   8749   CA  ARG   582
   8769    C  ARG   582
   8771    N  GLY   583
   8773   CA  GLY   583
   8776    C  GLY   583

===== ESTATÍSTICAS GERAIS =====
Total de caminhos válidos (23 passos, 30–40 Å): 1
Caminhos que passaram de 40 Å com menos de 23 passos: 0
Caminhos que chegaram ao alvo com 23 passos mas < 30 Å: 0
Caminhos que estouraram 23 pa

## GERANDO UM PDB COM BACKBONE REESTRUTURADO

In [ ]:
# -*- coding: utf-8 -*-
"""
Exploração de todos os caminhos possíveis de backbone N–CA–C
entre N(576) [8641] e C(583) [8776] com troca de coordenadas por caminho.

NOVO:
- Para cada caminho válido encontrado, gera um novo arquivo .pdb
  com as coordenadas trocadas daquele caminho.

- Lê um PDB via upload (no Colab).
- Reconstrói a ordem canônica do backbone (N, CA, C) dos resíduos 576 a 583.
- Explora todos os caminhos N–CA–C–N–CA–C... com:
    * restrições geométricas por tipo de ligação;
    * até 23 ligações (24 vértices);
    * comprimento total entre 30 e 40 Å para ser aceito.
"""

import math

# -------------------------------------------------------------
# Utilitário para upload no Google Colab
# -------------------------------------------------------------
def carregar_pdb_colab():
    """
    Tenta usar o botão de upload do Colab.
    Se não estiver no Colab, cai para input de nome de arquivo.
    """
    try:
        from google.colab import files  # type: ignore
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo enviado.")
        # Pega o primeiro arquivo enviado
        filename = next(iter(uploaded.keys()))
        print(f"PDB carregado: {filename}")
        return filename
    except ImportError:
        # Fallback para ambiente local / outro ambiente
        filename = input("Informe o caminho do arquivo PDB: ").strip()
        return filename


# -------------------------------------------------------------
# Parsing de PDB
# -------------------------------------------------------------
def parse_pdb_coords(filename):
    """
    Lê um arquivo PDB e retorna:
    - atoms: lista de dicionários com info geométrica
    - atom_by_serial: dict serial -> dict do átomo
    """
    atoms = []
    with open(filename, "r") as f:
        for line in f:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                continue
            # PDB format (colunas padrão)
            serial = int(line[6:11])
            name = line[12:16].strip()
            resname = line[17:20].strip()
            chain = line[21].strip()
            resseq = int(line[22:26])
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])

            atom = {
                "serial": serial,
                "name": name,
                "resname": resname,
                "chain": chain,
                "resseq": resseq,
                "x": x,
                "y": y,
                "z": z,
                "line": line.rstrip("\n")
            }
            atoms.append(atom)

    # Ordena por serial para garantir consistência
    atoms.sort(key=lambda a: a["serial"])
    atom_by_serial = {a["serial"]: a for a in atoms}
    print(f"Total de átomos lidos: {len(atoms)}")
    return atoms, atom_by_serial


# -------------------------------------------------------------
# Definição da ordem canônica do backbone
# -------------------------------------------------------------
def build_backbone_order(atoms, atom_by_serial,
                         start_serial=8641, end_serial=8776):
    """
    Constrói a ordem canônica do backbone (N, CA, C) do resíduo inicial
    ao final, com base nos próprios registros do PDB.

    Usa:
    - start_serial: ID do N inicial (ex.: 8641, N ILE B 576)
    - end_serial:   ID do C final   (ex.: 8776, C GLY B 583)

    Retorna:
    - backbone_ids: lista de seriais [N576, CA576, C576, N577, CA577, ... C583]
    """
    if start_serial not in atom_by_serial:
        raise ValueError(f"Start_serial {start_serial} não encontrado no PDB.")
    if end_serial not in atom_by_serial:
        raise ValueError(f"End_serial {end_serial} não encontrado no PDB.")

    start_atom = atom_by_serial[start_serial]
    end_atom = atom_by_serial[end_serial]

    chain = start_atom["chain"]
    res_start = start_atom["resseq"]
    res_end = end_atom["resseq"]

    if chain != end_atom["chain"]:
        raise ValueError("Start e end estão em cadeias diferentes, revise.")

    backbone_ids = []

    for res in range(res_start, res_end + 1):
        for aname in ("N", "CA", "C"):
            candidates = [
                a for a in atoms
                if a["chain"] == chain
                and a["resseq"] == res
                and a["name"] == aname
            ]
            if not candidates:
                raise ValueError(
                    f"Não encontrei átomo {aname} no resíduo {res} cadeia {chain}."
                )
            # Assume único
            backbone_ids.append(candidates[0]["serial"])

    # Checagens de consistência
    if backbone_ids[0] != start_serial:
        raise ValueError(
            f"O primeiro átomo do backbone ({backbone_ids[0]}) "
            f"não é o start_serial ({start_serial})."
        )
    if backbone_ids[-1] != end_serial:
        raise ValueError(
            f"O último átomo do backbone ({backbone_ids[-1]}) "
            f"não é o end_serial ({end_serial})."
        )

    print("Backbone canônico (serial, nome, res, resseq):")
    for s in backbone_ids:
        a = atom_by_serial[s]
        print(f"  {s:5d}  {a['name']:>3s}  {a['resname']:>3s}  {a['resseq']:4d}")

    return backbone_ids


# -------------------------------------------------------------
# Geometria: distâncias e janelas permitidas
# -------------------------------------------------------------
def dist(p1, p2):
    """Distância euclidiana entre duas coordenadas 3D."""
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    dz = p1[2] - p2[2]
    return math.sqrt(dx*dx + dy*dy + dz*dz)


def bond_limits(name1, name2):
    """
    Retorna (d_min, d_max) para o tipo de ligação entre name1 e name2
    (backbone N/CA/C).

    Janelas (em Å):
      N–CA:  1.40 – 1.60
      CA–C:  1.40 – 1.70
      C–N:   1.25 – 1.45
    """
    pair = (name1, name2)

    if pair in (("N", "CA"), ("CA", "N")):
        return (1.40, 1.60)
    if pair in (("CA", "C"), ("C", "CA")):
        return (1.40, 1.70)
    if pair in (("C", "N"), ("N", "C")):
        return (1.25, 1.45)

    # Se aparecer algo estranho, retornamos None para marcar erro
    return None


# -------------------------------------------------------------
# Busca de todos os caminhos com árvore de estados
# -------------------------------------------------------------
def find_all_backbone_paths(atoms, atom_by_serial, backbone_ids,
                            max_steps=23,
                            min_total=30.0,
                            max_total=40.0):
    """
    Explora todos os caminhos possíveis entre:
      backbone_ids[0] (N 576) e backbone_ids[-1] (C 583)
    seguindo a ordem canônica de backbone, mas permitindo que as
    COORDENADAS venham de quaisquer átomos candidatos (via swap).

    - Cada estado tem:
        * step: índice do último vértice canônico definido (0..23)
        * coords: dict serial -> (x, y, z) para ESTE caminho
        * locked: conjunto de seriais cuja coordenada já é vértice do backbone
        * total: soma das distâncias entre vértices do backbone
        * edges: lista de passos com metadados

    Retorna:
    - solutions: lista de estados finais válidos (23 passos, soma entre 30 e 40 Å)
    - stats: dicionário com contagens pedidas
    """
    # Matriz de coordenadas inicial (compartilhada apenas no início)
    coords0 = {a["serial"]: (a["x"], a["y"], a["z"]) for a in atoms}
    all_serials = [a["serial"] for a in atoms]

    L = len(backbone_ids)
    if L - 1 != max_steps:
        raise ValueError(
            f"Número de passos (backbone_ids - 1 = {L-1}) "
            f"não bate com max_steps={max_steps}."
        )

    start_id = backbone_ids[0]
    end_id = backbone_ids[-1]

    # Estatísticas
    stats = {
        "caminhos_estouraram_23_passos": 0,      # aqui tende a ficar 0
        "caminhos_passaram_40A_antes_23": 0,
        "caminhos_chegaram_alvo_23_menor_30A": 0,
        "caminhos_chegaram_alvo_23_entre_30e40A": 0,
        "caminhos_mortos_sem_candidato": 0
    }

    # Soluções finais
    solutions = []

    # Estado inicial
    inicial = {
        "step": 0,                    # já estamos no vértice 0 (N 576)
        "coords": coords0,
        "locked": {start_id},         # N(576) já é vértice e nunca mais troca
        "total": 0.0,
        "edges": []                   # nenhum passo ainda
    }

    # DFS com pilha
    stack = [inicial]

    while stack:
        state = stack.pop()
        k = state["step"]
        total = state["total"]

        # Se já chegamos ao último vértice canônico (C 583)
        if k == max_steps:
            # Verifica faixa de distância total
            if total < min_total:
                stats["caminhos_chegaram_alvo_23_menor_30A"] += 1
            elif total <= max_total:
                stats["caminhos_chegaram_alvo_23_entre_30e40A"] += 1
                solutions.append(state)
            else:
                stats["caminhos_passaram_40A_antes_23"] += 1
            continue

        # Ainda não completou backbone: expandimos
        curr_id = backbone_ids[k]
        next_id = backbone_ids[k + 1]

        curr_atom = atom_by_serial[curr_id]
        next_atom = atom_by_serial[next_id]

        limits = bond_limits(curr_atom["name"], next_atom["name"])
        if limits is None:
            raise ValueError(
                f"Par de backbone inesperado: {curr_atom['name']}-{next_atom['name']}"
            )
        d_min, d_max = limits

        curr_coord = state["coords"][curr_id]

        found_candidate = False

        # Loop em TODOS os átomos como candidatos de coordenadas
        for cand_id in all_serials:
            # Não podemos usar um átomo cuja coordenada já é vértice fixo
            if cand_id in state["locked"]:
                continue

            cand_coord = state["coords"][cand_id]
            d = dist(curr_coord, cand_coord)

            if d < d_min or d > d_max:
                continue

            # Candidato aceito: novo estado (novo caminho)
            found_candidate = True

            # Copia rasa da matriz de coordenadas (dict serial -> coord)
            new_coords = state["coords"].copy()

            # Swap de coordenadas entre next_id (vértice canônico do backbone)
            # e o cand_id (átomo candidato), se forem diferentes
            if cand_id != next_id:
                tmp = new_coords[next_id]
                new_coords[next_id] = new_coords[cand_id]
                new_coords[cand_id] = tmp

            new_total = total + d
            new_step = k + 1

            # Poda por comprimento > 40 Å antes de completar 23 passos
            if new_total > max_total and new_step < max_steps:
                stats["caminhos_passaram_40A_antes_23"] += 1
                continue

            # Atualiza conjunto de vértices fixos (backbone já definido)
            new_locked = set(state["locked"])
            new_locked.add(next_id)

            # Acrescenta aresta ao histórico
            new_edges = list(state["edges"])
            new_edges.append({
                "from": curr_id,
                "to": next_id,
                "donor": cand_id,      # átomo que forneceu a coordenada
                "distance": d
            })

            new_state = {
                "step": new_step,
                "coords": new_coords,
                "locked": new_locked,
                "total": new_total,
                "edges": new_edges
            }

            stack.append(new_state)

        # Se não houve nenhum candidato neste passo, o caminho morre aqui
        if not found_candidate:
            stats["caminhos_mortos_sem_candidato"] += 1

    return solutions, stats


# -------------------------------------------------------------
# Impressão legível dos caminhos
# -------------------------------------------------------------
def print_solution(sol, backbone_ids, atom_by_serial, idx=1):
    """
    Imprime um caminho em formato legível.
    """
    print("=" * 80)
    print(f"Caminho #{idx}  |  passos = {len(sol['edges'])}, "
          f"soma distâncias = {sol['total']:.3f} Å")
    print("-" * 80)

    coords = sol["coords"]

    # REF 1
    first_id = backbone_ids[0]
    a0 = atom_by_serial[first_id]
    x0, y0, z0 = coords[first_id]
    print(
        f"REF 1: {a0['line']}\n"
        f"       coords usadas neste caminho: "
        f"({x0:8.3f} {y0:8.3f} {z0:8.3f})"
    )
    print()

    # Cada aresta (ligação backbone)
    for i, edge in enumerate(sol["edges"], start=1):
        from_id = edge["from"]
        to_id = edge["to"]
        donor_id = edge["donor"]
        d = edge["distance"]

        af = atom_by_serial[from_id]
        at = atom_by_serial[to_id]
        xt, yt, zt = coords[to_id]

        print(
            f"PASSO {i:2d}: "
            f"{af['name']:>2s}({from_id:5d}) {af['resname']:>3s} {af['resseq']:4d}  ->  "
            f"{at['name']:>2s}({to_id:5d}) {at['resname']:>3s} {at['resseq']:4d}  "
            f"coord_to=({xt:8.3f} {yt:8.3f} {zt:8.3f})  "
            f"dist={d:6.3f}  donor_id={donor_id:5d}"
        )

    print("=" * 80)
    print()


# -------------------------------------------------------------
# Escrita de novo PDB com coordenadas de um caminho
# -------------------------------------------------------------
def write_pdb_from_solution(input_filename, output_filename, coords):
    """
    Lê o arquivo PDB original e escreve um novo arquivo PDB com as
    coordenadas substituídas pelas coordenadas do dicionário `coords`
    (serial -> (x, y, z)) para ATOM/HETATM.

    As demais linhas (REMARK, TER, etc.) são copiadas intactas.
    """
    with open(input_filename, "r") as fin, open(output_filename, "w") as fout:
        for line in fin:
            rec = line[0:6]
            if rec.strip() in ("ATOM", "HETATM"):
                raw = line.rstrip("\n")
                serial = int(raw[6:11])
                if serial in coords:
                    x, y, z = coords[serial]
                    # Garante que a linha tenha pelo menos 54 colunas
                    if len(raw) < 54:
                        raw = raw.ljust(54)
                    # Substitui colunas 30-54 (X, Y, Z)
                    raw = raw[:30] + f"{x:8.3f}{y:8.3f}{z:8.3f}" + raw[54:]
                    fout.write(raw + "\n")
                else:
                    # Se por algum motivo não estiver em coords, copia original
                    fout.write(raw + "\n")
            else:
                fout.write(line)

    print(f"Novo PDB escrito em: {output_filename}")


# -------------------------------------------------------------
# Função principal
# -------------------------------------------------------------
def main():
    # 1) Carrega PDB
    filename = carregar_pdb_colab()

    # 2) Faz parsing
    atoms, atom_by_serial = parse_pdb_coords(filename)

    # 3) Define backbone canônico entre N(576) [8641] e C(583) [8776]
    START_SERIAL = 8641
    END_SERIAL = 8776

    backbone_ids = build_backbone_order(
        atoms,
        atom_by_serial,
        start_serial=START_SERIAL,
        end_serial=END_SERIAL
    )

    # 4) Explora todos os caminhos com restrições geométricas
    solutions, stats = find_all_backbone_paths(
        atoms,
        atom_by_serial,
        backbone_ids,
        max_steps=23,
        min_total=30.0,
        max_total=40.0
    )

    # 5) Estatísticas globais
    print("\n===== ESTATÍSTICAS GERAIS =====")
    print(f"Total de caminhos válidos (23 passos, 30–40 Å): "
          f"{stats['caminhos_chegaram_alvo_23_entre_30e40A']}")
    print(f"Caminhos que passaram de 40 Å com menos de 23 passos: "
          f"{stats['caminhos_passaram_40A_antes_23']}")
    print(f"Caminhos que chegaram ao alvo com 23 passos mas < 30 Å: "
          f"{stats['caminhos_chegaram_alvo_23_menor_30A']}")
    print(f"Caminhos que estouraram 23 passos (teoricamente 0 aqui): "
          f"{stats['caminhos_estouraram_23_passos']}")
    print(f"Caminhos mortos por falta de candidato em algum passo: "
          f"{stats['caminhos_mortos_sem_candidato']}")

    # 6) Imprime caminhos válidos (se houver) e gera PDBs
    if solutions:
        print("\n===== CAMINHOS VÁLIDOS ENCONTRADOS =====\n")
        for i, sol in enumerate(solutions, start=1):
            print_solution(sol, backbone_ids, atom_by_serial, idx=i)

            # Gera um novo PDB para este caminho
            output_file = f"backbone_rebuilt_path{i}.pdb"
            write_pdb_from_solution(filename, output_file, sol["coords"])

            # Se estiver no Colab, já oferece download
            try:
                from google.colab import files  # type: ignore
                files.download(output_file)
            except ImportError:
                pass
    else:
        print("\nNenhum caminho válido encontrado com as restrições atuais.")


# -------------------------------------------------------------
# Execução
# -------------------------------------------------------------
if __name__ == "__main__":
    main()


## BUSCA E REESTRUTURAÇÃO DOS ÁTOMOS HN, HA E C

In [ ]:
# -*- coding: utf-8 -*-
"""
Script para:
- Ler backbone_rebuilt_path1.pdb (ou outro PDB com backbone já estabilizado).
- Considerar o backbone N–CA–C dos resíduos 576–583 como imutável.
- Para cada N, CA e C do backbone:
    - Encontrar o candidato de coordenada mais próximo para HN, HA e O,
      respeitando:
        * backbone intocado;
        * HN/HA/O já fixados não podem ser mexidos de novo;
        * swaps atualizam a matriz global de coordenadas (uma única matriz).
- Registrar, para cada ligação, um dicionário:
    REFERENCIA, LIGADO_ATUAL, CANDIDATO, DIST
- Escrever um novo PDB com essas coordenadas já trocadas.
"""

import math

# -------------------------------------------------------------
# Upload (Colab) ou caminho manual
# -------------------------------------------------------------
def carregar_pdb_colab():
    """
    Tenta usar o botão de upload do Colab.
    Se não estiver no Colab, cai para input de nome de arquivo.
    """
    try:
        from google.colab import files  # type: ignore
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo enviado.")
        filename = next(iter(uploaded.keys()))
        print(f"PDB carregado: {filename}")
        return filename
    except ImportError:
        filename = input("Informe o caminho do arquivo PDB: ").strip()
        return filename


# -------------------------------------------------------------
# Parsing de PDB
# -------------------------------------------------------------
def parse_pdb_coords(filename):
    """
    Lê um arquivo PDB e retorna:
    - atoms: lista de dicionários com info geométrica
    - atom_by_serial: dict serial -> dict do átomo
    """
    atoms = []
    with open(filename, "r") as f:
        for line in f:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                continue
            serial = int(line[6:11])
            name = line[12:16].strip()
            resname = line[17:20].strip()
            chain = line[21].strip()
            resseq = int(line[22:26])
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])

            atoms.append({
                "serial": serial,
                "name": name,
                "resname": resname,
                "chain": chain,
                "resseq": resseq,
                "x": x,
                "y": y,
                "z": z,
                "line": line.rstrip("\n"),
            })

    atoms.sort(key=lambda a: a["serial"])
    atom_by_serial = {a["serial"]: a for a in atoms}
    print(f"Total de átomos lidos: {len(atoms)}")
    return atoms, atom_by_serial


# -------------------------------------------------------------
# Backbone canônico entre N(576) e C(583)
# -------------------------------------------------------------
def build_backbone_order(atoms, atom_by_serial,
                         start_serial=8641, end_serial=8776):
    """
    Constrói a ordem N–CA–C para os resíduos entre o N inicial (576) e C final (583),
    usando os próprios tipos do PDB.

    Retorna:
      backbone_ids: [N576, CA576, C576, N577, CA577, C577, ..., N583, CA583, C583]
    """
    if start_serial not in atom_by_serial:
        raise ValueError(f"Start_serial {start_serial} não encontrado no PDB.")
    if end_serial not in atom_by_serial:
        raise ValueError(f"End_serial {end_serial} não encontrado no PDB.")

    start_atom = atom_by_serial[start_serial]
    end_atom = atom_by_serial[end_serial]

    chain = start_atom["chain"]
    res_start = start_atom["resseq"]
    res_end = end_atom["resseq"]

    if chain != end_atom["chain"]:
        raise ValueError("Start e end em cadeias diferentes, revise.")

    backbone_ids = []
    for res in range(res_start, res_end + 1):
        for aname in ("N", "CA", "C"):
            cand = [
                a for a in atoms
                if a["chain"] == chain
                and a["resseq"] == res
                and a["name"] == aname
            ]
            if not cand:
                raise ValueError(
                    f"Não encontrei átomo {aname} no resíduo {res} cadeia {chain}."
                )
            backbone_ids.append(cand[0]["serial"])

    if backbone_ids[0] != start_serial:
        raise ValueError("Primeiro átomo do backbone não é o N inicial esperado.")
    if backbone_ids[-1] != end_serial:
        raise ValueError("Último átomo do backbone não é o C final esperado.")

    print("Backbone canônico (serial, nome, res, resseq):")
    for s in backbone_ids:
        a = atom_by_serial[s]
        print(f"  {s:5d}  {a['name']:>3s}  {a['resname']:>3s}  {a['resseq']:4d}")

    return backbone_ids


# -------------------------------------------------------------
# Distância euclidiana
# -------------------------------------------------------------
def dist(p1, p2):
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    dz = p1[2] - p2[2]
    return math.sqrt(dx*dx + dy*dy + dz*dz)


# -------------------------------------------------------------
# Encontrar HN / HA / O "ligado atual" por resíduo
# -------------------------------------------------------------
def find_side_atom_for_residue(atoms, chain, resseq, target_type):
    """
    Encontra, em um resíduo (chain, resseq), qual é o átomo 'ligado atual'
    ao backbone para um dado tipo:

    target_type:
      - 'HN' -> procura exatamente 'HN'
      - 'HA' -> procura exatamente 'HA'
      - 'O'  -> procura exatamente 'O'

    Retorna:
      dicionário do átomo ou None se não encontrar.
    """
    res_atoms = [a for a in atoms if a["chain"] == chain and a["resseq"] == resseq]

    if target_type == "HN":
        cand = [a for a in res_atoms if a["name"] == "HN"]
        return cand[0] if cand else None

    elif target_type == "HA":
        cand = [a for a in res_atoms if a["name"] == "HA"]
        return cand[0] if cand else None

    elif target_type == "O":
        cand = [a for a in res_atoms if a["name"] == "O"]
        return cand[0] if cand else None

    else:
        return None


# -------------------------------------------------------------
# Loop principal: atribuição de HN / HA / O por menor distância
# -------------------------------------------------------------
def assign_hn_ha_o(atoms, atom_by_serial, backbone_ids):
    """
    Implementa a lógica:

    - coords: matriz única de coordenadas (serial -> (x, y, z))
    - backbone_ids: não podem ser usados como candidatos nem sofrer swap.
    - locked_sidechain: átomos que já foram fixados como HN/HA/O (nunca trocam).
    - Para cada N/CA/C do backbone, na ordem:
        * Decide o tipo alvo (HN, HA, O).
        * Acha o 'ligado atual' daquele resíduo (HN/HA/O exatos).
        * Busca o candidato mais próximo (menor distância) entre todos os átomos
          que não sejam backbone e não estejam em locked_sidechain.
        * Registra REF, LIGADO_ATUAL, CANDIDATO, DIST.
        * Faz swap de coordenadas entre 'ligado atual' e 'candidato' (se forem
          diferentes) e marca 'ligado atual' como locked_sidechain.
        * A matriz coords É ATUALIZADA e usada pelo próximo passo.
    """
    # Matriz única de coordenadas
    coords = {a["serial"]: (a["x"], a["y"], a["z"]) for a in atoms}
    all_serials = [a["serial"] for a in atoms]

    backbone_set = set(backbone_ids)
    locked_sidechain = set()  # HN/HA/O já fixados

    ligacoes = []  # lista de dicionários de resultados

    for serial_ref in backbone_ids:
        ref_atom = atom_by_serial[serial_ref]
        ref_name = ref_atom["name"]
        chain = ref_atom["chain"]
        resseq = ref_atom["resseq"]

        # Decide tipo alvo
        if ref_name == "N":
            target_type = "HN"
        elif ref_name == "CA":
            target_type = "HA"
        elif ref_name == "C":
            target_type = "O"
        else:
            # Não deveria acontecer, backbone é só N, CA, C
            continue

        # Encontra o "ligado atual" deste resíduo
        ligado_atual_atom = find_side_atom_for_residue(
            atoms, chain, resseq, target_type
        )
        if ligado_atual_atom is None:
            print(
                f"Aviso: não encontrei átomo {target_type} no resíduo "
                f"{resseq} cadeia {chain}. Pulando."
            )
            continue

        ligado_serial = ligado_atual_atom["serial"]

        # Coordenada de referência (backbone) - imutável
        ref_coord = coords[serial_ref]

        # Busca candidato mais próximo
        best_serial = None
        best_dist = None

        for s in all_serials:
            # Não usar átomo do backbone como candidato
            if s in backbone_set:
                continue
            # Não usar sidechain já fixado (HN/HA/O já definidos)
            if s in locked_sidechain:
                continue

            cand_coord = coords[s]
            d = dist(ref_coord, cand_coord)

            if best_dist is None or d < best_dist:
                best_dist = d
                best_serial = s

        if best_serial is None:
            print(
                f"Aviso: não encontrei candidato para {ref_name} "
                f"no resíduo {resseq} cadeia {chain}."
            )
            continue

        cand_atom = atom_by_serial[best_serial]

        # Antes do swap, registramos as informações (usando coords atuais)
        ref_info = {
            "id_atm": serial_ref,
            "atm": ref_atom["name"],
            "res": ref_atom["resname"],
            "num_res": ref_atom["resseq"],
            "x": ref_coord[0],
            "y": ref_coord[1],
            "z": ref_coord[2],
        }
        ligado_info = {
            "id_atm": ligado_serial,
            "atm": ligado_atual_atom["name"],
            "res": ligado_atual_atom["resname"],
            "num_res": ligado_atual_atom["resseq"],
            "x": coords[ligado_serial][0],
            "y": coords[ligado_serial][1],
            "z": coords[ligado_serial][2],
        }
        cand_info = {
            "id_atm": best_serial,
            "atm": cand_atom["name"],
            "res": cand_atom["resname"],
            "num_res": cand_atom["resseq"],
            "x": coords[best_serial][0],
            "y": coords[best_serial][1],
            "z": coords[best_serial][2],
        }

        ligacoes.append({
            "tipo": target_type,
            "referencia": ref_info,
            "ligado_atual": ligado_info,
            "candidato": cand_info,
            "dist": best_dist,
        })

        # Swap de coordenadas entre LIGADO_ATUAL e CANDIDATO (se forem diferentes)
        # Atualiza a matriz GLOBAL de coords, usada nos próximos passos.
        if best_serial != ligado_serial:
            coord_lig = coords[ligado_serial]
            coord_cand = coords[best_serial]
            coords[ligado_serial] = coord_cand
            coords[best_serial] = coord_lig

        # A partir de agora, esse átomo (ligado_serial) está fixado e nunca mais troca
        locked_sidechain.add(ligado_serial)

    return ligacoes, coords


# -------------------------------------------------------------
# Escrita de novo PDB com coordenadas atualizadas
# -------------------------------------------------------------
def write_pdb_with_coords(input_filename, output_filename, coords):
    """
    Lê o PDB original e escreve um novo PDB usando o dicionário coords
    (serial -> (x, y, z)) para ATOM/HETATM.
    """
    with open(input_filename, "r") as fin, open(output_filename, "w") as fout:
        for line in fin:
            rec = line[0:6]
            if rec.strip() in ("ATOM", "HETATM"):
                raw = line.rstrip("\n")
                serial = int(raw[6:11])
                if serial in coords:
                    x, y, z = coords[serial]
                    if len(raw) < 54:
                        raw = raw.ljust(54)
                    raw = raw[:30] + f"{x:8.3f}{y:8.3f}{z:8.3f}" + raw[54:]
                    fout.write(raw + "\n")
                else:
                    fout.write(raw + "\n")
            else:
                fout.write(line)
    print(f"Novo PDB escrito em: {output_filename}")


# -------------------------------------------------------------
# Impressão das ligações encontradas
# -------------------------------------------------------------
def print_ligacoes(ligacoes):
    """
    Imprime o resumo das ligações REF–(ligado_atual)–(candidato)–dist.
    """
    for i, lig in enumerate(ligacoes, start=1):
        tipo = lig["tipo"]
        ref = lig["referencia"]
        lig_at = lig["ligado_atual"]
        cand = lig["candidato"]
        d = lig["dist"]

        print(f"LIGAÇÃO {i:2d}  (tipo: {tipo})")
        print(
            "  REFERENCIA    : "
            f"{ref['id_atm']:5d}  {ref['atm']:>3s}  {ref['res']:>3s} {ref['num_res']:4d}  "
            f"({ref['x']:7.3f} {ref['y']:7.3f} {ref['z']:7.3f})"
        )
        print(
            "  LIGADO_ATUAL  : "
            f"{lig_at['id_atm']:5d}  {lig_at['atm']:>3s}  {lig_at['res']:>3s} {lig_at['num_res']:4d}  "
            f"({lig_at['x']:7.3f} {lig_at['y']:7.3f} {lig_at['z']:7.3f})"
        )
        print(
            "  CANDIDATO     : "
            f"{cand['id_atm']:5d}  {cand['atm']:>3s}  {cand['res']:>3s} {cand['num_res']:4d}  "
            f"({cand['x']:7.3f} {cand['y']:7.3f} {cand['z']:7.3f})"
        )
        print(f"  DIST          : {d:7.3f} Å")
        print()


# -------------------------------------------------------------
# Função principal
# -------------------------------------------------------------
def main():
    # 1) Carrega PDB (por exemplo, backbone_rebuilt_path1.pdb)
    filename = carregar_pdb_colab()

    # 2) Parse
    atoms, atom_by_serial = parse_pdb_coords(filename)

    # 3) Backbone imutável N576–C583
    START_SERIAL = 8641  # N ILE 576
    END_SERIAL = 8776    # C GLY 583

    backbone_ids = build_backbone_order(
        atoms,
        atom_by_serial,
        start_serial=START_SERIAL,
        end_serial=END_SERIAL
    )

    # 4) Atribui HN / HA / O por menor distância com swaps na matriz única
    ligacoes, coords_final = assign_hn_ha_o(atoms, atom_by_serial, backbone_ids)

    # 5) Imprime as ligações encontradas
    print("\n===== LIGAÇÕES HN / HA / O ENCONTRADAS =====\n")
    print_ligacoes(ligacoes)

    # 6) Escreve novo PDB com as coordenadas atualizadas
    output_pdb = "backbone_rebuilt_with_HNO.pdb"
    write_pdb_with_coords(filename, output_pdb, coords_final)

    # 7) Em Colab, disponibiliza download
    try:
        from google.colab import files  # type: ignore
        files.download(output_pdb)
    except ImportError:
        pass


# -------------------------------------------------------------
# Execução
# -------------------------------------------------------------
if __name__ == "__main__":
    main()


Saving backbone_rebuilt_path1.pdb to backbone_rebuilt_path1.pdb
PDB carregado: backbone_rebuilt_path1.pdb
Total de átomos lidos: 137
Backbone canônico (serial, nome, res, resseq):
   8641    N  ILE   576
   8643   CA  ILE   576
   8658    C  ILE   576
   8660    N  THR   577
   8662   CA  THR   577
   8672    C  THR   577
   8674    N  LEU   578
   8676   CA  LEU   578
   8691    C  LEU   578
   8693    N  TYR   579
   8695   CA  TYR   579
   8712    C  TYR   579
   8714    N  CYS   580
   8716   CA  CYS   580
   8723    C  CYS   580
   8725    N  LYS   581
   8727   CA  LYS   581
   8745    C  LYS   581
   8747    N  ARG   582
   8749   CA  ARG   582
   8769    C  ARG   582
   8771    N  GLY   583
   8773   CA  GLY   583
   8776    C  GLY   583
Aviso: não encontrei átomo HA no resíduo 583 cadeia B. Pulando.

===== LIGAÇÕES HN / HA / O ENCONTRADAS =====

LIGAÇÃO  1  (tipo: HN)
  REFERENCIA    :  8641    N  ILE  576  ( 74.770  70.720 169.110)
  LIGADO_ATUAL  :  8642   HN  ILE  576  ( 74

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## BUSCA E REORDENAÇÃO DO SIDE CHAIN

### CONEXAO CA-CB

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução dos átomos CB de cada resíduo 576–583
a partir de um PDB com backbone (N, CA, C) e HN/HA/O já estabilizados.

Regras:
- Backbone N, CA, C: imutáveis.
- HN, HA, O dos resíduos 576–583: imutáveis.
- Para cada CA de cada resíduo (exceto GLY):
    * procurar candidatos de CB por distância:
        1.40 Å <= CA–candidato <= 1.60 Å
    * se houver >1 candidato:
        - filtrar por ângulos:
          N–CA–CAND em [104°, 116°]
          C–CA–CAND em [105°, 118°]
        - se ainda >1:
          escolher aquele que minimiza
            |θ(N–CA–CAND) - 110| + |θ(C–CA–CAND) - 111|
    * trocar coordenadas entre o CB do resíduo e o candidato escolhido
      e marcar o CB como fixo.
- Matriz de coordenadas única e atualizada a cada swap.
"""

import math

# -------------------------------------------------------------
# Upload (Colab) ou caminho manual
# -------------------------------------------------------------
def carregar_pdb_colab():
    try:
        from google.colab import files  # type: ignore
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo enviado.")
        filename = next(iter(uploaded.keys()))
        print(f"PDB carregado: {filename}")
        return filename
    except ImportError:
        filename = input("Informe o caminho do arquivo PDB: ").strip()
        return filename


# -------------------------------------------------------------
# Parsing de PDB
# -------------------------------------------------------------
def parse_pdb_coords(filename):
    atoms = []
    with open(filename, "r") as f:
        for line in f:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                continue
            serial = int(line[6:11])
            name = line[12:16].strip()
            resname = line[17:20].strip()
            chain = line[21].strip()
            resseq = int(line[22:26])
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])

            atoms.append({
                "serial": serial,
                "name": name,
                "resname": resname,
                "chain": chain,
                "resseq": resseq,
                "x": x,
                "y": y,
                "z": z,
                "line": line.rstrip("\n"),
            })

    atoms.sort(key=lambda a: a["serial"])
    atom_by_serial = {a["serial"]: a for a in atoms}
    print(f"Total de átomos lidos: {len(atoms)}")
    return atoms, atom_by_serial


# -------------------------------------------------------------
# Backbone canônico entre N(576) e C(583)
# -------------------------------------------------------------
def build_backbone_order(atoms, atom_by_serial,
                         start_serial=8641, end_serial=8776):
    if start_serial not in atom_by_serial:
        raise ValueError(f"Start_serial {start_serial} não encontrado no PDB.")
    if end_serial not in atom_by_serial:
        raise ValueError(f"End_serial {end_serial} não encontrado no PDB.")

    start_atom = atom_by_serial[start_serial]
    end_atom = atom_by_serial[end_serial]

    chain = start_atom["chain"]
    res_start = start_atom["resseq"]
    res_end = end_atom["resseq"]

    if chain != end_atom["chain"]:
        raise ValueError("Start e end em cadeias diferentes, revise.")

    backbone_ids = []
    for res in range(res_start, res_end + 1):
        for aname in ("N", "CA", "C"):
            cand = [
                a for a in atoms
                if a["chain"] == chain
                and a["resseq"] == res
                and a["name"] == aname
            ]
            if not cand:
                raise ValueError(
                    f"Não encontrei átomo {aname} no resíduo {res} cadeia {chain}."
                )
            backbone_ids.append(cand[0]["serial"])

    if backbone_ids[0] != start_serial:
        raise ValueError("Primeiro átomo do backbone não é o N inicial esperado.")
    if backbone_ids[-1] != end_serial:
        raise ValueError("Último átomo do backbone não é o C final esperado.")

    print("Backbone canônico (serial, nome, res, resseq):")
    for s in backbone_ids:
        a = atom_by_serial[s]
        print(f"  {s:5d}  {a['name']:>3s}  {a['resname']:>3s}  {a['resseq']:4d}")

    return backbone_ids


# -------------------------------------------------------------
# Distância e ângulo
# -------------------------------------------------------------
def dist(p1, p2):
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    dz = p1[2] - p2[2]
    return math.sqrt(dx*dx + dy*dy + dz*dz)


def angle_deg(p1, p2, p3):
    """Ângulo em graus em p2 formado por p1-p2-p3."""
    v1 = (p1[0]-p2[0], p1[1]-p2[1], p1[2]-p2[2])
    v2 = (p3[0]-p2[0], p3[1]-p2[1], p3[2]-p2[2])
    n1 = math.sqrt(v1[0]**2 + v1[1]**2 + v1[2]**2)
    n2 = math.sqrt(v2[0]**2 + v2[1]**2 + v2[2]**2)
    if n1 < 1e-8 or n2 < 1e-8:
        return None
    dot = v1[0]*v2[0] + v1[1]*v2[1] + v1[2]*v2[2]
    cosang = dot / (n1*n2)
    cosang = max(-1.0, min(1.0, cosang))
    return math.degrees(math.acos(cosang))


# -------------------------------------------------------------
# Encontrar HN / HA / O dos resíduos 576–583 (já estabilizados)
# -------------------------------------------------------------
def find_side_atom_for_residue(atoms, chain, resseq, target_name):
    res_atoms = [a for a in atoms if a["chain"] == chain and a["resseq"] == resseq]
    cand = [a for a in res_atoms if a["name"] == target_name]
    return cand[0] if cand else None


def get_backbone_residues(backbone_ids, atom_by_serial):
    """
    Constrói uma lista de resíduos:
      [
        {
          'resseq': int,
          'resname': str,
          'chain': str,
          'N': serial,
          'CA': serial,
          'C': serial
        },
        ...
      ]
    assumindo que backbone_ids está em blocos N, CA, C.
    """
    residues = []
    if len(backbone_ids) % 3 != 0:
        raise ValueError("backbone_ids não está múltiplo de 3 (N,CA,C).")

    for i in range(0, len(backbone_ids), 3):
        N_id, CA_id, C_id = backbone_ids[i:i+3]
        N_atom = atom_by_serial[N_id]
        CA_atom = atom_by_serial[CA_id]
        C_atom = atom_by_serial[C_id]

        if not (N_atom["resseq"] == CA_atom["resseq"] == C_atom["resseq"] and
                N_atom["chain"] == CA_atom["chain"] == C_atom["chain"]):
            raise ValueError("Inconsistência N/CA/C no mesmo resíduo.")

        residues.append({
            "resseq": CA_atom["resseq"],
            "resname": CA_atom["resname"],
            "chain": CA_atom["chain"],
            "N": N_id,
            "CA": CA_id,
            "C": C_id,
        })

    return residues


# -------------------------------------------------------------
# Atribuir CBs por distância + ângulo
# -------------------------------------------------------------
def assign_cb(atoms, atom_by_serial, residues, locked_initial):
    """
    Matriz única de coordenadas (coords).
    locked_initial: conjunto com backbone + HN/HA/O que NÃO podem ser candidatos.

    Para cada resíduo em 'residues':
      - pula GLY (não tem CB).
      - identifica CB 'ligado atual' (nome 'CB') do resíduo.
      - busca candidatos (serial não em locked, distância CA–candidato entre 1.40 e 1.60).
      - se >1 candidato:
          * calcula ângulos N–CA–candidato e C–CA–candidato;
          * filtra com:
              104° <= N–CA–CB <= 116°
              105° <= C–CA–CB <= 118°
          * se ainda >1:
              escolhe o que minimiza |θ_N-110| + |θ_C-111|.
      - faz swap de coordenadas entre CB_atual e candidato escolhido;
      - adiciona CB_atual a locked (torna-se imutável).
    """
    coords = {a["serial"]: (a["x"], a["y"], a["z"]) for a in atoms}
    all_serials = [a["serial"] for a in atoms]

    locked = set(locked_initial)

    ligacoes = []

    for res in residues:
        resname = res["resname"]
        resseq = res["resseq"]
        chain = res["chain"]

        # Gly não tem CB
        if resname == "GLY":
            continue

        N_id = res["N"]
        CA_id = res["CA"]
        C_id = res["C"]

        # CB atual
        cb_atom = find_side_atom_for_residue(atoms, chain, resseq, "CB")
        if cb_atom is None:
            print(f"Aviso: resíduo {resname} {chain} {resseq} não tem CB no PDB. Pulando.")
            continue
        CB_id = cb_atom["serial"]

        CA_coord = coords[CA_id]
        N_coord = coords[N_id]
        C_coord = coords[C_id]

        # 1) candidatos por distância
        candidates = []
        for s in all_serials:
            if s in locked:
                continue
            # pode ser o próprio CB também (se ainda não está locked)
            cand_coord = coords[s]
            d = dist(CA_coord, cand_coord)
            if 1.40 <= d <= 1.60:
                candidates.append((s, d))

        if not candidates:
            print(
                f"Aviso: não encontrei candidatos de CB por distância para "
                f"{resname} {chain} {resseq}. Pulando."
            )
            continue

        # 2) se mais de um, aplicar filtros de ângulo
        if len(candidates) > 1:
            enriched = []
            for s, d in candidates:
                cand_coord = coords[s]
                theta_N = angle_deg(N_coord, CA_coord, cand_coord)
                theta_C = angle_deg(C_coord, CA_coord, cand_coord)
                # se algo der None (vetor degenerado), ignorar esse candidato
                if theta_N is None or theta_C is None:
                    continue
                enriched.append((s, d, theta_N, theta_C))

            if not enriched:
                # fallback: se todos degeneraram, usa só distancia
                chosen_serial, chosen_dist = min(candidates, key=lambda x: x[1])
                theta_N = None
                theta_C = None
            else:
                # filtra por janelas de ângulo
                filtered = [
                    (s, d, tN, tC)
                    for (s, d, tN, tC) in enriched
                    if 104.0 <= tN <= 116.0 and 105.0 <= tC <= 118.0
                ]
                if filtered:
                    working = filtered
                else:
                    # se nenhum satisfaz as janelas, usa todos, escolhendo o melhor pela proximidade
                    working = enriched

                def score(item):
                    _, _, tN, tC = item
                    return abs(tN - 110.0) + abs(tC - 111.0)

                chosen_s, chosen_dist, theta_N, theta_C = min(working, key=score)
                chosen_serial = chosen_s
        else:
            # só um candidato
            chosen_serial, chosen_dist = candidates[0]
            # calcular ângulos só para registrar
            cand_coord = coords[chosen_serial]
            theta_N = angle_deg(N_coord, CA_coord, cand_coord)
            theta_C = angle_deg(C_coord, CA_coord, cand_coord)

        cand_atom = atom_by_serial[chosen_serial]

        # Registro antes do swap (coords atuais)
        ref_info = {
            "id_atm": CA_id,
            "atm": atom_by_serial[CA_id]["name"],
            "res": atom_by_serial[CA_id]["resname"],
            "num_res": atom_by_serial[CA_id]["resseq"],
            "x": CA_coord[0],
            "y": CA_coord[1],
            "z": CA_coord[2],
        }
        cb_info = {
            "id_atm": CB_id,
            "atm": cb_atom["name"],
            "res": cb_atom["resname"],
            "num_res": cb_atom["resseq"],
            "x": coords[CB_id][0],
            "y": coords[CB_id][1],
            "z": coords[CB_id][2],
        }
        cand_info = {
            "id_atm": chosen_serial,
            "atm": cand_atom["name"],
            "res": cand_atom["resname"],
            "num_res": cand_atom["resseq"],
            "x": coords[chosen_serial][0],
            "y": coords[chosen_serial][1],
            "z": coords[chosen_serial][2],
        }

        ligacoes.append({
            "resname": resname,
            "chain": chain,
            "resseq": resseq,
            "referencia": ref_info,
            "cb_atual": cb_info,
            "candidato": cand_info,
            "dist": chosen_dist,
            "theta_NCA_CB": theta_N,
            "theta_CCA_CB": theta_C,
        })

        # 3) swap de coordenadas CA–CB? Não: quem troca é CB_atual x candidato.
        if chosen_serial != CB_id:
            coord_cb = coords[CB_id]
            coord_cand = coords[chosen_serial]
            coords[CB_id] = coord_cand
            coords[chosen_serial] = coord_cb

        # 4) CB agora é imutável
        locked.add(CB_id)

    return ligacoes, coords


# -------------------------------------------------------------
# Escrita de PDB com novas coordenadas
# -------------------------------------------------------------
def write_pdb_with_coords(input_filename, output_filename, coords):
    with open(input_filename, "r") as fin, open(output_filename, "w") as fout:
        for line in fin:
            rec = line[0:6]
            if rec.strip() in ("ATOM", "HETATM"):
                raw = line.rstrip("\n")
                serial = int(raw[6:11])
                if serial in coords:
                    x, y, z = coords[serial]
                    if len(raw) < 54:
                        raw = raw.ljust(54)
                    raw = raw[:30] + f"{x:8.3f}{y:8.3f}{z:8.3f}" + raw[54:]
                    fout.write(raw + "\n")
                else:
                    fout.write(raw + "\n")
            else:
                fout.write(line)
    print(f"Novo PDB escrito em: {output_filename}")


# -------------------------------------------------------------
# Impressão das ligações
# -------------------------------------------------------------
def print_ligacoes_cb(ligacoes):
    for i, lig in enumerate(ligacoes, start=1):
        ref = lig["referencia"]
        cb = lig["cb_atual"]
        cand = lig["candidato"]
        d = lig["dist"]
        tN = lig["theta_NCA_CB"]
        tC = lig["theta_CCA_CB"]

        print(f"LIGAÇÃO CB {i:2d}  ({lig['resname']} {lig['chain']} {lig['resseq']})")
        print("  CA (REF)      : "
              f"{ref['id_atm']:5d}  {ref['atm']:>3s}  {ref['res']:>3s} {ref['num_res']:4d}  "
              f"({ref['x']:7.3f} {ref['y']:7.3f} {ref['z']:7.3f})")
        print("  CB ATUAL      : "
              f"{cb['id_atm']:5d}  {cb['atm']:>3s}  {cb['res']:>3s} {cb['num_res']:4d}  "
              f"({cb['x']:7.3f} {cb['y']:7.3f} {cb['z']:7.3f})")
        print("  CANDIDATO     : "
              f"{cand['id_atm']:5d}  {cand['atm']:>3s}  {cand['res']:>3s} {cand['num_res']:4d}  "
              f"({cand['x']:7.3f} {cand['y']:7.3f} {cand['z']:7.3f})")
        print(f"  DIST CA–CB    : {d:7.3f} Å")
        if tN is not None and tC is not None:
            print(f"  ÂNG N-CA-CB   : {tN:7.3f} °")
            print(f"  ÂNG C-CA-CB   : {tC:7.3f} °")
        print()


# -------------------------------------------------------------
# Função principal
# -------------------------------------------------------------
def main():
    # 1) Carrega PDB (por ex.: backbone_rebuilt_with_HNO.pdb)
    filename = carregar_pdb_colab()

    # 2) Parse
    atoms, atom_by_serial = parse_pdb_coords(filename)

    # 3) Backbone N576–C583
    START_SERIAL = 8641  # N ILE 576
    END_SERIAL = 8776    # C GLY 583

    backbone_ids = build_backbone_order(
        atoms,
        atom_by_serial,
        start_serial=START_SERIAL,
        end_serial=END_SERIAL
    )

    # 4) Construir lista de resíduos do backbone
    residues = get_backbone_residues(backbone_ids, atom_by_serial)

    # 5) Construir conjunto locked inicial (backbone + HN/HA/O de 576–583)
    locked_initial = set(backbone_ids)

    for res in residues:
        chain = res["chain"]
        resseq = res["resseq"]
        # HN
        hn = find_side_atom_for_residue(atoms, chain, resseq, "HN")
        if hn is not None:
            locked_initial.add(hn["serial"])
        # HA
        ha = find_side_atom_for_residue(atoms, chain, resseq, "HA")
        if ha is not None:
            locked_initial.add(ha["serial"])
        # O
        o_at = find_side_atom_for_residue(atoms, chain, resseq, "O")
        if o_at is not None:
            locked_initial.add(o_at["serial"])

    # 6) Atribuir CBs
    ligacoes_cb, coords_final = assign_cb(atoms, atom_by_serial, residues, locked_initial)

    print("\n===== LIGAÇÕES CB ENCONTRADAS =====\n")
    print_ligacoes_cb(ligacoes_cb)

    # 7) Escrever novo PDB
    output_pdb = "backbone_rebuilt_with_HNO_CB.pdb"
    write_pdb_with_coords(filename, output_pdb, coords_final)

    # 8) Em Colab, disponibiliza download
    try:
        from google.colab import files  # type: ignore
        files.download(output_pdb)
    except ImportError:
        pass


# -------------------------------------------------------------
# Execução
# -------------------------------------------------------------
if __name__ == "__main__":
    main()


Saving backbone_rebuilt_with_HNO.pdb to backbone_rebuilt_with_HNO.pdb
PDB carregado: backbone_rebuilt_with_HNO.pdb
Total de átomos lidos: 137
Backbone canônico (serial, nome, res, resseq):
   8641    N  ILE   576
   8643   CA  ILE   576
   8658    C  ILE   576
   8660    N  THR   577
   8662   CA  THR   577
   8672    C  THR   577
   8674    N  LEU   578
   8676   CA  LEU   578
   8691    C  LEU   578
   8693    N  TYR   579
   8695   CA  TYR   579
   8712    C  TYR   579
   8714    N  CYS   580
   8716   CA  CYS   580
   8723    C  CYS   580
   8725    N  LYS   581
   8727   CA  LYS   581
   8745    C  LYS   581
   8747    N  ARG   582
   8749   CA  ARG   582
   8769    C  ARG   582
   8771    N  GLY   583
   8773   CA  GLY   583
   8776    C  GLY   583

===== LIGAÇÕES CB ENCONTRADAS =====

LIGAÇÃO CB  1  (ILE B 576)
  CA (REF)      :  8643   CA  ILE  576  ( 75.490  71.380 170.140)
  CB ATUAL      :  8645   CB  ILE  576  ( 76.130  72.700 169.650)
  CANDIDATO     :  8645   CB  ILE  576

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### REESTRUTURA 576

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução da cadeia lateral de ILE 576 (CG2, 1/2/3HG2, CG1, 1/2HG1, CD, HD1/2/3)
a partir de um PDB já com backbone + HN + HA + O + CB estabilizados.

Fluxo:
  1) Ler PDB (upload no Colab).
  2) Construir matriz global de coordenadas.
  3) Marcar como imutáveis todos:
       - N, CA, C (backbone)
       - HN, HA, O (backbone)
       - CB (todas as cadeias laterais)
  4) No resíduo 576 (ILE, cadeia B) reconstruir:
       - CG2 via CB–CG2 (distância + ângulo CA–CB–CG2)
       - 1HG2 / 2HG2 / 3HG2 via CG2–H (faixa dinâmica até 3 vizinhos)
       - CG1 via CB–CG1 (distância + ângulo CA–CB–CG1)
       - 1HG1 / 2HG1 / CD via CG1–X (faixa dinâmica até 3 vizinhos)
       - HD1 / HD2 / HD3 via CD–H (faixa dinâmica até 3 vizinhos)
  5) Cada átomo validado entra no conjunto de “locked” (imutáveis).
  6) Trocas de coordenadas são feitas sempre via swap entre dois átomos.
  7) Ao final, grava um novo PDB e imprime um relatório detalhado.
"""

import math

# ===========================
# Funções geométricas básicas
# ===========================

def dist(a, b):
    return math.sqrt((a['x'] - b['x'])**2 +
                     (a['y'] - b['y'])**2 +
                     (a['z'] - b['z'])**2)

def angle(a, b, c):
    """
    Ângulo A-B-C (v1 = A-B, v2 = C-B) em graus.
    """
    v1 = (a['x'] - b['x'], a['y'] - b['y'], a['z'] - b['z'])
    v2 = (c['x'] - b['x'], c['y'] - b['y'], c['z'] - b['z'])
    norm1 = math.sqrt(sum(v*v for v in v1))
    norm2 = math.sqrt(sum(v*v for v in v2))
    if norm1 < 1e-6 or norm2 < 1e-6:
        return 0.0
    dot = sum(v1[i]*v2[i] for i in range(3))
    cosang = max(-1.0, min(1.0, dot/(norm1*norm2)))
    return math.degrees(math.acos(cosang))

def same_coords(a, b, tol=1e-3):
    return (abs(a['x'] - b['x']) < tol and
            abs(a['y'] - b['y']) < tol and
            abs(a['z'] - b['z']) < tol)

# ===========================
# Parsing e escrita de PDB
# ===========================

def parse_pdb_lines(lines):
    atoms = []
    atom_line_idx = []
    for i, line in enumerate(lines):
        if line.startswith("ATOM") or line.startswith("HETATM"):
            try:
                rec = {
                    'line_idx': i,
                    'serial': int(line[6:11]),
                    'name':   line[12:16].strip(),
                    'altloc': line[16],
                    'resname': line[17:20].strip(),
                    'chain':  line[21].strip(),
                    'resseq': int(line[22:26]),
                    'icode':  line[26],
                    'x': float(line[30:38]),
                    'y': float(line[38:46]),
                    'z': float(line[46:54]),
                    'rest': line  # guardamos a linha crua para reescrever depois
                }
            except Exception:
                # linha mal formatada é ignorada
                continue
            atoms.append(rec)
            atom_line_idx.append(i)
    return atoms, atom_line_idx

def update_pdb_lines(lines, atoms):
    """
    Atualiza as coordenadas (colunas 31-54) das linhas ATOM/HETATM
    com base em atoms[*]['x','y','z'] mantendo o restante da linha.
    """
    for at in atoms:
        i = at['line_idx']
        line = lines[i]
        # Garante que a linha tenha pelo menos 54 colunas
        if len(line) < 54:
            line = line.ljust(54)
        new_line = (line[:30] +
                    f"{at['x']:8.3f}{at['y']:8.3f}{at['z']:8.3f}" +
                    line[54:])
        lines[i] = new_line
    return lines

# ===========================
# Utilitários de busca
# ===========================

def find_residue_atoms(atoms, resname, resseq, chain=None):
    idxs = []
    for i, a in enumerate(atoms):
        if a['resname'] == resname and a['resseq'] == resseq:
            if chain is None or a['chain'] == chain:
                idxs.append(i)
    return idxs

def build_name_index(atoms, residue_idxs):
    """
    Constrói um dict name -> idx (dentro de atoms) para 1 resíduo.
    """
    name2idx = {}
    for i in residue_idxs:
        nm = atoms[i]['name']
        if nm not in name2idx:
            name2idx[nm] = i
        else:
            # Se houver duplicata de nome no mesmo resíduo, mantemos o primeiro.
            pass
    return name2idx

def swap_coords(a, b):
    for coord in ('x', 'y', 'z'):
        a[coord], b[coord] = b[coord], a[coord]

# ===========================
# Seleção de CGx (CG2 / CG1)
# ===========================

def escolher_cg(atoms, locked, idx_cb, idx_ca,
                dmin=1.40, dmax=1.65,
                ang1_min=101.0, ang1_max=119.0,
                ang2_min=104.0, ang2_max=116.0,
                alvo_angulo=110.0,
                label="CG?"):
    """
    Escolhe um candidato CGx para CB de ILE com restrição:
      - distância CB–CGx em [dmin, dmax]
      - refino por ângulo CA–CB–CGx (faixas angulares)
    """
    ref_cb = atoms[idx_cb]
    ref_ca = atoms[idx_ca]

    candidatos = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_cb:
            continue
        d = dist(ref_cb, a)
        if dmin <= d <= dmax:
            ang = angle(ref_ca, ref_cb, a)
            candidatos.append((i, d, ang))

    if not candidatos:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} com CB em {dmin:.2f}-{dmax:.2f} Å.")

    # Primeiro filtro angular amplo
    cand1 = [(i, d, ang) for (i, d, ang) in candidatos if ang1_min <= ang <= ang1_max]
    if len(cand1) == 1:
        return cand1[0]

    if len(cand1) > 1:
        # Segundo filtro mais estreito
        cand2 = [(i, d, ang) for (i, d, ang) in cand1 if ang2_min <= ang <= ang2_max]
        if len(cand2) == 1:
            return cand2[0]
        if len(cand2) > 1:
            # Escolhe angulo mais próximo de alvo_angulo
            cand2.sort(key=lambda t: abs(t[2] - alvo_angulo))
            return cand2[0]
        # Nenhum em cand2, escolhe de cand1 o mais próximo do alvo
        cand1.sort(key=lambda t: abs(t[2] - alvo_angulo))
        return cand1[0]

    # Se cand1 ficou vazio, escolhe de todos os candidatos o mais próximo do alvo_angulo
    candidatos.sort(key=lambda t: abs(t[2] - alvo_angulo))
    return candidatos[0]

# ===========================
# Seleção genérica de 3 vizinhos
# ===========================

def escolher_3_vizinhos(atoms, locked, idx_ref,
                        dmin_init=0.990, dmax_init=1.150,
                        delta=0.005, max_iter=200,
                        label="H?"):
    """
    A partir de um átomo referência (idx_ref), encontra exatamente 3 vizinhos
    em torno do raio [dmin, dmax], ajustando a janela se necessário:

    - Se >3 candidatos: estreita a janela (dmin += delta, dmax -= delta).
    - Se <3 candidatos: alarga a janela (dmin -= delta, dmax += delta).

    Se após max_iter ainda não houver 3, escolhe os 3 mais próximos.
    Retorna lista de índices dos 3 vizinhos e um resumo da janela final.
    """
    ref = atoms[idx_ref]
    dmin = dmin_init
    dmax = dmax_init

    for it in range(max_iter):
        cand = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_ref:
                continue
            d = dist(ref, a)
            if dmin <= d <= dmax:
                cand.append((i, d))
        if len(cand) == 3:
            return [i for (i, _) in cand], dmin, dmax
        elif len(cand) > 3:
            dmin += delta
            dmax -= delta
            if dmin >= dmax:
                break
        else:  # len(cand) < 3
            dmin = max(0.0, dmin - delta)
            dmax += delta

    # Fallback: pega os 3 mais próximos entre os átomos livres
    cand_all = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_ref:
            continue
        d = dist(ref, a)
        cand_all.append((i, d))
    cand_all.sort(key=lambda t: t[1])
    chosen = cand_all[:3]
    return [i for (i, _) in chosen], dmin, dmax

def atribuir_coord_3_alvos(atoms, locked, idx_ref, alvo_idxs,
                           dmin_init=0.990, dmax_init=1.150,
                           delta=0.005, max_iter=200,
                           label="H?"):
    """
    Usa escolher_3_vizinhos para achar 3 candidatos em torno de idx_ref
    e distribui suas coordenadas entre os 3 átomos alvo (alvo_idxs).

    Regras:
      - Se a coordenada atual de um alvo já coincide com alguma coordenada candidata,
        mantemos e marcamos o alvo como locked (não há swap).
      - Para os alvos restantes, fazemos swap de coordenadas com candidatos restantes
        (um candidato por alvo).
      - Em todos os casos, o alvo é marcado como locked ao final.

    Retorna um dicionário de detalhes.
    """
    ref = atoms[idx_ref]
    cand_idxs, dmin_final, dmax_final = escolher_3_vizinhos(
        atoms, locked, idx_ref,
        dmin_init=dmin_init, dmax_init=dmax_init,
        delta=delta, max_iter=max_iter, label=label
    )
    cand_coords = {i: (atoms[i]['x'], atoms[i]['y'], atoms[i]['z']) for i in cand_idxs}
    used_cands = set()
    detalhes = {
        'ref': ref,
        'ref_idx': idx_ref,
        'janela_final': (dmin_final, dmax_final),
        'mapeamentos': []  # lista de dicts com info para cada alvo
    }

    # 1) Primeiro, tenta casar alvos cuja coord já coincide com alguma candidata
    for idx_alvo in alvo_idxs:
        alvo = atoms[idx_alvo]
        match = None
        for i_cand in cand_idxs:
            if i_cand in used_cands:
                continue
            cx, cy, cz = cand_coords[i_cand]
            if (abs(alvo['x'] - cx) < 1e-3 and
                abs(alvo['y'] - cy) < 1e-3 and
                abs(alvo['z'] - cz) < 1e-3):
                match = i_cand
                break
        if match is not None:
            # Já está na coordenada "boa" -> só marca como locked
            used_cands.add(match)
            locked.add(idx_alvo)
            d = dist(ref, alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': match,
                'cand': atoms[match],
                'dist_ref_alvo': d,
                'swap_feito': False
            })

    # 2) Agora lida com os alvos restantes, fazendo swaps se necessário
    for idx_alvo in alvo_idxs:
        if idx_alvo in locked:
            # já tratado acima
            continue
        alvo = atoms[idx_alvo]
        # escolhe um candidato ainda não usado
        cand_rest = [i for i in cand_idxs if i not in used_cands]
        if not cand_rest:
            # não deveria acontecer, mas se acontecer, apenas marca locked
            locked.add(idx_alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': None,
                'cand': None,
                'dist_ref_alvo': dist(ref, alvo),
                'swap_feito': False
            })
            continue
        # escolhe o candidato mais próximo do ref
        cand_rest.sort(key=lambda i: dist(ref, atoms[i]))
        i_cand = cand_rest[0]
        used_cands.add(i_cand)
        cand_atom = atoms[i_cand]

        # faz swap se as coordenadas forem diferentes
        d_before = dist(ref, alvo)
        if not same_coords(alvo, cand_atom):
            swap_coords(alvo, cand_atom)
            swap_feito = True
        else:
            swap_feito = False

        locked.add(idx_alvo)
        d_after = dist(ref, alvo)
        detalhes['mapeamentos'].append({
            'alvo_idx': idx_alvo,
            'alvo': alvo,
            'cand_idx': i_cand,
            'cand': cand_atom,
            'dist_ref_alvo_antes': d_before,
            'dist_ref_alvo_depois': d_after,
            'swap_feito': swap_feito
        })

    return detalhes

# ===========================
# Pipeline principal
# ===========================

def reconstruir_ile576_sidechain(pdb_filename, saida_filename="backbone_rebuilt_with_HNO_CB_ILE576.pdb",
                                 resname="ILE", resseq=576, chain=None):
    """
    Lê um PDB com backbone+HN+HA+O+CB estabilizados, reconstrói a cadeia lateral
    de ILE 576 e escreve um novo PDB.
    """
    with open(pdb_filename, 'r') as f:
        lines = f.read().splitlines()

    atoms, atom_line_idx = parse_pdb_lines(lines)
    n_atoms = len(atoms)
    print(f"PDB carregado: {pdb_filename}")
    print(f"Total de átomos lidos: {n_atoms}")

    # Conjunto de átomos imutáveis iniciais (backbone + HN + HA + O + CB)
    locked = set()
    nomes_locked_iniciais = {"N", "CA", "C", "HN", "HA", "O", "CB"}
    for i, a in enumerate(atoms):
        if a['name'] in nomes_locked_iniciais:
            locked.add(i)

    print(f"Átomos inicializados como imutáveis (locked): {len(locked)}")

    # Identifica os átomos do resíduo alvo
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    if not res_idxs:
        raise RuntimeError(f"Resíduo {resname} {resseq} não encontrado no PDB.")

    # Se chain for None, deduz a cadeia a partir do primeiro átomo do resíduo
    if chain is None:
        chain = atoms[res_idxs[0]]['chain']

    print(f"Reconstruindo cadeia lateral de {resname} {chain} {resseq}")
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    name2idx = build_name_index(atoms, res_idxs)

    obrigatorios = ["N", "CA", "C", "CB", "CG2", "CG1", "CD",
                    "1HG2", "2HG2", "3HG2",
                    "1HG1", "2HG1",
                    "HD1", "HD2", "HD3"]
    for nm in obrigatorios:
        if nm not in name2idx:
            print(f"AVISO: átomo {nm} não encontrado em {resname} {chain} {resseq}.")

    idx_N   = name2idx.get("N")
    idx_CA  = name2idx.get("CA")
    idx_C   = name2idx.get("C")
    idx_CB  = name2idx.get("CB")
    idx_CG2 = name2idx.get("CG2")
    idx_CG1 = name2idx.get("CG1")
    idx_CD  = name2idx.get("CD")

    # ===========================
    # 1) CB–CG2
    # ===========================
    print("\n===== ETAPA 1: CB–CG2 =====")
    cand_idx, d_cg2, ang_cg2 = escolher_cg(
        atoms, locked, idx_cb=idx_CB, idx_ca=idx_CA,
        dmin=1.40, dmax=1.65,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="CG2"
    )
    print(f"CG2 candidato: serial={atoms[cand_idx]['serial']} "
          f"{atoms[cand_idx]['name']} {atoms[cand_idx]['resname']} {atoms[cand_idx]['resseq']}")
    print(f"  Distância CB–candidato = {d_cg2:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_cg2:.3f} °")

    at_cg2 = atoms[idx_CG2]
    if not same_coords(at_cg2, atoms[cand_idx]):
        print("  -> Fazendo swap de coordenadas entre CG2 e candidato.")
        swap_coords(at_cg2, atoms[cand_idx])
    else:
        print("  -> CG2 já está usando essa coordenada (nenhum swap necessário).")
    locked.add(idx_CG2)

    # ===========================
    # 2) CG2–(1HG2,2HG2,3HG2)
    # ===========================
    print("\n===== ETAPA 2: CG2–HG2 (1HG2,2HG2,3HG2) =====")
    idx_1HG2 = name2idx.get("1HG2")
    idx_2HG2 = name2idx.get("2HG2")
    idx_3HG2 = name2idx.get("3HG2")
    alvo_hg2 = [idx for idx in (idx_1HG2, idx_2HG2, idx_3HG2) if idx is not None]

    det_hg2 = atribuir_coord_3_alvos(
        atoms, locked, idx_ref=idx_CG2,
        alvo_idxs=alvo_hg2,
        dmin_init=0.990, dmax_init=1.150,
        delta=0.005, max_iter=200,
        label="HG2"
    )
    print(f"Janela final CG2–H usada: {det_hg2['janela_final'][0]:.3f} – {det_hg2['janela_final'][1]:.3f} Å")
    for m in det_hg2['mapeamentos']:
        alvo = atoms[m['alvo_idx']]
        if m.get('cand_idx') is None:
            print(f"  [HG2] alvo {alvo['name']} serial {alvo['serial']} sem candidato explícito, apenas locked.")
            continue
        cand = atoms[m['cand_idx']]
        if m['swap_feito']:
            print(f"  [HG2] alvo {alvo['name']}({alvo['serial']}) "
                  f"<-> candidato {cand['name']}({cand['serial']}): "
                  f"swap feito. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                  f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
        else:
            print(f"  [HG2] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                  f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                  f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")

    # ===========================
    # 3) CB–CG1
    # ===========================
    print("\n===== ETAPA 3: CB–CG1 =====")
    cand_idx, d_cg1, ang_cg1 = escolher_cg(
        atoms, locked, idx_cb=idx_CB, idx_ca=idx_CA,
        dmin=1.40, dmax=1.65,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="CG1"
    )
    print(f"CG1 candidato: serial={atoms[cand_idx]['serial']} "
          f"{atoms[cand_idx]['name']} {atoms[cand_idx]['resname']} {atoms[cand_idx]['resseq']}")
    print(f"  Distância CB–candidato = {d_cg1:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_cg1:.3f} °")

    at_cg1 = atoms[idx_CG1]
    if not same_coords(at_cg1, atoms[cand_idx]):
        print("  -> Fazendo swap de coordenadas entre CG1 e candidato.")
        swap_coords(at_cg1, atoms[cand_idx])
    else:
        print("  -> CG1 já está usando essa coordenada (nenhum swap necessário).")
    locked.add(idx_CG1)

    # ===========================
    # 4) CG1–(1HG1,2HG1,CD)
    # ===========================
    print("\n===== ETAPA 4: CG1–(1HG1,2HG1,CD) =====")
    idx_1HG1 = name2idx.get("1HG1")
    idx_2HG1 = name2idx.get("2HG1")
    idx_CD   = name2idx.get("CD")
    alvo_cg1_viz = [idx for idx in (idx_1HG1, idx_2HG1, idx_CD) if idx is not None]

    det_cg1 = atribuir_coord_3_alvos(
        atoms, locked, idx_ref=idx_CG1,
        alvo_idxs=alvo_cg1_viz,
        dmin_init=0.990, dmax_init=1.150,
        delta=0.005, max_iter=200,
        label="CG1_neighbors"
    )
    print(f"Janela final CG1–(H,H,C) usada: {det_cg1['janela_final'][0]:.3f} – {det_cg1['janela_final'][1]:.3f} Å")
    for m in det_cg1['mapeamentos']:
        alvo = atoms[m['alvo_idx']]
        if m.get('cand_idx') is None:
            print(f"  [CG1-viz] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
            continue
        cand = atoms[m['cand_idx']]
        if m['swap_feito']:
            print(f"  [CG1-viz] alvo {alvo['name']}({alvo['serial']}) "
                  f"<-> candidato {cand['name']}({cand['serial']}): "
                  f"swap feito. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                  f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
        else:
            print(f"  [CG1-viz] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                  f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                  f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")

    # ===========================
    # 5) CD–(HD1,HD2,HD3)
    # ===========================
    print("\n===== ETAPA 5: CD–(HD1,HD2,HD3) =====")
    idx_CD = name2idx.get("CD")
    idx_HD1 = name2idx.get("HD1")
    idx_HD2 = name2idx.get("HD2")
    idx_HD3 = name2idx.get("HD3")
    alvo_cd_viz = [idx for idx in (idx_HD1, idx_HD2, idx_HD3) if idx is not None]

    det_cd = atribuir_coord_3_alvos(
        atoms, locked, idx_ref=idx_CD,
        alvo_idxs=alvo_cd_viz,
        dmin_init=0.990, dmax_init=1.150,
        delta=0.005, max_iter=200,
        label="CD_neighbors"
    )
    print(f"Janela final CD–H usada: {det_cd['janela_final'][0]:.3f} – {det_cd['janela_final'][1]:.3f} Å")
    for m in det_cd['mapeamentos']:
        alvo = atoms[m['alvo_idx']]
        if m.get('cand_idx') is None:
            print(f"  [CD-viz] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
            continue
        cand = atoms[m['cand_idx']]
        if m['swap_feito']:
            print(f"  [CD-viz] alvo {alvo['name']}({alvo['serial']}) "
                  f"<-> candidato {cand['name']}({cand['serial']}): "
                  f"swap feito. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                  f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
        else:
            print(f"  [CD-viz] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                  f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                  f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")

    # ===========================
    # Escrita do novo PDB
    # ===========================
    lines_out = update_pdb_lines(lines, atoms)
    with open(saida_filename, 'w') as f:
        f.write("\n".join(lines_out) + "\n")
    print(f"\nNovo PDB escrito em: {saida_filename}")

    return saida_filename

# ===========================
# Bloco para uso no Google Colab
# ===========================

if __name__ == "__main__":
    try:
        # Bloco para uso direto no Google Colab
        from google.colab import files  # type: ignore
        print("Selecione o arquivo PDB (ex.: backbone_rebuilt_with_HNO_CB.pdb):")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo foi carregado.")
        pdb_in = list(uploaded.keys())[0]
        saida = "backbone_rebuilt_with_HNO_CB_ILE576.pdb"
        reconstruir_ile576_sidechain(pdb_in, saida_filename=saida,
                                     resname="ILE", resseq=576, chain="B")
        files.download(saida)
    except ImportError:
        # Uso local (fora do Colab): ajuste os nomes de arquivo manualmente
        pdb_in = "backbone_rebuilt_with_HNO_CB.pdb"
        saida = "backbone_rebuilt_with_HNO_CB_ILE576.pdb"
        reconstruir_ile576_sidechain(pdb_in, saida_filename=saida,
                                     resname="ILE", resseq=576, chain="B")


Selecione o arquivo PDB (ex.: backbone_rebuilt_with_HNO_CB.pdb):


Saving backbone_rebuilt_with_HNO_CB.pdb to backbone_rebuilt_with_HNO_CB (2).pdb
PDB carregado: backbone_rebuilt_with_HNO_CB (2).pdb
Total de átomos lidos: 137
Átomos inicializados como imutáveis (locked): 54
Reconstruindo cadeia lateral de ILE B 576

===== ETAPA 1: CB–CG2 =====
CG2 candidato: serial=8647 CG2 ILE 576
  Distância CB–candidato = 1.515 Å
  Ângulo CA–CB–candidato = 116.416 °
  -> CG2 já está usando essa coordenada (nenhum swap necessário).

===== ETAPA 2: CG2–HG2 (1HG2,2HG2,3HG2) =====
Janela final CG2–H usada: 0.990 – 1.150 Å
  [HG2] alvo 1HG2(8648) já coincidia com candidato 1HG2(8648), sem swap. dist_ref=1.109 Å
  [HG2] alvo 2HG2(8649) já coincidia com candidato 2HG2(8649), sem swap. dist_ref=1.115 Å
  [HG2] alvo 3HG2(8650) já coincidia com candidato 3HG2(8650), sem swap. dist_ref=1.106 Å

===== ETAPA 3: CB–CG1 =====
CG1 candidato: serial=8651 CG1 ILE 576
  Distância CB–candidato = 1.561 Å
  Ângulo CA–CB–candidato = 103.313 °
  -> CG1 já está usando essa coordenada (nenh

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### REESTRUTURA 577

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução/validação da cadeia lateral de THR 577 (CB–OG1–HG1, CB–HB, CB–CG2–HG2)
seguindo regras de distância e ângulo fornecidas.

- Matriz única de coordenadas
- Conjunto global de locked (imutáveis)
- Seleção de OG1 e CG2 por:
    * distância CB–candidato
    * ângulo CA–CB–candidato (3 refinamentos)
- Seleção de HG1, HB, 1HG2,2HG2,3HG2 com janelas dinâmicas de distância.

Resíduo alvo: THR B 577.
Também bloqueia todo o resíduo ILE B 576 (já estabilizado).
"""

import math

# ===========================
# Funções geométricas básicas
# ===========================

def dist(a, b):
    return math.sqrt((a['x'] - b['x'])**2 +
                     (a['y'] - b['y'])**2 +
                     (a['z'] - b['z'])**2)

def angle(a, b, c):
    """
    Ângulo A-B-C (v1 = A-B, v2 = C-B) em graus.
    """
    v1 = (a['x'] - b['x'], a['y'] - b['y'], a['z'] - b['z'])
    v2 = (c['x'] - b['x'], c['y'] - b['y'], c['z'] - b['z'])
    n1 = math.sqrt(sum(v*v for v in v1))
    n2 = math.sqrt(sum(v*v for v in v2))
    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    dot = sum(v1[i]*v2[i] for i in range(3))
    cosang = max(-1.0, min(1.0, dot/(n1*n2)))
    return math.degrees(math.acos(cosang))

def same_coords(a, b, tol=1e-3):
    return (abs(a['x'] - b['x']) < tol and
            abs(a['y'] - b['y']) < tol and
            abs(a['z'] - b['z']) < tol)

# ===========================
# Parsing e escrita de PDB
# ===========================

def parse_pdb_lines(lines):
    atoms = []
    atom_line_idx = []
    for i, line in enumerate(lines):
        if line.startswith("ATOM") or line.startswith("HETATM"):
            try:
                rec = {
                    'line_idx': i,
                    'serial': int(line[6:11]),
                    'name':   line[12:16].strip(),
                    'altloc': line[16],
                    'resname': line[17:20].strip(),
                    'chain':  line[21].strip(),
                    'resseq': int(line[22:26]),
                    'icode':  line[26],
                    'x': float(line[30:38]),
                    'y': float(line[38:46]),
                    'z': float(line[46:54]),
                    'raw_line': line
                }
            except Exception:
                continue
            atoms.append(rec)
            atom_line_idx.append(i)
    return atoms, atom_line_idx

def update_pdb_lines(lines, atoms):
    """
    Atualiza apenas coordenadas de ATOM/HETATM (colunas 31–54).
    """
    for at in atoms:
        i = at['line_idx']
        line = lines[i]
        if len(line) < 54:
            line = line.ljust(54)
        new_line = (line[:30] +
                    f"{at['x']:8.3f}{at['y']:8.3f}{at['z']:8.3f}" +
                    line[54:])
        lines[i] = new_line
    return lines

# ===========================
# Utilitários de busca
# ===========================

def find_residue_atoms(atoms, resname, resseq, chain=None):
    idxs = []
    for i, a in enumerate(atoms):
        if a['resname'] == resname and a['resseq'] == resseq:
            if chain is None or a['chain'] == chain:
                idxs.append(i)
    return idxs

def build_name_index(atoms, residue_idxs):
    """
    name -> idx (primeira ocorrência no resíduo).
    """
    name2idx = {}
    for i in residue_idxs:
        nm = atoms[i]['name']
        if nm not in name2idx:
            name2idx[nm] = i
    return name2idx

def swap_coords(a, b):
    for coord in ('x', 'y', 'z'):
        a[coord], b[coord] = b[coord], a[coord]

# ===========================
# Seleção OG1/CG2 por distância + ângulo
# ===========================

def escolher_cg_avancado(atoms, locked, idx_cb, idx_ca,
                          dmin, dmax,
                          ang1_min=101.0, ang1_max=119.0,
                          ang2_min=104.0, ang2_max=116.0,
                          alvo_angulo=110.0,
                          label="CG?"):
    """
    Seleciona um candidato para OG1/CG2 com:
      - distância CB–candidato em [dmin, dmax]
      - refinamentos de ângulo CA–CB–candidato:
          REF1: 101–119°
          REF2: 104–116°
          REF3: mais próximo de alvo_angulo (110°)

    Regra extra:
      Se após REF1 ou REF2 a lista ir de >=2 candidatos para 0,
      volta para a lista anterior e aplica REF3 nela.
    """
    ref_cb = atoms[idx_cb]
    ref_ca = atoms[idx_ca]

    base = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_cb:
            continue
        d = dist(ref_cb, a)
        if dmin <= d <= dmax:
            ang = angle(ref_ca, ref_cb, a)
            base.append((i, d, ang))

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} em {dmin:.2f}-{dmax:.2f} Å.")

    if len(base) == 1:
        return base[0]

    # REF1: 101–119°
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]

    if len(ref1) == 0 and prev_count >= 2:
        # volta para prev_list e aplica REF3
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]

    if len(ref1) == 1:
        return ref1[0]

    # tem >1 em ref1 -> seguir
    prev_list = ref1
    prev_count = len(prev_list)

    # REF2: 104–116°
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]

    if len(ref2) == 0 and prev_count >= 2:
        # volta para prev_list (ref1) e aplica REF3
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]

    if len(ref2) == 1:
        return ref2[0]

    # Ainda >1 -> REF3 direto em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Janela dinâmica de vizinhos
# ===========================

def escolher_vizinhos_dinamico(atoms, locked, idx_ref,
                               target_count,
                               dmin_init, dmax_init,
                               delta=0.005, max_iter=200,
                               label="H?"):
    """
    Janela dinâmica em torno de [dmin_init, dmax_init] para encontrar
    'target_count' vizinhos:

    - Se len(cand) > target_count: estreita (dmin += delta, dmax -= delta)
    - Se len(cand) < target_count: alarga (dmin -= delta, dmax += delta)

    Se não convergir, pega os 'target_count' mais próximos.
    """
    ref = atoms[idx_ref]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        cand = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_ref:
                continue
            d = dist(ref, a)
            if dmin <= d <= dmax:
                cand.append((i, d))
        if len(cand) == target_count:
            return [i for (i, _) in cand], dmin, dmax
        elif len(cand) > target_count:
            dmin += delta
            dmax -= delta
            if dmin >= dmax:
                break
        else:  # len(cand) < target_count
            dmin = max(0.0, dmin - delta)
            dmax += delta

    # fallback: pega os 'target_count' mais próximos em tudo que estiver livre
    cand_all = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_ref:
            continue
        d = dist(ref, a)
        cand_all.append((i, d))
    cand_all.sort(key=lambda t: t[1])
    chosen = cand_all[:target_count]
    return [i for (i, _) in chosen], dmin, dmax

def atribuir_coord_alvos(atoms, locked, idx_ref, alvo_idxs,
                         dmin_init, dmax_init,
                         target_count,
                         delta=0.005, max_iter=200,
                         label="H?"):
    """
    Usa janela dinâmica para encontrar 'target_count' candidatos em torno
    de idx_ref e distribui entre os átomos alvo (alvo_idxs).

    - Primeiro casa alvos que já coincidam com alguma coord candidata.
    - Depois faz swap para os demais alvos com candidatos restantes.
    - Todos os alvos tornam-se locked ao final.
    """
    ref = atoms[idx_ref]
    cand_idxs, dmin_final, dmax_final = escolher_vizinhos_dinamico(
        atoms, locked, idx_ref, target_count,
        dmin_init, dmax_init, delta, max_iter, label
    )

    cand_coords = {i: (atoms[i]['x'], atoms[i]['y'], atoms[i]['z']) for i in cand_idxs}
    used_cands = set()

    detalhes = {
        'ref': ref,
        'ref_idx': idx_ref,
        'janela_final': (dmin_final, dmax_final),
        'mapeamentos': []
    }

    # 1) Tenta casar alvos que já têm coord igual a algum candidato
    for idx_alvo in alvo_idxs:
        alvo = atoms[idx_alvo]
        match = None
        for i_cand in cand_idxs:
            if i_cand in used_cands:
                continue
            cx, cy, cz = cand_coords[i_cand]
            if (abs(alvo['x'] - cx) < 1e-3 and
                abs(alvo['y'] - cy) < 1e-3 and
                abs(alvo['z'] - cz) < 1e-3):
                match = i_cand
                break
        if match is not None:
            used_cands.add(match)
            locked.add(idx_alvo)
            d = dist(ref, alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': match,
                'cand': atoms[match],
                'dist_ref_alvo': d,
                'swap_feito': False
            })

    # 2) Para alvos restantes, faz swap com candidatos não utilizados
    for idx_alvo in alvo_idxs:
        if idx_alvo in locked:
            continue
        alvo = atoms[idx_alvo]
        cand_rest = [i for i in cand_idxs if i not in used_cands]
        if not cand_rest:
            locked.add(idx_alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': None,
                'cand': None,
                'dist_ref_alvo': dist(ref, alvo),
                'swap_feito': False
            })
            continue

        # escolhe candidato mais próximo do ref
        cand_rest.sort(key=lambda i: dist(ref, atoms[i]))
        i_cand = cand_rest[0]
        used_cands.add(i_cand)
        cand_atom = atoms[i_cand]

        d_before = dist(ref, alvo)
        if not same_coords(alvo, cand_atom):
            swap_coords(alvo, cand_atom)
            swap_feito = True
        else:
            swap_feito = False

        locked.add(idx_alvo)
        d_after = dist(ref, alvo)
        detalhes['mapeamentos'].append({
            'alvo_idx': idx_alvo,
            'alvo': alvo,
            'cand_idx': i_cand,
            'cand': cand_atom,
            'dist_ref_alvo_antes': d_before,
            'dist_ref_alvo_depois': d_after,
            'swap_feito': swap_feito
        })

    return detalhes

# ===========================
# Pipeline principal THR 577
# ===========================

def reconstruir_thr577_sidechain(pdb_filename,
                                 saida_filename="backbone_rebuilt_with_HNO_CB_THR577.pdb",
                                 resname="THR", resseq=577, chain="B"):
    """
    Reconstrói/valida a cadeia lateral de THR 577, garantindo:
      - CB–OG1: 1.30–1.50 Å + ângulo CA–CB–OG1 (101–119; 104–116; ~110)
      - OG1–HG1: 0.90–1.10 Å (janela dinâmica, 1 alvo)
      - CB–HB: 1.00–1.20 Å (janela dinâmica, 1 alvo)
      - CB–CG2: 1.40–1.60 Å + ângulo CA–CB–CG2 (mesma lógica de OG1)
      - CG2–(1HG2,2HG2,3HG2): 0.995–1.115 Å (janela dinâmica, 3 alvos)
    """
    with open(pdb_filename, 'r') as f:
        lines = f.read().splitlines()

    atoms, atom_line_idx = parse_pdb_lines(lines)
    print(f"PDB carregado: {pdb_filename}")
    print(f"Total de átomos lidos: {len(atoms)}")

    # locked inicial: N, CA, C, HN, HA, O, CB
    locked = set()
    base_locked_names = {"N", "CA", "C", "HN", "HA", "O", "CB"}
    for i, a in enumerate(atoms):
        if a['name'] in base_locked_names:
            locked.add(i)

    # Bloqueia todo resíduo ILE 576 (já estabilizado)
    ile576_idxs = find_residue_atoms(atoms, resname="ILE", resseq=576, chain=chain)
    for i in ile576_idxs:
        locked.add(i)

    print(f"Átomos inicializados como imutáveis (locked): {len(locked)}")

    # Resíduo THR 577
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    if not res_idxs:
        raise RuntimeError(f"Resíduo {resname} {chain} {resseq} não encontrado no PDB.")

    print(f"Reconstruindo cadeia lateral de {resname} {chain} {resseq}")
    name2idx = build_name_index(atoms, res_idxs)

    # indices chave
    idx_CA  = name2idx.get("CA")
    idx_CB  = name2idx.get("CB")
    idx_OG1 = name2idx.get("OG1")
    idx_HG1 = name2idx.get("HG1")
    idx_HB  = name2idx.get("HB")
    idx_CG2 = name2idx.get("CG2")
    idx_1HG2 = name2idx.get("1HG2")
    idx_2HG2 = name2idx.get("2HG2")
    idx_3HG2 = name2idx.get("3HG2")

    obrigatorios = ["CA", "CB", "OG1", "HG1", "HB", "CG2", "1HG2", "2HG2", "3HG2"]
    for nm in obrigatorios:
        if nm not in name2idx:
            print(f"AVISO: átomo {nm} não encontrado em {resname} {chain} {resseq}.")

    # -----------------------
    # 1) CB–OG1
    # -----------------------
    print("\n===== ETAPA 1: CB–OG1 =====")
    cand_idx, d_og1, ang_og1 = escolher_cg_avancado(
        atoms, locked,
        idx_cb=idx_CB, idx_ca=idx_CA,
        dmin=1.30, dmax=1.50,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="OG1"
    )
    cand = atoms[cand_idx]
    print(f"OG1 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CB–candidato = {d_og1:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_og1:.3f} °")

    at_og1 = atoms[idx_OG1]
    if not same_coords(at_og1, cand):
        print("  -> Fazendo swap de coordenadas entre OG1 e candidato.")
        swap_coords(at_og1, cand)
    else:
        print("  -> OG1 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_OG1)

    # -----------------------
    # 2) OG1–HG1
    # -----------------------
    print("\n===== ETAPA 2: OG1–HG1 =====")
    if idx_HG1 is not None:
        det_hg1 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_OG1,
            alvo_idxs=[idx_HG1],
            dmin_init=0.90, dmax_init=1.10,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HG1"
        )
        print(f"Janela final OG1–HG1 usada: {det_hg1['janela_final'][0]:.3f} – {det_hg1['janela_final'][1]:.3f} Å")
        for m in det_hg1['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HG1] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HG1] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HG1] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HG1 não encontrado, pulando etapa.")

    # -----------------------
    # 3) CB–HB
    # -----------------------
    print("\n===== ETAPA 3: CB–HB =====")
    if idx_HB is not None:
        det_hb = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CB,
            alvo_idxs=[idx_HB],
            dmin_init=1.00, dmax_init=1.20,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HB"
        )
        print(f"Janela final CB–HB usada: {det_hb['janela_final'][0]:.3f} – {det_hb['janela_final'][1]:.3f} Å")
        for m in det_hb['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HB não encontrado, pulando etapa.")

    # -----------------------
    # 4) CB–CG2
    # -----------------------
    print("\n===== ETAPA 4: CB–CG2 =====")
    cand_idx, d_cg2, ang_cg2 = escolher_cg_avancado(
        atoms, locked,
        idx_cb=idx_CB, idx_ca=idx_CA,
        dmin=1.40, dmax=1.60,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="CG2"
    )
    cand = atoms[cand_idx]
    print(f"CG2 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CB–candidato = {d_cg2:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_cg2:.3f} °")

    at_cg2 = atoms[idx_CG2]
    if not same_coords(at_cg2, cand):
        print("  -> Fazendo swap de coordenadas entre CG2 e candidato.")
        swap_coords(at_cg2, cand)
    else:
        print("  -> CG2 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CG2)

    # -----------------------
    # 5) CG2–(1HG2,2HG2,3HG2)
    # -----------------------
    print("\n===== ETAPA 5: CG2–(1HG2,2HG2,3HG2) =====")
    alvo_hg2 = [idx for idx in (idx_1HG2, idx_2HG2, idx_3HG2) if idx is not None]
    if alvo_hg2:
        det_hg2 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CG2,
            alvo_idxs=alvo_hg2,
            dmin_init=0.995, dmax_init=1.115,
            target_count=3,
            delta=0.005, max_iter=200,
            label="HG2"
        )
        print(f"Janela final CG2–HG2 usada: {det_hg2['janela_final'][0]:.3f} – {det_hg2['janela_final'][1]:.3f} Å")
        for m in det_hg2['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HG2] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HG2] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HG2] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HG2 não encontrados, pulando etapa.")

    # ===========================
    # Escrita do novo PDB
    # ===========================
    lines_out = update_pdb_lines(lines, atoms)
    with open(saida_filename, 'w') as f:
        f.write("\n".join(lines_out) + "\n")
    print(f"\nNovo PDB escrito em: {saida_filename}")

    return saida_filename

# ===========================
# Bloco para uso no Google Colab
# ===========================

if __name__ == "__main__":
    try:
        from google.colab import files  # type: ignore
        print("Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_ILE576.pdb):")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo foi carregado.")
        pdb_in = list(uploaded.keys())[0]
        saida = "backbone_rebuilt_with_HNO_CB_THR577.pdb"
        reconstruir_thr577_sidechain(pdb_in, saida_filename=saida,
                                     resname="THR", resseq=577, chain="B")
        files.download(saida)
    except ImportError:
        # Uso fora do Colab: ajuste os caminhos manualmente
        pdb_in = "backbone_rebuilt_with_HNO_CB_ILE576.pdb"
        saida = "backbone_rebuilt_with_HNO_CB_THR577.pdb"
        reconstruir_thr577_sidechain(pdb_in, saida_filename=saida,
                                     resname="THR", resseq=577, chain="B")


Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_ILE576.pdb):


Saving backbone_rebuilt_with_HNO_CB_ILE576.pdb to backbone_rebuilt_with_HNO_CB_ILE576 (2).pdb
PDB carregado: backbone_rebuilt_with_HNO_CB_ILE576 (2).pdb
Total de átomos lidos: 137
Átomos inicializados como imutáveis (locked): 66
Reconstruindo cadeia lateral de THR B 577

===== ETAPA 1: CB–OG1 =====
OG1 candidato: serial=8666 OG1 THR 577
  Distância CB–candidato = 1.449 Å
  Ângulo CA–CB–candidato = 110.144 °
  -> OG1 já está usando essa coordenada (nenhum swap).

===== ETAPA 2: OG1–HG1 =====
Janela final OG1–HG1 usada: 0.900 – 1.100 Å
  [HG1] alvo HG1(8667) já coincidia com candidato HG1(8667), sem swap. dist_ref=0.957 Å

===== ETAPA 3: CB–HB =====
Janela final CB–HB usada: 1.000 – 1.200 Å
  [HB] alvo HB(8665) já coincidia com candidato HB(8665), sem swap. dist_ref=1.109 Å

===== ETAPA 4: CB–CG2 =====
CG2 candidato: serial=8668 CG2 THR 577
  Distância CB–candidato = 1.535 Å
  Ângulo CA–CB–candidato = 112.516 °
  -> CG2 já está usando essa coordenada (nenhum swap).

===== ETAPA 5: CG2–(1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### REORDENAÇÃO 578

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução/validação da cadeia lateral de LEU 578 (CB–HB1/HB2–CG–HG–CD1/2–HD1/HD2)
seguindo as regras de distância e ângulo especificadas.

Resíduo alvo: LEU B 578.
Bloqueia (locked) todo ILE 576 e THR 577, além de N/CA/C/HN/HA/O/CB globais.

TOLERÂNCIAS DE DISTÂNCIA:
CB - HB1 entre 0.995 a 1.115 Å
CB - HB2 entre 0.995 a 1.115 Å
CB - CG  entre 1.40 a 1.60 Å
CG - HG  entre 0.995 a 1.115 Å
CG - CD1 entre 1.40 a 1.60 Å
CD1 - 1HD1/2HD1/3HD1 entre 0.995 a 1.115 Å
CG - CD2 entre 1.40 a 1.60 Å
CD2 - 1HD2/2HD2/3HD2 entre 0.995 a 1.115 Å
"""

import math

# ===========================
# Funções geométricas
# ===========================

def dist(a, b):
    return math.sqrt((a['x'] - b['x'])**2 +
                     (a['y'] - b['y'])**2 +
                     (a['z'] - b['z'])**2)

def angle(a, b, c):
    """
    Ângulo A-B-C em graus (v1 = A-B, v2 = C-B).
    """
    v1 = (a['x'] - b['x'], a['y'] - b['y'], a['z'] - b['z'])
    v2 = (c['x'] - b['x'], c['y'] - b['y'], c['z'] - b['z'])
    n1 = math.sqrt(sum(v*v for v in v1))
    n2 = math.sqrt(sum(v*v for v in v2))
    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    dot = sum(v1[i]*v2[i] for i in range(3))
    cosang = max(-1.0, min(1.0, dot/(n1*n2)))
    return math.degrees(math.acos(cosang))

def same_coords(a, b, tol=1e-3):
    return (abs(a['x'] - b['x']) < tol and
            abs(a['y'] - b['y']) < tol and
            abs(a['z'] - b['z']) < tol)

# ===========================
# Parsing / escrita de PDB
# ===========================

def parse_pdb_lines(lines):
    atoms = []
    atom_line_idx = []
    for i, line in enumerate(lines):
        if line.startswith("ATOM") or line.startswith("HETATM"):
            try:
                rec = {
                    'line_idx': i,
                    'serial': int(line[6:11]),
                    'name':   line[12:16].strip(),
                    'altloc': line[16],
                    'resname': line[17:20].strip(),
                    'chain':  line[21].strip(),
                    'resseq': int(line[22:26]),
                    'icode':  line[26],
                    'x': float(line[30:38]),
                    'y': float(line[38:46]),
                    'z': float(line[46:54]),
                    'raw_line': line
                }
            except Exception:
                continue
            atoms.append(rec)
            atom_line_idx.append(i)
    return atoms, atom_line_idx

def update_pdb_lines(lines, atoms):
    for at in atoms:
        i = at['line_idx']
        line = lines[i]
        if len(line) < 54:
            line = line.ljust(54)
        new_line = (line[:30] +
                    f"{at['x']:8.3f}{at['y']:8.3f}{at['z']:8.3f}" +
                    line[54:])
        lines[i] = new_line
    return lines

# ===========================
# Utilitários de busca
# ===========================

def find_residue_atoms(atoms, resname, resseq, chain=None):
    idxs = []
    for i, a in enumerate(atoms):
        if a['resname'] == resname and a['resseq'] == resseq:
            if chain is None or a['chain'] == chain:
                idxs.append(i)
    return idxs

def build_name_index(atoms, residue_idxs):
    name2idx = {}
    for i in residue_idxs:
        nm = atoms[i]['name']
        if nm not in name2idx:
            name2idx[nm] = i
    return name2idx

def swap_coords(a, b):
    for coord in ('x', 'y', 'z'):
        a[coord], b[coord] = b[coord], a[coord]

# ===========================
# Seleção de pesado (CG, CD1, CD2) por distância + ângulo CA–CB–cand
# ===========================

def escolher_peso_avancado(atoms, locked,
                            idx_central_dist,  # índice usado na distância (CB ou CG)
                            idx_CA, idx_CB,    # para ângulo CA–CB–candidato
                            dmin, dmax,
                            ang1_min=101.0, ang1_max=119.0,
                            ang2_min=104.0, ang2_max=116.0,
                            alvo_angulo=110.0,
                            label="PESO"):
    """
    Seleciona um candidato pesado (CG/CD1/CD2) com:
      - distância em [dmin, dmax] a partir de idx_central_dist (CB ou CG)
      - ângulo CA–CB–candidato refinado em 3 passos:
          REF1: 101–119°
          REF2: 104–116°
          REF3: mais próximo de alvo_angulo
      Se em REF1 ou REF2 a lista vai de >=2 para 0, volta para lista anterior e aplica REF3.
    """
    ref_center = atoms[idx_central_dist]
    ref_CA = atoms[idx_CA]
    ref_CB = atoms[idx_CB]

    base = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_central_dist:
            continue
        d = dist(ref_center, a)
        if dmin <= d <= dmax:
            ang = angle(ref_CA, ref_CB, a)
            base.append((i, d, ang))

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} em {dmin:.2f}-{dmax:.2f} Å.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]

    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]

    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Janela dinâmica p/ hidrogênios
# ===========================

def escolher_vizinhos_dinamico(atoms, locked, idx_ref,
                               target_count,
                               dmin_init, dmax_init,
                               delta=0.005, max_iter=200,
                               label="H?"):
    """
    Janela dinâmica [dmin,dmax] para encontrar target_count vizinhos.
    > target_count: estreita (dmin += delta, dmax -= delta)
    < target_count: alarga (dmin -= delta, dmax += delta).
    Se não convergir, pega os target_count mais próximos.
    """
    ref = atoms[idx_ref]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        cand = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_ref:
                continue
            d = dist(ref, a)
            if dmin <= d <= dmax:
                cand.append((i, d))
        if len(cand) == target_count:
            return [i for (i, _) in cand], dmin, dmax
        elif len(cand) > target_count:
            dmin += delta
            dmax -= delta
            if dmin >= dmax:
                break
        else:  # len(cand) < target_count
            dmin = max(0.0, dmin - delta)
            dmax += delta

    # fallback
    cand_all = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_ref:
            continue
        d = dist(ref, a)
        cand_all.append((i, d))
    cand_all.sort(key=lambda t: t[1])
    chosen = cand_all[:target_count]
    return [i for (i, _) in chosen], dmin, dmax

def atribuir_coord_alvos(atoms, locked, idx_ref, alvo_idxs,
                         dmin_init, dmax_init,
                         target_count,
                         delta=0.005, max_iter=200,
                         label="H?"):
    """
    Usa janela dinâmica para achar target_count candidatos ao redor de idx_ref,
    e distribui coordenadas entre os átomos alvo.

    - Primeiro fixa alvos que já têm coord igual a alguma candidata (sem swap).
    - Depois faz swap para os demais alvos com candidatos restantes.
    - Todos os alvos entram em locked ao final.
    """
    ref = atoms[idx_ref]
    cand_idxs, dmin_final, dmax_final = escolher_vizinhos_dinamico(
        atoms, locked, idx_ref, target_count,
        dmin_init, dmax_init, delta, max_iter, label
    )
    cand_coords = {i: (atoms[i]['x'], atoms[i]['y'], atoms[i]['z']) for i in cand_idxs}
    used_cands = set()

    detalhes = {
        'ref': ref,
        'ref_idx': idx_ref,
        'janela_final': (dmin_final, dmax_final),
        'mapeamentos': []
    }

    # 1) Alvos já coincidindo com algum candidato
    for idx_alvo in alvo_idxs:
        alvo = atoms[idx_alvo]
        match = None
        for i_cand in cand_idxs:
            if i_cand in used_cands:
                continue
            cx, cy, cz = cand_coords[i_cand]
            if (abs(alvo['x'] - cx) < 1e-3 and
                abs(alvo['y'] - cy) < 1e-3 and
                abs(alvo['z'] - cz) < 1e-3):
                match = i_cand
                break
        if match is not None:
            used_cands.add(match)
            locked.add(idx_alvo)
            d = dist(ref, alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': match,
                'cand': atoms[match],
                'dist_ref_alvo': d,
                'swap_feito': False
            })

    # 2) Alvos restantes com swap
    for idx_alvo in alvo_idxs:
        if idx_alvo in locked:
            continue
        alvo = atoms[idx_alvo]
        cand_rest = [i for i in cand_idxs if i not in used_cands]
        if not cand_rest:
            locked.add(idx_alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': None,
                'cand': None,
                'dist_ref_alvo': dist(ref, alvo),
                'swap_feito': False
            })
            continue

        cand_rest.sort(key=lambda i: dist(ref, atoms[i]))
        i_cand = cand_rest[0]
        used_cands.add(i_cand)
        cand_atom = atoms[i_cand]

        d_before = dist(ref, alvo)
        if not same_coords(alvo, cand_atom):
            swap_coords(alvo, cand_atom)
            swap_feito = True
        else:
            swap_feito = False

        locked.add(idx_alvo)
        d_after = dist(ref, alvo)
        detalhes['mapeamentos'].append({
            'alvo_idx': idx_alvo,
            'alvo': alvo,
            'cand_idx': i_cand,
            'cand': cand_atom,
            'dist_ref_alvo_antes': d_before,
            'dist_ref_alvo_depois': d_after,
            'swap_feito': swap_feito
        })

    return detalhes

# ===========================
# Pipeline principal LEU 578
# ===========================

def reconstruir_leu578_sidechain(pdb_filename,
                                 saida_filename="backbone_rebuilt_with_HNO_CB_LEU578.pdb",
                                 resname="LEU", resseq=578, chain="B"):
    with open(pdb_filename, 'r') as f:
        lines = f.read().splitlines()

    atoms, atom_line_idx = parse_pdb_lines(lines)
    print(f"PDB carregado: {pdb_filename}")
    print(f"Total de átomos lidos: {len(atoms)}")

    # locked inicial: N, CA, C, HN, HA, O, CB
    locked = set()
    base_locked_names = {"N", "CA", "C", "HN", "HA", "O", "CB"}
    for i, a in enumerate(atoms):
        if a['name'] in base_locked_names:
            locked.add(i)

    # Bloqueia todo ILE 576 e THR 577
    for (rname, rseq) in [("ILE", 576), ("THR", 577)]:
        idxs_res = find_residue_atoms(atoms, resname=rname, resseq=rseq, chain=chain)
        for i in idxs_res:
            locked.add(i)

    print(f"Átomos inicializados como imutáveis (locked): {len(locked)}")

    # Resíduo LEU 578
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    if not res_idxs:
        raise RuntimeError(f"Resíduo {resname} {chain} {resseq} não encontrado.")

    print(f"Reconstruindo cadeia lateral de {resname} {chain} {resseq}")
    name2idx = build_name_index(atoms, res_idxs)

    # Índices importantes
    idx_CA   = name2idx.get("CA")
    idx_CB   = name2idx.get("CB")
    idx_HB1  = name2idx.get("HB1")
    idx_HB2  = name2idx.get("HB2")
    idx_CG   = name2idx.get("CG")
    idx_HG   = name2idx.get("HG")
    idx_CD1  = name2idx.get("CD1")
    idx_CD2  = name2idx.get("CD2")
    idx_1HD1 = name2idx.get("1HD1")
    idx_2HD1 = name2idx.get("2HD1")
    idx_3HD1 = name2idx.get("3HD1")
    idx_1HD2 = name2idx.get("1HD2")
    idx_2HD2 = name2idx.get("2HD2")
    idx_3HD2 = name2idx.get("3HD2")

    obrigatorios = ["CA", "CB", "HB1", "HB2", "CG", "HG",
                    "CD1", "CD2", "1HD1", "2HD1", "3HD1", "1HD2", "2HD2", "3HD2"]
    for nm in obrigatorios:
        if nm not in name2idx:
            print(f"AVISO: átomo {nm} não encontrado em {resname} {chain} {resseq}.")

    # -----------------------
    # 1) CB–HB1 / HB2
    # -----------------------
    print("\n===== ETAPA 1: CB–HB1/HB2 =====")
    alvo_hb = [idx for idx in (idx_HB1, idx_HB2) if idx is not None]
    if alvo_hb:
        det_hb = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CB,
            alvo_idxs=alvo_hb,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hb),
            delta=0.005, max_iter=200,
            label="HB1/HB2"
        )
        print(f"Janela final CB–HB1/HB2 usada: {det_hb['janela_final'][0]:.3f} – {det_hb['janela_final'][1]:.3f} Å")
        for m in det_hb['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HB1/HB2 não encontrados, pulando etapa.")

    # -----------------------
    # 2) CB–CG
    # -----------------------
    print("\n===== ETAPA 2: CB–CG =====")
    cand_idx, d_cg, ang_cg = escolher_peso_avancado(
        atoms, locked,
        idx_central_dist=idx_CB,
        idx_CA=idx_CA, idx_CB=idx_CB,
        dmin=1.40, dmax=1.60,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="CG"
    )
    cand = atoms[cand_idx]
    print(f"CG candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CB–candidato = {d_cg:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_cg:.3f} °")

    at_cg = atoms[idx_CG]
    if not same_coords(at_cg, cand):
        print("  -> Fazendo swap de coordenadas entre CG e candidato.")
        swap_coords(at_cg, cand)
    else:
        print("  -> CG já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CG)

    # -----------------------
    # 3) CG–HG
    # -----------------------
    print("\n===== ETAPA 3: CG–HG =====")
    if idx_HG is not None:
        det_hg = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CG,
            alvo_idxs=[idx_HG],
            dmin_init=0.995, dmax_init=1.115,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HG"
        )
        print(f"Janela final CG–HG usada: {det_hg['janela_final'][0]:.3f} – {det_hg['janela_final'][1]:.3f} Å")
        for m in det_hg['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HG não encontrado, pulando etapa.")

    # -----------------------
    # 4) CG–CD1
    # -----------------------
    print("\n===== ETAPA 4: CG–CD1 =====")
    cand_idx, d_cd1, ang_cd1 = escolher_peso_avancado(
        atoms, locked,
        idx_central_dist=idx_CG,   # distância CG–candidato
        idx_CA=idx_CA, idx_CB=idx_CB,  # ângulo CA–CB–cand
        dmin=1.40, dmax=1.60,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="CD1"
    )
    cand = atoms[cand_idx]
    print(f"CD1 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CG–candidato = {d_cd1:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_cd1:.3f} °")

    at_cd1 = atoms[idx_CD1]
    if not same_coords(at_cd1, cand):
        print("  -> Fazendo swap de coordenadas entre CD1 e candidato.")
        swap_coords(at_cd1, cand)
    else:
        print("  -> CD1 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CD1)

    # -----------------------
    # 5) CG–CD2
    # -----------------------
    print("\n===== ETAPA 5: CG–CD2 =====")
    cand_idx, d_cd2, ang_cd2 = escolher_peso_avancado(
        atoms, locked,
        idx_central_dist=idx_CG,
        idx_CA=idx_CA, idx_CB=idx_CB,
        dmin=1.40, dmax=1.60,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="CD2"
    )
    cand = atoms[cand_idx]
    print(f"CD2 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CG–candidato = {d_cd2:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_cd2:.3f} °")

    at_cd2 = atoms[idx_CD2]
    if not same_coords(at_cd2, cand):
        print("  -> Fazendo swap de coordenadas entre CD2 e candidato.")
        swap_coords(at_cd2, cand)
    else:
        print("  -> CD2 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CD2)

    # -----------------------
    # 6) CD1–(1HD1,2HD1,3HD1)
    # -----------------------
    print("\n===== ETAPA 6: CD1–(1HD1,2HD1,3HD1) =====")
    alvo_cd1_h = [idx for idx in (idx_1HD1, idx_2HD1, idx_3HD1) if idx is not None]
    if alvo_cd1_h:
        det_cd1_h = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CD1,
            alvo_idxs=alvo_cd1_h,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_cd1_h),
            delta=0.005, max_iter=200,
            label="CD1_H"
        )
        print(f"Janela final CD1–HD1 usada: {det_cd1_h['janela_final'][0]:.3f} – {det_cd1_h['janela_final'][1]:.3f} Å")
        for m in det_cd1_h['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [CD1-H] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [CD1-H] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [CD1-H] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> 1HD1/2HD1/3HD1 não encontrados, pulando etapa.")

    # -----------------------
    # 7) CD2–(1HD2,2HD2,3HD2)
    # -----------------------
    print("\n===== ETAPA 7: CD2–(1HD2,2HD2,3HD2) =====")
    alvo_cd2_h = [idx for idx in (idx_1HD2, idx_2HD2, idx_3HD2) if idx is not None]
    if alvo_cd2_h:
        det_cd2_h = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CD2,
            alvo_idxs=alvo_cd2_h,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_cd2_h),
            delta=0.005, max_iter=200,
            label="CD2_H"
        )
        print(f"Janela final CD2–HD2 usada: {det_cd2_h['janela_final'][0]:.3f} – {det_cd2_h['janela_final'][1]:.3f} Å")
        for m in det_cd2_h['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [CD2-H] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [CD2-H] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [CD2-H] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> 1HD2/2HD2/3HD2 não encontrados, pulando etapa.")

    # ===========================
    # Escrita do novo PDB
    # ===========================
    lines_out = update_pdb_lines(lines, atoms)
    with open(saida_filename, 'w') as f:
        f.write("\n".join(lines_out) + "\n")
    print(f"\nNovo PDB escrito em: {saida_filename}")

    return saida_filename

# ===========================
# Bloco Colab
# ===========================

if __name__ == "__main__":
    try:
        from google.colab import files  # type: ignore
        print("Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_THR577.pdb):")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo foi carregado.")
        pdb_in = list(uploaded.keys())[0]
        saida = "backbone_rebuilt_with_HNO_CB_LEU578.pdb"
        reconstruir_leu578_sidechain(pdb_in, saida_filename=saida,
                                     resname="LEU", resseq=578, chain="B")
        files.download(saida)
    except ImportError:
        # Uso local
        pdb_in = "backbone_rebuilt_with_HNO_CB_THR577.pdb"
        saida = "backbone_rebuilt_with_HNO_CB_LEU578.pdb"
        reconstruir_leu578_sidechain(pdb_in, saida_filename=saida,
                                     resname="LEU", resseq=578, chain="B")


Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_THR577.pdb):


Saving backbone_rebuilt_with_HNO_CB_THR577.pdb to backbone_rebuilt_with_HNO_CB_THR577 (2).pdb
PDB carregado: backbone_rebuilt_with_HNO_CB_THR577 (2).pdb
Total de átomos lidos: 137
Átomos inicializados como imutáveis (locked): 73
Reconstruindo cadeia lateral de LEU B 578

===== ETAPA 1: CB–HB1/HB2 =====
Janela final CB–HB1/HB2 usada: 0.990 – 1.120 Å
  [HB] alvo HB1(8679) <-> candidato CD2(8687): swap. dist_ref_antes=13.818 Å, dist_ref_depois=1.110 Å
  [HB] alvo HB2(8680) <-> candidato 1HD2(8688): swap. dist_ref_antes=14.859 Å, dist_ref_depois=1.116 Å

===== ETAPA 2: CB–CG =====
CG candidato: serial=8689 2HD2 LEU 578
  Distância CB–candidato = 1.527 Å
  Ângulo CA–CB–candidato = 119.419 °
  -> Fazendo swap de coordenadas entre CG e candidato.

===== ETAPA 3: CG–HG =====
Janela final CG–HG usada: 0.990 – 1.120 Å
  [HG] alvo HG(8682) <-> candidato 3HD2(8690): swap. dist_ref_antes=9.096 Å, dist_ref_depois=1.118 Å

===== ETAPA 4: CG–CD1 =====
CD1 candidato: serial=8737 HD2 LYS 581
  Distância

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### REESTRUTURA 579

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução/validação da cadeia lateral de TYR 579
seguindo as regras de distância e ângulo especificadas.

Resíduo alvo: TYR B 579.
Bloqueia (locked) todo ILE 576, THR 577 e LEU 578, além de N/CA/C/HN/HA/O/CB globais.

TOLERÂNCIAS DE DISTÂNCIA:
CB - HB1 entre 0.995 a 1.115 Å
CB - HB2 entre 0.995 a 1.115 Å
CB - CG  entre 1.40 a 1.60 Å
CG - HG  entre 0.995 a 1.115 Å
CG - CD1 entre 1.30 a 1.50 Å
CD1 - HD1 entre 0.995 a 1.115 Å
CD1 - CE1 entre 1.30 a 1.50 Å
CE1 - HE1 entre 0.995 a 1.115 Å
CE1 - CZ  entre 1.30 a 1.50 Å
CZ  - OH  entre 1.30 a 1.50 Å
OH  - HH  entre 0.850 a 1.050 Å
CG - CD2 entre 1.30 a 1.50 Å
CD2 - HD2 entre 0.995 a 1.115 Å
CD2 - CE2 entre 1.30 a 1.50 Å
CE2 - HE2 entre 0.995 a 1.115 Å
CE2 - CZ  entre 1.30 a 1.50 Å

ÂNGULOS:
- CA–CB–CG (sp3): alvo 110°, REF1 101–119°, REF2 104–116°, REF3 mais próximo de 110°.
- Aromáticos (CB–CG–CD1/2, CG–CD1–CE1, CG–CD2–CE2, CE1/CE2–CZ–OH):
  alvo 120°, REF1 114–126°, REF2 117–123°, REF3 mais próximo de 120°.
"""

import math

# ===========================
# Funções geométricas
# ===========================

def dist(a, b):
    return math.sqrt((a['x'] - b['x'])**2 +
                     (a['y'] - b['y'])**2 +
                     (a['z'] - b['z'])**2)

def angle(a, b, c):
    """
    Ângulo A-B-C (v1 = A-B, v2 = C-B) em graus.
    """
    v1 = (a['x'] - b['x'], a['y'] - b['y'], a['z'] - b['z'])
    v2 = (c['x'] - b['x'], c['y'] - b['y'], c['z'] - b['z'])
    n1 = math.sqrt(sum(v*v for v in v1))
    n2 = math.sqrt(sum(v*v for v in v2))
    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    dot = sum(v1[i]*v2[i] for i in range(3))
    cosang = max(-1.0, min(1.0, dot/(n1*n2)))
    return math.degrees(math.acos(cosang))

def same_coords(a, b, tol=1e-3):
    return (abs(a['x'] - b['x']) < tol and
            abs(a['y'] - b['y']) < tol and
            abs(a['z'] - b['z']) < tol)

# ===========================
# Parsing / escrita de PDB
# ===========================

def parse_pdb_lines(lines):
    atoms = []
    atom_line_idx = []
    for i, line in enumerate(lines):
        if line.startswith("ATOM") or line.startswith("HETATM"):
            try:
                rec = {
                    'line_idx': i,
                    'serial': int(line[6:11]),
                    'name':   line[12:16].strip(),
                    'altloc': line[16],
                    'resname': line[17:20].strip(),
                    'chain':  line[21].strip(),
                    'resseq': int(line[22:26]),
                    'icode':  line[26],
                    'x': float(line[30:38]),
                    'y': float(line[38:46]),
                    'z': float(line[46:54]),
                    'raw_line': line
                }
            except Exception:
                continue
            atoms.append(rec)
            atom_line_idx.append(i)
    return atoms, atom_line_idx

def update_pdb_lines(lines, atoms):
    for at in atoms:
        i = at['line_idx']
        line = lines[i]
        if len(line) < 54:
            line = line.ljust(54)
        new_line = (line[:30] +
                    f"{at['x']:8.3f}{at['y']:8.3f}{at['z']:8.3f}" +
                    line[54:])
        lines[i] = new_line
    return lines

# ===========================
# Utilitários de busca
# ===========================

def find_residue_atoms(atoms, resname, resseq, chain=None):
    idxs = []
    for i, a in enumerate(atoms):
        if a['resname'] == resname and a['resseq'] == resseq:
            if chain is None or a['chain'] == chain:
                idxs.append(i)
    return idxs

def build_name_index(atoms, residue_idxs):
    name2idx = {}
    for i in residue_idxs:
        nm = atoms[i]['name']
        if nm not in name2idx:
            name2idx[nm] = i
    return name2idx

def swap_coords(a, b):
    for coord in ('x', 'y', 'z'):
        a[coord], b[coord] = b[coord], a[coord]

# ===========================
# Seleção de pesado: CA–CB–cand (CG)
# ===========================

def escolher_peso_alifatico(atoms, locked,
                            idx_central_dist,  # CB
                            idx_CA, idx_CB,    # CA/CB
                            dmin, dmax,
                            ang1_min=101.0, ang1_max=119.0,
                            ang2_min=104.0, ang2_max=116.0,
                            alvo_angulo=110.0,
                            label="PESO"):
    """
    Seleciona CG de TYR usando:
      - distância CB–cand em [dmin,dmax]
      - ângulo CA–CB–cand com refinamentos 101–119 / 104–116 / mais próximo de 110.
    """
    ref_center = atoms[idx_central_dist]
    ref_CA = atoms[idx_CA]
    ref_CB = atoms[idx_CB]

    base = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_central_dist:
            continue
        d = dist(ref_center, a)
        if dmin <= d <= dmax:
            ang = angle(ref_CA, ref_CB, a)
            base.append((i, d, ang))

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} em {dmin:.2f}-{dmax:.2f} Å.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Seleção de pesado aromático: n1–center–cand ~120°
# ===========================

def escolher_peso_aromatico(atoms, locked,
                            idx_center, idx_n1,
                            dmin, dmax,
                            ang1_min=114.0, ang1_max=126.0,
                            ang2_min=117.0, ang2_max=123.0,
                            alvo_angulo=120.0,
                            label="AROM"):
    """
    Seleciona pesado aromático (CD1/CD2/CE1/CE2) com:
      - distância center–cand em [dmin,dmax]
      - ângulo n1–center–cand com refinamentos 114–126 / 117–123 / mais próximo de 120.
    """
    ref_center = atoms[idx_center]
    ref_n1 = atoms[idx_n1]

    base = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_center:
            continue
        d = dist(ref_center, a)
        if dmin <= d <= dmax:
            ang = angle(ref_n1, ref_center, a)
            base.append((i, d, ang))

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} em {dmin:.2f}-{dmax:.2f} Å.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Seleção CZ via CE1/CE2
# ===========================

def escolher_cz(atoms, locked,
                idx_CE1, idx_CE2,
                dmin=1.30, dmax=1.50,
                d_alvo=1.40):
    """
    Escolhe CZ como átomo que esteja entre CE1 e CE2 com:
      d(CE1,CZ) e d(CE2,CZ) em [dmin,dmax],
    minimizando |d1 - d_alvo| + |d2 - d_alvo|.
    """
    CE1 = atoms[idx_CE1]
    CE2 = atoms[idx_CE2]

    candidatos = []
    for i, a in enumerate(atoms):
        if i in locked:
            continue
        d1 = dist(CE1, a)
        d2 = dist(CE2, a)
        if dmin <= d1 <= dmax and dmin <= d2 <= dmax:
            score = abs(d1 - d_alvo) + abs(d2 - d_alvo)
            candidatos.append((i, d1, d2, score))

    if not candidatos:
        raise RuntimeError("Nenhum candidato encontrado para CZ respeitando CE1/CE2.")

    candidatos.sort(key=lambda t: t[3])
    return candidatos[0]  # (idx, d1, d2, score)

# ===========================
# Seleção OH via CZ + CE1/CE2 (ângulos ~120°)
# ===========================

def escolher_oh(atoms, locked,
                idx_CZ, idx_CE1, idx_CE2,
                dmin=1.30, dmax=1.50,
                ang1_min=114.0, ang1_max=126.0,
                ang2_min=117.0, ang2_max=123.0,
                alvo_angulo=120.0):
    """
    Seleciona OH com:
      - d(CZ,OH) em [dmin,dmax]
      - usa ângulo efetivo = média de CE1–CZ–cand e CE2–CZ–cand
        para aplicar os refinamentos 114–126 / 117–123 / ~120.
    """
    CZ = atoms[idx_CZ]
    CE1 = atoms[idx_CE1]
    CE2 = atoms[idx_CE2]

    base = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_CZ:
            continue
        d = dist(CZ, a)
        if dmin <= d <= dmax:
            ang1 = angle(CE1, CZ, a)
            ang2 = angle(CE2, CZ, a)
            ang_eff = 0.5 * (ang1 + ang2)
            base.append((i, d, ang_eff, ang1, ang2))

    if not base:
        raise RuntimeError("Nenhum candidato encontrado para OH em torno de CZ.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i,d,ang_eff,a1,a2) for (i,d,ang_eff,a1,a2) in prev_list
            if ang1_min <= ang_eff <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i,d,ang_eff,a1,a2) for (i,d,ang_eff,a1,a2) in prev_list
            if ang2_min <= ang_eff <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Janela dinâmica p/ hidrogênios
# ===========================

def escolher_vizinhos_dinamico(atoms, locked, idx_ref,
                               target_count,
                               dmin_init, dmax_init,
                               delta=0.005, max_iter=200,
                               label="H?"):
    """
    Janela dinâmica [dmin,dmax] para encontrar target_count vizinhos.
    > target_count: estreita (dmin += delta, dmax -= delta)
    < target_count: alarga (dmin -= delta, dmax += delta).
    Se não convergir, pega os target_count mais próximos.
    """
    ref = atoms[idx_ref]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        cand = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_ref:
                continue
            d = dist(ref, a)
            if dmin <= d <= dmax:
                cand.append((i, d))
        if len(cand) == target_count:
            return [i for (i, _) in cand], dmin, dmax
        elif len(cand) > target_count:
            dmin += delta
            dmax -= delta
            if dmin >= dmax:
                break
        else:  # len(cand) < target_count
            dmin = max(0.0, dmin - delta)
            dmax += delta

    # fallback
    cand_all = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_ref:
            continue
        d = dist(ref, a)
        cand_all.append((i, d))
    cand_all.sort(key=lambda t: t[1])
    chosen = cand_all[:target_count]
    return [i for (i, _) in chosen], dmin, dmax

def atribuir_coord_alvos(atoms, locked, idx_ref, alvo_idxs,
                         dmin_init, dmax_init,
                         target_count,
                         delta=0.005, max_iter=200,
                         label="H?"):
    """
    Usa janela dinâmica para achar target_count candidatos ao redor de idx_ref,
    e distribui coordenadas entre os átomos alvo.

    - Primeiro fixa alvos que já têm coord igual a alguma candidata (sem swap).
    - Depois faz swap para os demais alvos com candidatos restantes.
    - Todos os alvos entram em locked ao final.
    """
    ref = atoms[idx_ref]
    cand_idxs, dmin_final, dmax_final = escolher_vizinhos_dinamico(
        atoms, locked, idx_ref, target_count,
        dmin_init, dmax_init, delta, max_iter, label
    )
    cand_coords = {i: (atoms[i]['x'], atoms[i]['y'], atoms[i]['z']) for i in cand_idxs}
    used_cands = set()

    detalhes = {
        'ref': ref,
        'ref_idx': idx_ref,
        'janela_final': (dmin_final, dmax_final),
        'mapeamentos': []
    }

    # 1) Alvos já coincidindo com algum candidato
    for idx_alvo in alvo_idxs:
        alvo = atoms[idx_alvo]
        match = None
        for i_cand in cand_idxs:
            if i_cand in used_cands:
                continue
            cx, cy, cz = cand_coords[i_cand]
            if (abs(alvo['x'] - cx) < 1e-3 and
                abs(alvo['y'] - cy) < 1e-3 and
                abs(alvo['z'] - cz) < 1e-3):
                match = i_cand
                break
        if match is not None:
            used_cands.add(match)
            locked.add(idx_alvo)
            d = dist(ref, alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': match,
                'cand': atoms[match],
                'dist_ref_alvo': d,
                'swap_feito': False
            })

    # 2) Alvos restantes com swap
    for idx_alvo in alvo_idxs:
        if idx_alvo in locked:
            continue
        alvo = atoms[idx_alvo]
        cand_rest = [i for i in cand_idxs if i not in used_cands]
        if not cand_rest:
            locked.add(idx_alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': None,
                'cand': None,
                'dist_ref_alvo': dist(ref, alvo),
                'swap_feito': False
            })
            continue

        cand_rest.sort(key=lambda i: dist(ref, atoms[i]))
        i_cand = cand_rest[0]
        used_cands.add(i_cand)
        cand_atom = atoms[i_cand]

        d_before = dist(ref, alvo)
        if not same_coords(alvo, cand_atom):
            swap_coords(alvo, cand_atom)
            swap_feito = True
        else:
            swap_feito = False

        locked.add(idx_alvo)
        d_after = dist(ref, alvo)
        detalhes['mapeamentos'].append({
            'alvo_idx': idx_alvo,
            'alvo': alvo,
            'cand_idx': i_cand,
            'cand': cand_atom,
            'dist_ref_alvo_antes': d_before,
            'dist_ref_alvo_depois': d_after,
            'swap_feito': swap_feito
        })

    return detalhes

# ===========================
# Pipeline principal TYR 579
# ===========================

def reconstruir_tyr579_sidechain(pdb_filename,
                                 saida_filename="backbone_rebuilt_with_HNO_CB_TYR579.pdb",
                                 resname="TYR", resseq=579, chain="B"):
    with open(pdb_filename, 'r') as f:
        lines = f.read().splitlines()

    atoms, atom_line_idx = parse_pdb_lines(lines)
    print(f"PDB carregado: {pdb_filename}")
    print(f"Total de átomos lidos: {len(atoms)}")

    # locked inicial: N, CA, C, HN, HA, O, CB
    locked = set()
    base_locked_names = {"N", "CA", "C", "HN", "HA", "O", "CB"}
    for i, a in enumerate(atoms):
        if a['name'] in base_locked_names:
            locked.add(i)

    # Bloqueia todo ILE 576, THR 577, LEU 578
    for (rname, rseq) in [("ILE", 576), ("THR", 577), ("LEU", 578)]:
        idxs_res = find_residue_atoms(atoms, resname=rname, resseq=rseq, chain=chain)
        for i in idxs_res:
            locked.add(i)

    print(f"Átomos inicializados como imutáveis (locked): {len(locked)}")

    # Resíduo TYR 579
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    if not res_idxs:
        raise RuntimeError(f"Resíduo {resname} {chain} {resseq} não encontrado.")

    print(f"Reconstruindo cadeia lateral de {resname} {chain} {resseq}")
    name2idx = build_name_index(atoms, res_idxs)

    # Índices importantes
    idx_CA   = name2idx.get("CA")
    idx_CB   = name2idx.get("CB")
    idx_HB1  = name2idx.get("HB1")
    idx_HB2  = name2idx.get("HB2")
    idx_CG   = name2idx.get("CG")
    idx_HG   = name2idx.get("HG")
    idx_CD1  = name2idx.get("CD1")
    idx_HD1  = name2idx.get("HD1")
    idx_CE1  = name2idx.get("CE1")
    idx_HE1  = name2idx.get("HE1")
    idx_CZ   = name2idx.get("CZ")
    idx_OH   = name2idx.get("OH")
    idx_HH   = name2idx.get("HH")
    idx_CD2  = name2idx.get("CD2")
    idx_HD2  = name2idx.get("HD2")
    idx_CE2  = name2idx.get("CE2")
    idx_HE2  = name2idx.get("HE2")

    obrigatorios = [
        "CA", "CB", "HB1", "HB2", "CG", "HG",
        "CD1", "HD1", "CE1", "HE1",
        "CZ", "OH", "HH",
        "CD2", "HD2", "CE2", "HE2"
    ]
    for nm in obrigatorios:
        if nm not in name2idx:
            print(f"AVISO: átomo {nm} não encontrado em {resname} {chain} {resseq}.")

    # -----------------------
    # 1) CB–HB1/HB2
    # -----------------------
    print("\n===== ETAPA 1: CB–HB1/HB2 =====")
    alvo_hb = [idx for idx in (idx_HB1, idx_HB2) if idx is not None]
    if alvo_hb:
        det_hb = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CB,
            alvo_idxs=alvo_hb,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hb),
            delta=0.005, max_iter=200,
            label="HB"
        )
        print(f"Janela final CB–HB1/HB2 usada: {det_hb['janela_final'][0]:.3f} – {det_hb['janela_final'][1]:.3f} Å")
        for m in det_hb['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HB1/HB2 não encontrados, pulando etapa.")

    # -----------------------
    # 2) CB–CG (pesado alifático)
    # -----------------------
    print("\n===== ETAPA 2: CB–CG =====")
    cand_idx, d_cg, ang_cg = escolher_peso_alifatico(
        atoms, locked,
        idx_central_dist=idx_CB,
        idx_CA=idx_CA, idx_CB=idx_CB,
        dmin=1.40, dmax=1.60,
        ang1_min=101.0, ang1_max=119.0,
        ang2_min=104.0, ang2_max=116.0,
        alvo_angulo=110.0,
        label="CG"
    )
    cand = atoms[cand_idx]
    print(f"CG candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CB–candidato = {d_cg:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_cg:.3f} °")

    at_cg = atoms[idx_CG]
    if not same_coords(at_cg, cand):
        print("  -> Fazendo swap de coordenadas entre CG e candidato.")
        swap_coords(at_cg, cand)
    else:
        print("  -> CG já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CG)

    # -----------------------
    # 3) CG–HG
    # -----------------------
    print("\n===== ETAPA 3: CG–HG =====")
    if idx_HG is not None:
        det_hg = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CG,
            alvo_idxs=[idx_HG],
            dmin_init=0.995, dmax_init=1.115,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HG"
        )
        print(f"Janela final CG–HG usada: {det_hg['janela_final'][0]:.3f} – {det_hg['janela_final'][1]:.3f} Å")
        for m in det_hg['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HG não encontrado, pulando etapa.")

    # -----------------------
    # 4) CG–CD1 (aromático)
    # -----------------------
    print("\n===== ETAPA 4: CG–CD1 =====")
    cand_idx, d_cd1, ang_cd1 = escolher_peso_aromatico(
        atoms, locked,
        idx_center=idx_CG, idx_n1=idx_CB,
        dmin=1.30, dmax=1.50,
        ang1_min=114.0, ang1_max=126.0,
        ang2_min=117.0, ang2_max=123.0,
        alvo_angulo=120.0,
        label="CD1"
    )
    cand = atoms[cand_idx]
    print(f"CD1 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CG–candidato = {d_cd1:.3f} Å")
    print(f"  Ângulo CB–CG–candidato = {ang_cd1:.3f} °")

    at_cd1 = atoms[idx_CD1]
    if not same_coords(at_cd1, cand):
        print("  -> Fazendo swap de coordenadas entre CD1 e candidato.")
        swap_coords(at_cd1, cand)
    else:
        print("  -> CD1 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CD1)

    # -----------------------
    # 5) CD1–HD1
    # -----------------------
    print("\n===== ETAPA 5: CD1–HD1 =====")
    if idx_HD1 is not None:
        det_hd1 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CD1,
            alvo_idxs=[idx_HD1],
            dmin_init=0.995, dmax_init=1.115,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HD1"
        )
        print(f"Janela final CD1–HD1 usada: {det_hd1['janela_final'][0]:.3f} – {det_hd1['janela_final'][1]:.3f} Å")
        for m in det_hd1['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HD1] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HD1] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HD1] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HD1 não encontrado, pulando etapa.")

    # -----------------------
    # 6) CG–CD2 (aromático)
    # -----------------------
    print("\n===== ETAPA 6: CG–CD2 =====")
    cand_idx, d_cd2, ang_cd2 = escolher_peso_aromatico(
        atoms, locked,
        idx_center=idx_CG, idx_n1=idx_CB,
        dmin=1.30, dmax=1.50,
        ang1_min=114.0, ang1_max=126.0,
        ang2_min=117.0, ang2_max=123.0,
        alvo_angulo=120.0,
        label="CD2"
    )
    cand = atoms[cand_idx]
    print(f"CD2 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CG–candidato = {d_cd2:.3f} Å")
    print(f"  Ângulo CB–CG–candidato = {ang_cd2:.3f} °")

    at_cd2 = atoms[idx_CD2]
    if not same_coords(at_cd2, cand):
        print("  -> Fazendo swap de coordenadas entre CD2 e candidato.")
        swap_coords(at_cd2, cand)
    else:
        print("  -> CD2 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CD2)

    # -----------------------
    # 7) CD2–HD2
    # -----------------------
    print("\n===== ETAPA 7: CD2–HD2 =====")
    if idx_HD2 is not None:
        det_hd2 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CD2,
            alvo_idxs=[idx_HD2],
            dmin_init=0.995, dmax_init=1.115,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HD2"
        )
        print(f"Janela final CD2–HD2 usada: {det_hd2['janela_final'][0]:.3f} – {det_hd2['janela_final'][1]:.3f} Å")
        for m in det_hd2['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HD2] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HD2] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HD2] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HD2 não encontrado, pulando etapa.")

    # -----------------------
    # 8) CD1–CE1 (aromático)
    # -----------------------
    print("\n===== ETAPA 8: CD1–CE1 =====")
    cand_idx, d_ce1, ang_ce1 = escolher_peso_aromatico(
        atoms, locked,
        idx_center=idx_CD1, idx_n1=idx_CG,
        dmin=1.30, dmax=1.50,
        ang1_min=114.0, ang1_max=126.0,
        ang2_min=117.0, ang2_max=123.0,
        alvo_angulo=120.0,
        label="CE1"
    )
    cand = atoms[cand_idx]
    print(f"CE1 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CD1–candidato = {d_ce1:.3f} Å")
    print(f"  Ângulo CG–CD1–candidato = {ang_ce1:.3f} °")

    at_ce1 = atoms[idx_CE1]
    if not same_coords(at_ce1, cand):
        print("  -> Fazendo swap de coordenadas entre CE1 e candidato.")
        swap_coords(at_ce1, cand)
    else:
        print("  -> CE1 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CE1)

    # -----------------------
    # 9) CE1–HE1
    # -----------------------
    print("\n===== ETAPA 9: CE1–HE1 =====")
    if idx_HE1 is not None:
        det_he1 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CE1,
            alvo_idxs=[idx_HE1],
            dmin_init=0.995, dmax_init=1.115,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HE1"
        )
        print(f"Janela final CE1–HE1 usada: {det_he1['janela_final'][0]:.3f} – {det_he1['janela_final'][1]:.3f} Å")
        for m in det_he1['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HE1] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HE1] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HE1] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HE1 não encontrado, pulando etapa.")

    # -----------------------
    # 10) CD2–CE2 (aromático)
    # -----------------------
    print("\n===== ETAPA 10: CD2–CE2 =====")
    cand_idx, d_ce2, ang_ce2 = escolher_peso_aromatico(
        atoms, locked,
        idx_center=idx_CD2, idx_n1=idx_CG,
        dmin=1.30, dmax=1.50,
        ang1_min=114.0, ang1_max=126.0,
        ang2_min=117.0, ang2_max=123.0,
        alvo_angulo=120.0,
        label="CE2"
    )
    cand = atoms[cand_idx]
    print(f"CE2 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CD2–candidato = {d_ce2:.3f} Å")
    print(f"  Ângulo CG–CD2–candidato = {ang_ce2:.3f} °")

    at_ce2 = atoms[idx_CE2]
    if not same_coords(at_ce2, cand):
        print("  -> Fazendo swap de coordenadas entre CE2 e candidato.")
        swap_coords(at_ce2, cand)
    else:
        print("  -> CE2 já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CE2)

    # -----------------------
    # 11) CE2–HE2
    # -----------------------
    print("\n===== ETAPA 11: CE2–HE2 =====")
    if idx_HE2 is not None:
        det_he2 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CE2,
            alvo_idxs=[idx_HE2],
            dmin_init=0.995, dmax_init=1.115,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HE2"
        )
        print(f"Janela final CE2–HE2 usada: {det_he2['janela_final'][0]:.3f} – {det_he2['janela_final'][1]:.3f} Å")
        for m in det_he2['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HE2] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HE2] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HE2] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HE2 não encontrado, pulando etapa.")

    # -----------------------
    # 12) CE1/CE2–CZ (distância dupla)
    # -----------------------
    print("\n===== ETAPA 12: CE1/CE2–CZ =====")
    cand_idx, d1_cz, d2_cz, score_cz = escolher_cz(
        atoms, locked,
        idx_CE1=idx_CE1, idx_CE2=idx_CE2,
        dmin=1.30, dmax=1.50, d_alvo=1.40
    )
    cand = atoms[cand_idx]
    print(f"CZ candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Dist CE1–cand = {d1_cz:.3f} Å")
    print(f"  Dist CE2–cand = {d2_cz:.3f} Å")
    print(f"  Score CZ (|d1-1.40|+|d2-1.40|) = {score_cz:.3f}")

    at_cz = atoms[idx_CZ]
    if not same_coords(at_cz, cand):
        print("  -> Fazendo swap de coordenadas entre CZ e candidato.")
        swap_coords(at_cz, cand)
    else:
        print("  -> CZ já está usando essa coordenada (nenhum swap).")
    locked.add(idx_CZ)

    # -----------------------
    # 13) CZ–OH (aromático com CE1/CE2)
    # -----------------------
    print("\n===== ETAPA 13: CZ–OH =====")
    cand_idx, d_oh, ang_eff, ang_ce1, ang_ce2 = escolher_oh(
        atoms, locked,
        idx_CZ=idx_CZ, idx_CE1=idx_CE1, idx_CE2=idx_CE2,
        dmin=1.30, dmax=1.50,
        ang1_min=114.0, ang1_max=126.0,
        ang2_min=117.0, ang2_max=123.0,
        alvo_angulo=120.0
    )
    cand = atoms[cand_idx]
    print(f"OH candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Dist CZ–cand = {d_oh:.3f} Å")
    print(f"  Âng CE1–CZ–cand = {ang_ce1:.3f} °")
    print(f"  Âng CE2–CZ–cand = {ang_ce2:.3f} °")
    print(f"  Ângulo efetivo (média) = {ang_eff:.3f} °")

    at_oh = atoms[idx_OH]
    if not same_coords(at_oh, cand):
        print("  -> Fazendo swap de coordenadas entre OH e candidato.")
        swap_coords(at_oh, cand)
    else:
        print("  -> OH já está usando essa coordenada (nenhum swap).")
    locked.add(idx_OH)

    # -----------------------
    # 14) OH–HH
    # -----------------------
    print("\n===== ETAPA 14: OH–HH =====")
    if idx_HH is not None:
        det_hh = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_OH,
            alvo_idxs=[idx_HH],
            dmin_init=0.850, dmax_init=1.050,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HH"
        )
        print(f"Janela final OH–HH usada: {det_hh['janela_final'][0]:.3f} – {det_hh['janela_final'][1]:.3f} Å")
        for m in det_hh['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HH] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HH] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HH] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HH não encontrado, pulando etapa.")

    # ===========================
    # Escrita do novo PDB
    # ===========================
    lines_out = update_pdb_lines(lines, atoms)
    with open(saida_filename, 'w') as f:
        f.write("\n".join(lines_out) + "\n")
    print(f"\nNovo PDB escrito em: {saida_filename}")

    return saida_filename

# ===========================
# Bloco Colab
# ===========================

if __name__ == "__main__":
    try:
        from google.colab import files  # type: ignore
        print("Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_LEU578.pdb):")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo foi carregado.")
        pdb_in = list(uploaded.keys())[0]
        saida = "backbone_rebuilt_with_HNO_CB_TYR579.pdb"
        reconstruir_tyr579_sidechain(pdb_in, saida_filename=saida,
                                     resname="TYR", resseq=579, chain="B")
        files.download(saida)
    except ImportError:
        # Uso local fora do Colab
        pdb_in = "backbone_rebuilt_with_HNO_CB_LEU578.pdb"
        saida = "backbone_rebuilt_with_HNO_CB_TYR579.pdb"
        reconstruir_tyr579_sidechain(pdb_in, saida_filename=saida,
                                     resname="TYR", resseq=579, chain="B")


Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_LEU578.pdb):


Saving backbone_rebuilt_with_HNO_CB_LEU578.pdb to backbone_rebuilt_with_HNO_CB_LEU578 (2).pdb
PDB carregado: backbone_rebuilt_with_HNO_CB_LEU578 (2).pdb
Total de átomos lidos: 137
Átomos inicializados como imutáveis (locked): 85
Reconstruindo cadeia lateral de TYR B 579
AVISO: átomo HG não encontrado em TYR B 579.

===== ETAPA 1: CB–HB1/HB2 =====
Janela final CB–HB1/HB2 usada: 0.995 – 1.115 Å
  [HB] alvo HB1(8698) <-> candidato HZ3(8744): swap. dist_ref_antes=8.018 Å, dist_ref_depois=1.107 Å
  [HB] alvo HB2(8699) <-> candidato HH(8707): swap. dist_ref_antes=8.373 Å, dist_ref_depois=1.112 Å

===== ETAPA 2: CB–CG =====
CG candidato: serial=8708 CD2 TYR 579
  Distância CB–candidato = 1.479 Å
  Ângulo CA–CB–candidato = 113.651 °
  -> Fazendo swap de coordenadas entre CG e candidato.

===== ETAPA 3: CG–HG =====
  -> HG não encontrado, pulando etapa.

===== ETAPA 4: CG–CD1 =====
CD1 candidato: serial=8709 HD2 TYR 579
  Distância CG–candidato = 1.401 Å
  Ângulo CB–CG–candidato = 120.015 °
  -

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### REESTRUTURAR 580

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução/validação da cadeia lateral de CYS 580
seguindo as regras de distância e ângulo especificadas.

Resíduo alvo: CYS B 580.
Bloqueia (locked) todo ILE 576, THR 577, LEU 578, TYR 579,
além de N/CA/C/HN/HA/O/CB globais.

TOLERÂNCIAS DE DISTÂNCIA:
CB - HB1 entre 0.995 a 1.115 Å
CB - HB2 entre 0.995 a 1.115 Å
CB - SG  entre 1.60 a 1.90 Å
SG - HG1 entre 1.20 a 1.45 Å

ÂNGULO CA–CB–SG:
- REF1: 100–125°
- REF2: 104–121°
- REF3: mais próximo de 112.5°
Se em REF1 ou REF2 ficar com 0 candidatos (tendo ≥2 antes), volta à lista anterior e aplica REF3.

Para CB–SG, se não houver nenhum candidato em [1.60, 1.90],
aplicar min -= 0.005, max += 0.005 até aparecer pelo menos 1.

Para H:
- Se número de candidatos > número de hidrogênios: min += 0.005, max -= 0.005
- Se número de candidatos < número de hidrogênios: min -= 0.005, max += 0.005
e reavalia a cada rodada.
"""

import math

# ===========================
# Funções geométricas
# ===========================

def dist(a, b):
    return math.sqrt((a['x'] - b['x'])**2 +
                     (a['y'] - b['y'])**2 +
                     (a['z'] - b['z'])**2)

def angle(a, b, c):
    """
    Ângulo A-B-C (v1 = A-B, v2 = C-B) em graus.
    """
    v1 = (a['x'] - b['x'], a['y'] - b['y'], a['z'] - b['z'])
    v2 = (c['x'] - b['x'], c['y'] - b['y'], c['z'] - b['z'])
    n1 = math.sqrt(sum(v*v for v in v1))
    n2 = math.sqrt(sum(v*v for v in v2))
    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    dot = sum(v1[i]*v2[i] for i in range(3))
    cosang = max(-1.0, min(1.0, dot/(n1*n2)))
    return math.degrees(math.acos(cosang))

def same_coords(a, b, tol=1e-3):
    return (abs(a['x'] - b['x']) < tol and
            abs(a['y'] - b['y']) < tol and
            abs(a['z'] - b['z']) < tol)

# ===========================
# Parsing / escrita de PDB
# ===========================

def parse_pdb_lines(lines):
    atoms = []
    atom_line_idx = []
    for i, line in enumerate(lines):
        if line.startswith("ATOM") or line.startswith("HETATM"):
            try:
                rec = {
                    'line_idx': i,
                    'serial': int(line[6:11]),
                    'name':   line[12:16].strip(),
                    'altloc': line[16],
                    'resname': line[17:20].strip(),
                    'chain':  line[21].strip(),
                    'resseq': int(line[22:26]),
                    'icode':  line[26],
                    'x': float(line[30:38]),
                    'y': float(line[38:46]),
                    'z': float(line[46:54]),
                    'raw_line': line
                }
            except Exception:
                continue
            atoms.append(rec)
            atom_line_idx.append(i)
    return atoms, atom_line_idx

def update_pdb_lines(lines, atoms):
    for at in atoms:
        i = at['line_idx']
        line = lines[i]
        if len(line) < 54:
            line = line.ljust(54)
        new_line = (line[:30] +
                    f"{at['x']:8.3f}{at['y']:8.3f}{at['z']:8.3f}" +
                    line[54:])
        lines[i] = new_line
    return lines

# ===========================
# Utilitários de busca
# ===========================

def find_residue_atoms(atoms, resname, resseq, chain=None):
    idxs = []
    for i, a in enumerate(atoms):
        if a['resname'] == resname and a['resseq'] == resseq:
            if chain is None or a['chain'] == chain:
                idxs.append(i)
    return idxs

def build_name_index(atoms, residue_idxs):
    name2idx = {}
    for i in residue_idxs:
        nm = atoms[i]['name']
        if nm not in name2idx:
            name2idx[nm] = i
    return name2idx

def swap_coords(a, b):
    for coord in ('x', 'y', 'z'):
        a[coord], b[coord] = b[coord], a[coord]

# ===========================
# Seleção SG (CB–SG com distância dinâmica + ângulo CA–CB–SG)
# ===========================

def escolher_sg_cys(atoms, locked,
                    idx_CB, idx_CA,
                    dmin_init=1.60, dmax_init=1.90,
                    delta=0.005, max_iter=200,
                    ang1_min=100.0, ang1_max=125.0,
                    ang2_min=104.0, ang2_max=121.0,
                    alvo_angulo=112.5,
                    label="SG"):
    """
    Seleciona SG de CYS com:
      - distância CB–cand em [dmin,dmax], com expansão se nenhum candidato;
      - ângulo CA–CB–cand com refinamentos:
          REF1: 100–125°
          REF2: 104–121°
          REF3: mais próximo de 112.5°
      Se em REF1 ou REF2 a lista vai de >=2 para 0, volta para a lista anterior e aplica REF3.
    """
    CB = atoms[idx_CB]
    CA = atoms[idx_CA]
    dmin = dmin_init
    dmax = dmax_init

    base = []
    for _ in range(max_iter):
        base = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_CB:
                continue
            d = dist(CB, a)
            if dmin <= d <= dmax:
                ang = angle(CA, CB, a)
                base.append((i, d, ang))
        if base:
            break
        # nenhum candidato -> expandir tolerância
        dmin = max(0.0, dmin - delta)
        dmax += delta

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} mesmo após expandir a janela.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        # fallback: REF3 na lista anterior
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Janela dinâmica p/ hidrogênios
# ===========================

def escolher_vizinhos_dinamico(atoms, locked, idx_ref,
                               target_count,
                               dmin_init, dmax_init,
                               delta=0.005, max_iter=200,
                               label="H?"):
    """
    Janela dinâmica [dmin,dmax] para encontrar target_count vizinhos.
    > target_count: estreita (dmin += delta, dmax -= delta)
    < target_count: alarga (dmin -= delta, dmax += delta).
    Se não convergir, pega os target_count mais próximos.
    """
    ref = atoms[idx_ref]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        cand = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_ref:
                continue
            d = dist(ref, a)
            if dmin <= d <= dmax:
                cand.append((i, d))
        if len(cand) == target_count:
            return [i for (i, _) in cand], dmin, dmax
        elif len(cand) > target_count:
            dmin += delta
            dmax -= delta
            if dmin >= dmax:
                break
        else:  # len(cand) < target_count
            dmin = max(0.0, dmin - delta)
            dmax += delta

    # fallback
    cand_all = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_ref:
            continue
        d = dist(ref, a)
        cand_all.append((i, d))
    cand_all.sort(key=lambda t: t[1])
    chosen = cand_all[:target_count]
    return [i for (i, _) in chosen], dmin, dmax

def atribuir_coord_alvos(atoms, locked, idx_ref, alvo_idxs,
                         dmin_init, dmax_init,
                         target_count,
                         delta=0.005, max_iter=200,
                         label="H?"):
    """
    Usa janela dinâmica para achar target_count candidatos ao redor de idx_ref,
    e distribui coordenadas entre os átomos alvo.

    - Primeiro fixa alvos que já têm coord igual a alguma candidata (sem swap).
    - Depois faz swap para os demais alvos com candidatos restantes.
    - Todos os alvos entram em locked ao final.
    """
    ref = atoms[idx_ref]
    cand_idxs, dmin_final, dmax_final = escolher_vizinhos_dinamico(
        atoms, locked, idx_ref, target_count,
        dmin_init, dmax_init, delta, max_iter, label
    )
    cand_coords = {i: (atoms[i]['x'], atoms[i]['y'], atoms[i]['z']) for i in cand_idxs}
    used_cands = set()

    detalhes = {
        'ref': ref,
        'ref_idx': idx_ref,
        'janela_final': (dmin_final, dmax_final),
        'mapeamentos': []
    }

    # 1) Alvos já coincidindo com algum candidato
    for idx_alvo in alvo_idxs:
        alvo = atoms[idx_alvo]
        match = None
        for i_cand in cand_idxs:
            if i_cand in used_cands:
                continue
            cx, cy, cz = cand_coords[i_cand]
            if (abs(alvo['x'] - cx) < 1e-3 and
                abs(alvo['y'] - cy) < 1e-3 and
                abs(alvo['z'] - cz) < 1e-3):
                match = i_cand
                break
        if match is not None:
            used_cands.add(match)
            locked.add(idx_alvo)
            d = dist(ref, alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': match,
                'cand': atoms[match],
                'dist_ref_alvo': d,
                'swap_feito': False
            })

    # 2) Alvos restantes com swap
    for idx_alvo in alvo_idxs:
        if idx_alvo in locked:
            continue
        alvo = atoms[idx_alvo]
        cand_rest = [i for i in cand_idxs if i not in used_cands]
        if not cand_rest:
            locked.add(idx_alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': None,
                'cand': None,
                'dist_ref_alvo': dist(ref, alvo),
                'swap_feito': False
            })
            continue

        cand_rest.sort(key=lambda i: dist(ref, atoms[i]))
        i_cand = cand_rest[0]
        used_cands.add(i_cand)
        cand_atom = atoms[i_cand]

        d_before = dist(ref, alvo)
        if not same_coords(alvo, cand_atom):
            swap_coords(alvo, cand_atom)
            swap_feito = True
        else:
            swap_feito = False

        locked.add(idx_alvo)
        d_after = dist(ref, alvo)
        detalhes['mapeamentos'].append({
            'alvo_idx': idx_alvo,
            'alvo': alvo,
            'cand_idx': i_cand,
            'cand': cand_atom,
            'dist_ref_alvo_antes': d_before,
            'dist_ref_alvo_depois': d_after,
            'swap_feito': swap_feito
        })

    return detalhes

# ===========================
# Pipeline principal CYS 580
# ===========================

def reconstruir_cys580_sidechain(pdb_filename,
                                 saida_filename="backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb",
                                 resname="CYS", resseq=580, chain="B"):
    with open(pdb_filename, 'r') as f:
        lines = f.read().splitlines()

    atoms, atom_line_idx = parse_pdb_lines(lines)
    print(f"PDB carregado: {pdb_filename}")
    print(f"Total de átomos lidos: {len(atoms)}")

    # locked inicial: N, CA, C, HN, HA, O, CB
    locked = set()
    base_locked_names = {"N", "CA", "C", "HN", "HA", "O", "CB"}
    for i, a in enumerate(atoms):
        if a['name'] in base_locked_names:
            locked.add(i)

    # Bloqueia todo ILE 576, THR 577, LEU 578, TYR 579
    for (rname, rseq) in [("ILE", 576), ("THR", 577), ("LEU", 578), ("TYR", 579)]:
        idxs_res = find_residue_atoms(atoms, resname=rname, resseq=rseq, chain=chain)
        for i in idxs_res:
            locked.add(i)

    print(f"Átomos inicializados como imutáveis (locked): {len(locked)}")

    # Resíduo CYS 580
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    if not res_idxs:
        raise RuntimeError(f"Resíduo {resname} {chain} {resseq} não encontrado.")

    print(f"Reconstruindo cadeia lateral de {resname} {chain} {resseq}")
    name2idx = build_name_index(atoms, res_idxs)

    # Índices importantes
    idx_CA   = name2idx.get("CA")
    idx_CB   = name2idx.get("CB")
    idx_HB1  = name2idx.get("HB1")
    idx_HB2  = name2idx.get("HB2")
    idx_SG   = name2idx.get("SG")
    idx_HG1  = name2idx.get("HG1")

    obrigatorios = ["CA", "CB", "HB1", "HB2", "SG", "HG1"]
    for nm in obrigatorios:
        if nm not in name2idx:
            print(f"AVISO: átomo {nm} não encontrado em {resname} {chain} {resseq}.")

    # -----------------------
    # 1) CB–HB1/HB2
    # -----------------------
    print("\n===== ETAPA 1: CB–HB1/HB2 =====")
    alvo_hb = [idx for idx in (idx_HB1, idx_HB2) if idx is not None]
    if alvo_hb:
        det_hb = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CB,
            alvo_idxs=alvo_hb,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hb),
            delta=0.005, max_iter=200,
            label="HB"
        )
        print(f"Janela final CB–HB1/HB2 usada: {det_hb['janela_final'][0]:.3f} – {det_hb['janela_final'][1]:.3f} Å")
        for m in det_hb['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HB1/HB2 não encontrados, pulando etapa.")

    # -----------------------
    # 2) CB–SG com distância dinâmica + ângulo CA–CB–SG
    # -----------------------
    print("\n===== ETAPA 2: CB–SG =====")
    cand_idx, d_sg, ang_sg = escolher_sg_cys(
        atoms, locked,
        idx_CB=idx_CB, idx_CA=idx_CA,
        dmin_init=1.60, dmax_init=1.90,
        delta=0.005, max_iter=200,
        ang1_min=100.0, ang1_max=125.0,
        ang2_min=104.0, ang2_max=121.0,
        alvo_angulo=112.5,
        label="SG"
    )
    cand = atoms[cand_idx]
    print(f"SG candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
    print(f"  Distância CB–candidato = {d_sg:.3f} Å")
    print(f"  Ângulo CA–CB–candidato = {ang_sg:.3f} °")

    at_sg = atoms[idx_SG]
    if not same_coords(at_sg, cand):
        print("  -> Fazendo swap de coordenadas entre SG e candidato.")
        swap_coords(at_sg, cand)
    else:
        print("  -> SG já está usando essa coordenada (nenhum swap).")
    locked.add(idx_SG)

    # -----------------------
    # 3) SG–HG1
    # -----------------------
    print("\n===== ETAPA 3: SG–HG1 =====")
    if idx_HG1 is not None:
        det_hg1 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_SG,
            alvo_idxs=[idx_HG1],
            dmin_init=1.20, dmax_init=1.45,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HG1"
        )
        print(f"Janela final SG–HG1 usada: {det_hg1['janela_final'][0]:.3f} – {det_hg1['janela_final'][1]:.3f} Å")
        for m in det_hg1['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HG1] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HG1] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HG1] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HG1 não encontrado, pulando etapa.")

    # ===========================
    # Escrita do novo PDB
    # ===========================
    lines_out = update_pdb_lines(lines, atoms)
    with open(saida_filename, 'w') as f:
        f.write("\n".join(lines_out) + "\n")
    print(f"\nNovo PDB escrito em: {saida_filename}")

    return saida_filename

# ===========================
# Bloco Colab
# ===========================

if __name__ == "__main__":
    try:
        from google.colab import files  # type: ignore
        print("Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_TYR579.pdb):")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo foi carregado.")
        pdb_in = list(uploaded.keys())[0]
        saida = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb"
        reconstruir_cys580_sidechain(pdb_in, saida_filename=saida,
                                     resname="CYS", resseq=580, chain="B")
        files.download(saida)
    except ImportError:
        # Uso local fora do Colab
        pdb_in = "backbone_rebuilt_with_HNO_CB_TYR579.pdb"
        saida = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb"
        reconstruir_cys580_sidechain(pdb_in, saida_filename=saida,
                                     resname="CYS", resseq=580, chain="B")


Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_TYR579.pdb):


Saving backbone_rebuilt_with_HNO_CB_TYR579.pdb to backbone_rebuilt_with_HNO_CB_TYR579 (2).pdb
PDB carregado: backbone_rebuilt_with_HNO_CB_TYR579 (2).pdb
Total de átomos lidos: 137
Átomos inicializados como imutáveis (locked): 99
Reconstruindo cadeia lateral de CYS B 580

===== ETAPA 1: CB–HB1/HB2 =====
Janela final CB–HB1/HB2 usada: 0.990 – 1.120 Å
  [HB] alvo HB1(8719) <-> candidato HG2(8734): swap. dist_ref_antes=8.849 Å, dist_ref_depois=1.115 Å
  [HB] alvo HB2(8720) <-> candidato CD(8735): swap. dist_ref_antes=9.826 Å, dist_ref_depois=1.115 Å

===== ETAPA 2: CB–SG =====
SG candidato: serial=8737 HD2 LYS 581
  Distância CB–candidato = 1.823 Å
  Ângulo CA–CB–candidato = 113.672 °
  -> Fazendo swap de coordenadas entre SG e candidato.

===== ETAPA 3: SG–HG1 =====
Janela final SG–HG1 usada: 1.200 – 1.450 Å
  [HG1] alvo HG1(8722) <-> candidato HB2(8731): swap. dist_ref_antes=6.274 Å, dist_ref_depois=1.320 Å

Novo PDB escrito em: backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### REESTRUTURAR O 581

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução/validação da cadeia lateral de LYS 581
seguindo as regras de distância e ângulo especificadas.

Resíduo alvo: LYS B 581.
Bloqueia (locked) todo ILE 576, THR 577, LEU 578, TYR 579, CYS 580,
além de N/CA/C/HN/HA/O/CB globais.

TOLERÂNCIAS DE DISTÂNCIA:
CB - HB1 entre 0.995 a 1.115 Å
CB - HB2 entre 0.995 a 1.115 Å
CB - CG  entre 1.40 a 1.60 Å
CG - HG1 entre 0.995 a 1.115 Å
CG - HG2 entre 0.995 a 1.115 Å
CG - CD  entre 1.40 a 1.60 Å
CD - HD1 entre 0.995 a 1.115 Å
CD - HD2 entre 0.995 a 1.115 Å
CD - CE  entre 1.40 a 1.60 Å
CE - HE1 entre 0.995 a 1.115 Å
CE - HE2 entre 0.995 a 1.115 Å
CE - NZ  entre 1.38 a 1.58 Å
NZ - HZ1 entre 0.900 a 1.100 Å
NZ - HZ2 entre 0.900 a 1.100 Å
NZ - HZ3 entre 0.900 a 1.100 Å

HIDROGÊNIOS:
Todos C (CB, CG, CD, CE) ligam com 2 H
NZ liga com 3 H (HZ1, HZ2, HZ3)

Janela dinâmica de H:
- Se #candidatos > #H: dmin += 0.005; dmax -= 0.005
- Se #candidatos < #H: dmin -= 0.005; dmax += 0.005

ÂNGULOS PESADOS:
1) CA–CB–CG, CB–CG–CD, CG–CD–CE:
   - REF1: 100–127°
   - REF2: 103–124°
   - REF3: mais próximo de 113.6°
   Se em REF1 ou REF2 a lista cai de >=2 para 0, volta à lista anterior e aplica REF3.

2) CD–CE–NZ:
   - REF1: 95–125°
   - REF2: 100–120°
   - REF3: mais próximo de 110.0°
   Mesma lógica de fallback.

Para CB–CG, CG–CD, CD–CE, CE–NZ:
- Se NUM DE CANDIDATO = 0 na janela inicial:
  dmin -= 0.005; dmax += 0.005 e reavalia (até max_iter).
"""

import math

# ===========================
# Funções geométricas
# ===========================

def dist(a, b):
    return math.sqrt((a['x'] - b['x'])**2 +
                     (a['y'] - b['y'])**2 +
                     (a['z'] - b['z'])**2)

def angle(a, b, c):
    """
    Ângulo A-B-C (v1 = A-B, v2 = C-B) em graus.
    """
    v1 = (a['x'] - b['x'], a['y'] - b['y'], a['z'] - b['z'])
    v2 = (c['x'] - b['x'], c['y'] - b['y'], c['z'] - b['z'])
    n1 = math.sqrt(sum(v*v for v in v1))
    n2 = math.sqrt(sum(v*v for v in v2))
    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    dot = sum(v1[i]*v2[i] for i in range(3))
    cosang = max(-1.0, min(1.0, dot/(n1*n2)))
    return math.degrees(math.acos(cosang))

def same_coords(a, b, tol=1e-3):
    return (abs(a['x'] - b['x']) < tol and
            abs(a['y'] - b['y']) < tol and
            abs(a['z'] - b['z']) < tol)

# ===========================
# Parsing / escrita de PDB
# ===========================

def parse_pdb_lines(lines):
    atoms = []
    atom_line_idx = []
    for i, line in enumerate(lines):
        if line.startswith("ATOM") or line.startswith("HETATM"):
            try:
                rec = {
                    'line_idx': i,
                    'serial': int(line[6:11]),
                    'name':   line[12:16].strip(),
                    'altloc': line[16],
                    'resname': line[17:20].strip(),
                    'chain':  line[21].strip(),
                    'resseq': int(line[22:26]),
                    'icode':  line[26],
                    'x': float(line[30:38]),
                    'y': float(line[38:46]),
                    'z': float(line[46:54]),
                    'raw_line': line
                }
            except Exception:
                continue
            atoms.append(rec)
            atom_line_idx.append(i)
    return atoms, atom_line_idx

def update_pdb_lines(lines, atoms):
    for at in atoms:
        i = at['line_idx']
        line = lines[i]
        if len(line) < 54:
            line = line.ljust(54)
        new_line = (line[:30] +
                    f"{at['x']:8.3f}{at['y']:8.3f}{at['z']:8.3f}" +
                    line[54:])
        lines[i] = new_line
    return lines

# ===========================
# Utilitários de busca
# ===========================

def find_residue_atoms(atoms, resname, resseq, chain=None):
    idxs = []
    for i, a in enumerate(atoms):
        if a['resname'] == resname and a['resseq'] == resseq:
            if chain is None or a['chain'] == chain:
                idxs.append(i)
    return idxs

def build_name_index(atoms, residue_idxs):
    name2idx = {}
    for i in residue_idxs:
        nm = atoms[i]['name']
        if nm not in name2idx:
            name2idx[nm] = i
    return name2idx

def swap_coords(a, b):
    for coord in ('x', 'y', 'z'):
        a[coord], b[coord] = b[coord], a[coord]

# ===========================
# Seleção de pesado (CB–CG, CG–CD, CG–CD–CE)
# ===========================

def escolher_peso_lys(atoms, locked,
                      idx_center, idx_prev,
                      dmin_init, dmax_init,
                      delta=0.005, max_iter=200,
                      ang1_min=100.0, ang1_max=127.0,
                      ang2_min=103.0, ang2_max=124.0,
                      alvo_angulo=113.6,
                      label="PESO"):
    """
    Seleciona pesado de LYS (CG, CD, CE) com:
      - distância center–cand em [dmin,dmax], expandindo se nenhum candidato;
      - ângulo prev–center–cand com refinamentos:
          REF1: [ang1_min, ang1_max]
          REF2: [ang2_min, ang2_max]
          REF3: mais próximo de alvo_angulo
      Se REF1 ou REF2 geram 0 candidatos saindo de uma lista >=2, volta à lista anterior e aplica REF3.
    """
    center = atoms[idx_center]
    prev   = atoms[idx_prev]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        base = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_center:
                continue
            d = dist(center, a)
            if dmin <= d <= dmax:
                ang = angle(prev, center, a)
                base.append((i, d, ang))
        if base:
            break
        # nenhum candidato -> expandir a janela
        dmin = max(0.0, dmin - delta)
        dmax += delta

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} mesmo após expandir janela.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

def escolher_peso_nz(atoms, locked,
                     idx_center, idx_prev,
                     dmin_init=1.38, dmax_init=1.58,
                     delta=0.005, max_iter=200,
                     ang1_min=95.0, ang1_max=125.0,
                     ang2_min=100.0, ang2_max=120.0,
                     alvo_angulo=110.0,
                     label="NZ"):
    """
    Seleciona NZ em LYS com:
      - distância CE–cand em [dmin,dmax], expandindo se nenhum candidato;
      - ângulo CD–CE–cand com refinamentos:
          REF1: 95–125°
          REF2: 100–120°
          REF3: mais próximo de 110°
      Se REF1 ou REF2 geram 0 candidatos saindo de uma lista >=2, volta à lista anterior e aplica REF3.
    """
    center = atoms[idx_center]  # CE
    prev   = atoms[idx_prev]    # CD
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        base = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_center:
                continue
            d = dist(center, a)
            if dmin <= d <= dmax:
                ang = angle(prev, center, a)
                base.append((i, d, ang))
        if base:
            break
        dmin = max(0.0, dmin - delta)
        dmax += delta

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} mesmo após expandir janela.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Janela dinâmica p/ hidrogênios
# ===========================

def escolher_vizinhos_dinamico(atoms, locked, idx_ref,
                               target_count,
                               dmin_init, dmax_init,
                               delta=0.005, max_iter=200,
                               label="H?"):
    """
    Janela dinâmica [dmin,dmax] para encontrar target_count vizinhos.
    > target_count: estreita (dmin += delta, dmax -= delta)
    < target_count: alarga (dmin -= delta, dmax += delta).
    Se não convergir, pega os target_count mais próximos.
    """
    ref = atoms[idx_ref]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        cand = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_ref:
                continue
            d = dist(ref, a)
            if dmin <= d <= dmax:
                cand.append((i, d))
        if len(cand) == target_count:
            return [i for (i, _) in cand], dmin, dmax
        elif len(cand) > target_count:
            dmin += delta
            dmax -= delta
            if dmin >= dmax:
                break
        else:  # len(cand) < target_count
            dmin = max(0.0, dmin - delta)
            dmax += delta

    # fallback
    cand_all = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_ref:
            continue
        d = dist(ref, a)
        cand_all.append((i, d))
    cand_all.sort(key=lambda t: t[1])
    chosen = cand_all[:target_count]
    return [i for (i, _) in chosen], dmin, dmax

def atribuir_coord_alvos(atoms, locked, idx_ref, alvo_idxs,
                         dmin_init, dmax_init,
                         target_count,
                         delta=0.005, max_iter=200,
                         label="H?"):
    """
    Usa janela dinâmica para achar target_count candidatos ao redor de idx_ref,
    e distribui coordenadas entre os átomos alvo.

    - Primeiro fixa alvos que já têm coord igual a alguma candidata (sem swap).
    - Depois faz swap para os demais alvos com candidatos restantes.
    - Todos os alvos entram em locked ao final.
    """
    ref = atoms[idx_ref]
    cand_idxs, dmin_final, dmax_final = escolher_vizinhos_dinamico(
        atoms, locked, idx_ref, target_count,
        dmin_init, dmax_init, delta, max_iter, label
    )
    cand_coords = {i: (atoms[i]['x'], atoms[i]['y'], atoms[i]['z']) for i in cand_idxs}
    used_cands = set()

    detalhes = {
        'ref': ref,
        'ref_idx': idx_ref,
        'janela_final': (dmin_final, dmax_final),
        'mapeamentos': []
    }

    # 1) Alvos já coincidindo com algum candidato
    for idx_alvo in alvo_idxs:
        alvo = atoms[idx_alvo]
        match = None
        for i_cand in cand_idxs:
            if i_cand in used_cands:
                continue
            cx, cy, cz = cand_coords[i_cand]
            if (abs(alvo['x'] - cx) < 1e-3 and
                abs(alvo['y'] - cy) < 1e-3 and
                abs(alvo['z'] - cz) < 1e-3):
                match = i_cand
                break
        if match is not None:
            used_cands.add(match)
            locked.add(idx_alvo)
            d = dist(ref, alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': match,
                'cand': atoms[match],
                'dist_ref_alvo': d,
                'swap_feito': False
            })

    # 2) Alvos restantes com swap
    for idx_alvo in alvo_idxs:
        if idx_alvo in locked:
            continue
        alvo = atoms[idx_alvo]
        cand_rest = [i for i in cand_idxs if i not in used_cands]
        if not cand_rest:
            locked.add(idx_alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': None,
                'cand': None,
                'dist_ref_alvo': dist(ref, alvo),
                'swap_feito': False
            })
            continue

        cand_rest.sort(key=lambda i: dist(ref, atoms[i]))
        i_cand = cand_rest[0]
        used_cands.add(i_cand)
        cand_atom = atoms[i_cand]

        d_before = dist(ref, alvo)
        if not same_coords(alvo, cand_atom):
            swap_coords(alvo, cand_atom)
            swap_feito = True
        else:
            swap_feito = False

        locked.add(idx_alvo)
        d_after = dist(ref, alvo)
        detalhes['mapeamentos'].append({
            'alvo_idx': idx_alvo,
            'alvo': alvo,
            'cand_idx': i_cand,
            'cand': cand_atom,
            'dist_ref_alvo_antes': d_before,
            'dist_ref_alvo_depois': d_after,
            'swap_feito': swap_feito
        })

    return detalhes

# ===========================
# Pipeline principal LYS 581
# ===========================

def reconstruir_lys581_sidechain(pdb_filename,
                                 saida_filename="backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581.pdb",
                                 resname="LYS", resseq=581, chain="B"):
    with open(pdb_filename, 'r') as f:
        lines = f.read().splitlines()

    atoms, atom_line_idx = parse_pdb_lines(lines)
    print(f"PDB carregado: {pdb_filename}")
    print(f"Total de átomos lidos: {len(atoms)}")

    # locked inicial: N, CA, C, HN, HA, O, CB
    locked = set()
    base_locked_names = {"N", "CA", "C", "HN", "HA", "O", "CB"}
    for i, a in enumerate(atoms):
        if a['name'] in base_locked_names:
            locked.add(i)

    # Bloqueia todo ILE 576, THR 577, LEU 578, TYR 579, CYS 580
    for (rname, rseq) in [("ILE", 576),
                          ("THR", 577),
                          ("LEU", 578),
                          ("TYR", 579),
                          ("CYS", 580)]:
        idxs_res = find_residue_atoms(atoms, resname=rname, resseq=rseq, chain=chain)
        for i in idxs_res:
            locked.add(i)

    print(f"Átomos inicializados como imutáveis (locked): {len(locked)}")

    # Resíduo LYS 581
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    if not res_idxs:
        raise RuntimeError(f"Resíduo {resname} {chain} {resseq} não encontrado.")

    print(f"Reconstruindo cadeia lateral de {resname} {chain} {resseq}")
    name2idx = build_name_index(atoms, res_idxs)

    # Índices importantes
    idx_CA   = name2idx.get("CA")
    idx_CB   = name2idx.get("CB")
    idx_HB1  = name2idx.get("HB1")
    idx_HB2  = name2idx.get("HB2")
    idx_CG   = name2idx.get("CG")
    idx_HG1  = name2idx.get("HG1")
    idx_HG2  = name2idx.get("HG2")
    idx_CD   = name2idx.get("CD")
    idx_HD1  = name2idx.get("HD1")
    idx_HD2  = name2idx.get("HD2")
    idx_CE   = name2idx.get("CE")
    idx_HE1  = name2idx.get("HE1")
    idx_HE2  = name2idx.get("HE2")
    idx_NZ   = name2idx.get("NZ")
    idx_HZ1  = name2idx.get("HZ1")
    idx_HZ2  = name2idx.get("HZ2")
    idx_HZ3  = name2idx.get("HZ3")

    obrigatorios = [
        "CA", "CB", "HB1", "HB2", "CG", "HG1", "HG2",
        "CD", "HD1", "HD2", "CE", "HE1", "HE2",
        "NZ", "HZ1", "HZ2", "HZ3"
    ]
    for nm in obrigatorios:
        if nm not in name2idx:
            print(f"AVISO: átomo {nm} não encontrado em {resname} {chain} {resseq}.")

    # -----------------------
    # 1) CB–HB1/HB2
    # -----------------------
    print("\n===== ETAPA 1: CB–HB1/HB2 =====")
    alvo_hb = [idx for idx in (idx_HB1, idx_HB2) if idx is not None]
    if alvo_hb:
        det_hb = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CB,
            alvo_idxs=alvo_hb,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hb),
            delta=0.005, max_iter=200,
            label="HB"
        )
        print(f"Janela final CB–HB1/HB2 usada: {det_hb['janela_final'][0]:.3f} – {det_hb['janela_final'][1]:.3f} Å")
        for m in det_hb['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HB1/HB2 não encontrados, pulando etapa.")

    # -----------------------
    # 2) CB–CG (pesado)
    # -----------------------
    print("\n===== ETAPA 2: CB–CG =====")
    if idx_CG is not None and idx_CB is not None and idx_CA is not None:
        cand_idx, d_cg, ang_cg = escolher_peso_lys(
            atoms, locked,
            idx_center=idx_CB, idx_prev=idx_CA,
            dmin_init=1.40, dmax_init=1.60,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=127.0,
            ang2_min=103.0, ang2_max=124.0,
            alvo_angulo=113.6,
            label="CG"
        )
        cand = atoms[cand_idx]
        print(f"CG candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CB–candidato = {d_cg:.3f} Å")
        print(f"  Ângulo CA–CB–candidato = {ang_cg:.3f} °")

        at_cg = atoms[idx_CG]
        if not same_coords(at_cg, cand):
            print("  -> Fazendo swap de coordenadas entre CG e candidato.")
            swap_coords(at_cg, cand)
        else:
            print("  -> CG já está usando essa coordenada (nenhum swap).")
        locked.add(idx_CG)
    else:
        print("  -> CA/CB/CG ausente(s), pulando etapa CB–CG.")

    # -----------------------
    # 3) CG–HG1/HG2
    # -----------------------
    print("\n===== ETAPA 3: CG–HG1/HG2 =====")
    alvo_hg = [idx for idx in (idx_HG1, idx_HG2) if idx is not None]
    if alvo_hg and idx_CG is not None:
        det_hg = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CG,
            alvo_idxs=alvo_hg,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hg),
            delta=0.005, max_iter=200,
            label="HG"
        )
        print(f"Janela final CG–HG1/HG2 usada: {det_hg['janela_final'][0]:.3f} – {det_hg['janela_final'][1]:.3f} Å")
        for m in det_hg['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HG1/HG2 não encontrados ou CG ausente, pulando etapa.")

    # -----------------------
    # 4) CG–CD (pesado)
    # -----------------------
    print("\n===== ETAPA 4: CG–CD =====")
    if idx_CD is not None and idx_CG is not None and idx_CB is not None:
        cand_idx, d_cd, ang_cd = escolher_peso_lys(
            atoms, locked,
            idx_center=idx_CG, idx_prev=idx_CB,
            dmin_init=1.40, dmax_init=1.60,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=127.0,
            ang2_min=103.0, ang2_max=124.0,
            alvo_angulo=113.6,
            label="CD"
        )
        cand = atoms[cand_idx]
        print(f"CD candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CG–candidato = {d_cd:.3f} Å")
        print(f"  Ângulo CB–CG–candidato = {ang_cd:.3f} °")

        at_cd = atoms[idx_CD]
        if not same_coords(at_cd, cand):
            print("  -> Fazendo swap de coordenadas entre CD e candidato.")
            swap_coords(at_cd, cand)
        else:
            print("  -> CD já está usando essa coordenada (nenhum swap).")
        locked.add(idx_CD)
    else:
        print("  -> CB/CG/CD ausente(s), pulando etapa CG–CD.")

    # -----------------------
    # 5) CD–HD1/HD2
    # -----------------------
    print("\n===== ETAPA 5: CD–HD1/HD2 =====")
    alvo_hd = [idx for idx in (idx_HD1, idx_HD2) if idx is not None]
    if alvo_hd and idx_CD is not None:
        det_hd = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CD,
            alvo_idxs=alvo_hd,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hd),
            delta=0.005, max_iter=200,
            label="HD"
        )
        print(f"Janela final CD–HD1/HD2 usada: {det_hd['janela_final'][0]:.3f} – {det_hd['janela_final'][1]:.3f} Å")
        for m in det_hd['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HD] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HD] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HD] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HD1/HD2 não encontrados ou CD ausente, pulando etapa.")

    # -----------------------
    # 6) CD–CE (pesado)
    # -----------------------
    print("\n===== ETAPA 6: CD–CE =====")
    if idx_CE is not None and idx_CD is not None and idx_CG is not None:
        cand_idx, d_ce, ang_ce = escolher_peso_lys(
            atoms, locked,
            idx_center=idx_CD, idx_prev=idx_CG,
            dmin_init=1.40, dmax_init=1.60,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=127.0,
            ang2_min=103.0, ang2_max=124.0,
            alvo_angulo=113.6,
            label="CE"
        )
        cand = atoms[cand_idx]
        print(f"CE candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CD–candidato = {d_ce:.3f} Å")
        print(f"  Ângulo CG–CD–candidato = {ang_ce:.3f} °")

        at_ce = atoms[idx_CE]
        if not same_coords(at_ce, cand):
            print("  -> Fazendo swap de coordenadas entre CE e candidato.")
            swap_coords(at_ce, cand)
        else:
            print("  -> CE já está usando essa coordenada (nenhum swap).")
        locked.add(idx_CE)
    else:
        print("  -> CG/CD/CE ausente(s), pulando etapa CD–CE.")

    # -----------------------
    # 7) CE–HE1/HE2
    # -----------------------
    print("\n===== ETAPA 7: CE–HE1/HE2 =====")
    alvo_he = [idx for idx in (idx_HE1, idx_HE2) if idx is not None]
    if alvo_he and idx_CE is not None:
        det_he = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CE,
            alvo_idxs=alvo_he,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_he),
            delta=0.005, max_iter=200,
            label="HE"
        )
        print(f"Janela final CE–HE1/HE2 usada: {det_he['janela_final'][0]:.3f} – {det_he['janela_final'][1]:.3f} Å")
        for m in det_he['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HE] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HE] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HE] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HE1/HE2 não encontrados ou CE ausente, pulando etapa.")

    # -----------------------
    # 8) CE–NZ (pesado)
    # -----------------------
    print("\n===== ETAPA 8: CE–NZ =====")
    if idx_NZ is not None and idx_CE is not None and idx_CD is not None:
        cand_idx, d_nz, ang_nz = escolher_peso_nz(
            atoms, locked,
            idx_center=idx_CE, idx_prev=idx_CD,
            dmin_init=1.38, dmax_init=1.58,
            delta=0.005, max_iter=200,
            ang1_min=95.0,  ang1_max=125.0,
            ang2_min=100.0, ang2_max=120.0,
            alvo_angulo=110.0,
            label="NZ"
        )
        cand = atoms[cand_idx]
        print(f"NZ candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CE–candidato = {d_nz:.3f} Å")
        print(f"  Ângulo CD–CE–candidato = {ang_nz:.3f} °")

        at_nz = atoms[idx_NZ]
        if not same_coords(at_nz, cand):
            print("  -> Fazendo swap de coordenadas entre NZ e candidato.")
            swap_coords(at_nz, cand)
        else:
            print("  -> NZ já está usando essa coordenada (nenhum swap).")
        locked.add(idx_NZ)
    else:
        print("  -> CD/CE/NZ ausente(s), pulando etapa CE–NZ.")

    # -----------------------
    # 9) NZ–HZ1/HZ2/HZ3
    # -----------------------
    print("\n===== ETAPA 9: NZ–HZ1/HZ2/HZ3 =====")
    alvo_hz = [idx for idx in (idx_HZ1, idx_HZ2, idx_HZ3) if idx is not None]
    if alvo_hz and idx_NZ is not None:
        det_hz = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_NZ,
            alvo_idxs=alvo_hz,
            dmin_init=0.900, dmax_init=1.100,
            target_count=len(alvo_hz),
            delta=0.005, max_iter=200,
            label="HZ"
        )
        print(f"Janela final NZ–HZ1/HZ2/HZ3 usada: {det_hz['janela_final'][0]:.3f} – {det_hz['janela_final'][1]:.3f} Å")
        for m in det_hz['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HZ] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HZ] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HZ] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HZ1/HZ2/HZ3 não encontrados ou NZ ausente, pulando etapa.")

    # ===========================
    # Escrita do novo PDB
    # ===========================
    lines_out = update_pdb_lines(lines, atoms)
    with open(saida_filename, 'w') as f:
        f.write("\n".join(lines_out) + "\n")
    print(f"\nNovo PDB escrito em: {saida_filename}")

    return saida_filename

# ===========================
# Bloco Colab
# ===========================

if __name__ == "__main__":
    try:
        from google.colab import files  # type: ignore
        print("Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb):")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo foi carregado.")
        pdb_in = list(uploaded.keys())[0]
        saida = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581.pdb"
        reconstruir_lys581_sidechain(pdb_in, saida_filename=saida,
                                     resname="LYS", resseq=581, chain="B")
        files.download(saida)
    except ImportError:
        # Uso local fora do Colab
        pdb_in = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb"
        saida = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581.pdb"
        reconstruir_lys581_sidechain(pdb_in, saida_filename=saida,
                                     resname="LYS", resseq=581, chain="B")


Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb):


Saving backbone_rebuilt_with_HNO_CB_TYR579_CYS580.pdb to backbone_rebuilt_with_HNO_CB_TYR579_CYS580 (1).pdb
PDB carregado: backbone_rebuilt_with_HNO_CB_TYR579_CYS580 (1).pdb
Total de átomos lidos: 137
Átomos inicializados como imutáveis (locked): 103
Reconstruindo cadeia lateral de LYS B 581

===== ETAPA 1: CB–HB1/HB2 =====
Janela final CB–HB1/HB2 usada: 0.995 – 1.115 Å
  [HB] alvo HB1(8730) <-> candidato CE(8738): swap. dist_ref_antes=5.925 Å, dist_ref_depois=1.103 Å
  [HB] alvo HB2(8731) <-> candidato HE1(8739): swap. dist_ref_antes=4.884 Å, dist_ref_depois=1.105 Å

===== ETAPA 2: CB–CG =====
CG candidato: serial=8740 HE2 LYS 581
  Distância CB–candidato = 1.574 Å
  Ângulo CA–CB–candidato = 117.069 °
  -> Fazendo swap de coordenadas entre CG e candidato.

===== ETAPA 3: CG–HG1/HG2 =====
Janela final CG–HG1/HG2 usada: 0.990 – 1.120 Å
  [HG] alvo HG1(8733) <-> candidato HZ1(8742): swap. dist_ref_antes=5.139 Å, dist_ref_depois=1.111 Å
  [HG] alvo HG2(8734) <-> candidato NZ(8741): swap. 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### REESTRUTURA 582

In [ ]:
# -*- coding: utf-8 -*-
"""
Reconstrução/validação da cadeia lateral de ARG 582
seguindo as regras de distância e ângulo especificadas.

Resíduo alvo: ARG B 582.
Bloqueia (locked) todo ILE 576, THR 577, LEU 578, TYR 579, CYS 580, LYS 581,
além de N/CA/C/HN/HA/O/CB globais.

TOLERÂNCIAS DE DISTÂNCIA:
CB - HB1 entre 0.995 a 1.115 Å
CB - HB2 entre 0.995 a 1.115 Å
CB - CG  entre 1.40 a 1.60 Å
CG - HG1 entre 0.995 a 1.115 Å
CG - HG2 entre 0.995 a 1.115 Å
CG - CD  entre 1.40 a 1.60 Å
CD - HD1 entre 0.995 a 1.115 Å
CD - HD2 entre 0.995 a 1.115 Å
CD - NE  entre 1.40 a 1.60 Å
NE - HE  entre 0.900 a 1.100 Å
NE - CZ  entre 1.25 a 1.45 Å
CZ - NH1 entre 1.25 a 1.45 Å
NH1 - 1HH1 entre 0.900 a 1.100 Å
NH1 - 2HH1 entre 0.900 a 1.100 Å
CZ - NH2 entre 1.25 a 1.45 Å
NH2 - 1HH2 entre 0.900 a 1.100 Å
NH2 - 2HH2 entre 0.900 a 1.100 Å

REGRAS HIDROGÊNIO:
CB, CG, CD ligam com 2 H (HB1/HB2, HG1/HG2, HD1/HD2)
NE   liga com 1 H (HE)
NH1  liga com 2 H (1HH1, 2HH1)
NH2  liga com 2 H (1HH2, 2HH2)

Janela dinâmica de H:
- Se #candidatos > #H: dmin += 0.005; dmax -= 0.005
- Se #candidatos < #H: dmin -= 0.005; dmax += 0.005

ÂNGULOS PESADOS:

1) CA–CB–CG, CB–CG–CD:
   - REF1: 100–127°
   - REF2: 103–124°
   - REF3: mais próximo de 113.6°
   Se REF1 ou REF2 geram 0 candidatos saindo de uma lista >=2, volta à lista anterior e aplica REF3.

2) CG–CD–NE:
   - REF1: 103–133°
   - REF2: 108–128°
   - REF3: mais próximo de 118.0°

3) CD–NE–CZ, NE–CZ–NH1, NE–CZ–NH2:
   - REF1: 100–130°
   - REF2: 114–126°
   - REF3: mais próximo de 120.0°

Para CB–CG, CG–CD, CD–NE, NE–CZ, CZ–NH1, CZ–NH2:
- Se NUM DE CANDIDATO = 0 na janela inicial:
  dmin -= 0.005; dmax += 0.005 e reavalia (até max_iter).
"""

import math

# ===========================
# Funções geométricas
# ===========================

def dist(a, b):
    return math.sqrt((a['x'] - b['x'])**2 +
                     (a['y'] - b['y'])**2 +
                     (a['z'] - b['z'])**2)

def angle(a, b, c):
    """
    Ângulo A-B-C (v1 = A-B, v2 = C-B) em graus.
    """
    v1 = (a['x'] - b['x'], a['y'] - b['y'], a['z'] - b['z'])
    v2 = (c['x'] - b['x'], c['y'] - b['y'], c['z'] - b['z'])
    n1 = math.sqrt(sum(v*v for v in v1))
    n2 = math.sqrt(sum(v*v for v in v2))
    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    dot = sum(v1[i]*v2[i] for i in range(3))
    cosang = max(-1.0, min(1.0, dot/(n1*n2)))
    return math.degrees(math.acos(cosang))

def same_coords(a, b, tol=1e-3):
    return (abs(a['x'] - b['x']) < tol and
            abs(a['y'] - b['y']) < tol and
            abs(a['z'] - b['z']) < tol)

# ===========================
# Parsing / escrita de PDB
# ===========================

def parse_pdb_lines(lines):
    atoms = []
    atom_line_idx = []
    for i, line in enumerate(lines):
        if line.startswith("ATOM") or line.startswith("HETATM"):
            try:
                rec = {
                    'line_idx': i,
                    'serial': int(line[6:11]),
                    'name':   line[12:16].strip(),
                    'altloc': line[16],
                    'resname': line[17:20].strip(),
                    'chain':  line[21].strip(),
                    'resseq': int(line[22:26]),
                    'icode':  line[26],
                    'x': float(line[30:38]),
                    'y': float(line[38:46]),
                    'z': float(line[46:54]),
                    'raw_line': line
                }
            except Exception:
                continue
            atoms.append(rec)
            atom_line_idx.append(i)
    return atoms, atom_line_idx

def update_pdb_lines(lines, atoms):
    for at in atoms:
        i = at['line_idx']
        line = lines[i]
        if len(line) < 54:
            line = line.ljust(54)
        new_line = (line[:30] +
                    f"{at['x']:8.3f}{at['y']:8.3f}{at['z']:8.3f}" +
                    line[54:])
        lines[i] = new_line
    return lines

# ===========================
# Utilitários de busca
# ===========================

def find_residue_atoms(atoms, resname, resseq, chain=None):
    idxs = []
    for i, a in enumerate(atoms):
        if a['resname'] == resname and a['resseq'] == resseq:
            if chain is None or a['chain'] == chain:
                idxs.append(i)
    return idxs

def build_name_index(atoms, residue_idxs):
    name2idx = {}
    for i in residue_idxs:
        nm = atoms[i]['name']
        if nm not in name2idx:
            name2idx[nm] = i
    return name2idx

def swap_coords(a, b):
    for coord in ('x', 'y', 'z'):
        a[coord], b[coord] = b[coord], a[coord]

# ===========================
# Seleção pesados tipo LYS (CA–CB–CG, CB–CG–CD)
# ===========================

def escolher_peso_arg_sp3(atoms, locked,
                          idx_center, idx_prev,
                          dmin_init, dmax_init,
                          delta=0.005, max_iter=200,
                          ang1_min=100.0, ang1_max=127.0,
                          ang2_min=103.0, ang2_max=124.0,
                          alvo_angulo=113.6,
                          label="PESO"):
    """
    Seleciona pesado sp3 (CG, CD) em ARG com:
      - distância center–cand em [dmin,dmax], expandindo se nenhum candidato;
      - ângulo prev–center–cand com refinamentos:
          REF1: [ang1_min, ang1_max]
          REF2: [ang2_min, ang2_max]
          REF3: mais próximo de alvo_angulo
      Se REF1 ou REF2 geram 0 candidatos saindo de uma lista >=2, volta à lista anterior e aplica REF3.
    """
    center = atoms[idx_center]
    prev   = atoms[idx_prev]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        base = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_center:
                continue
            d = dist(center, a)
            if dmin <= d <= dmax:
                ang = angle(prev, center, a)
                base.append((i, d, ang))
        if base:
            break
        # nenhum candidato -> expandir a janela
        dmin = max(0.0, dmin - delta)
        dmax += delta

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} mesmo após expandir janela.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Seleção NE (CG–CD–NE)
# ===========================

def escolher_peso_arg_ne(atoms, locked,
                         idx_center, idx_prev,
                         dmin_init=1.40, dmax_init=1.60,
                         delta=0.005, max_iter=200,
                         ang1_min=103.0, ang1_max=133.0,
                         ang2_min=108.0, ang2_max=128.0,
                         alvo_angulo=118.0,
                         label="NE"):
    """
    Seleciona NE em ARG com:
      - distância CD–cand em [dmin,dmax], expandindo se nenhum candidato;
      - ângulo CG–CD–cand com refinamentos:
          REF1: 103–133°
          REF2: 108–128°
          REF3: mais próximo de 118°
      Se REF1 ou REF2 geram 0 candidatos saindo de uma lista >=2, volta à lista anterior e aplica REF3.
    """
    center = atoms[idx_center]  # CD
    prev   = atoms[idx_prev]    # CG
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        base = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_center:
                continue
            d = dist(center, a)
            if dmin <= d <= dmax:
                ang = angle(prev, center, a)
                base.append((i, d, ang))
        if base:
            break
        dmin = max(0.0, dmin - delta)
        dmax += delta

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} mesmo após expandir janela.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Seleção planar (CD–NE–CZ, NE–CZ–NH1, NE–CZ–NH2)
# ===========================

def escolher_peso_arg_planar(atoms, locked,
                             idx_center, idx_prev,
                             dmin_init, dmax_init,
                             delta=0.005, max_iter=200,
                             ang1_min=100.0, ang1_max=130.0,
                             ang2_min=114.0, ang2_max=126.0,
                             alvo_angulo=120.0,
                             label="PLANAR"):
    """
    Seleção planar (CZ, NH1, NH2) com:
      - distância center–cand em [dmin,dmax] (expande se 0 candidatos);
      - ângulo prev–center–cand com:
          REF1: [ang1_min, ang1_max]
          REF2: [ang2_min, ang2_max]
          REF3: mais próximo de alvo_angulo
      Fallback se REF1/REF2 zeram a lista vinda com >=2 candidatos.
    """
    center = atoms[idx_center]
    prev   = atoms[idx_prev]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        base = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_center:
                continue
            d = dist(center, a)
            if dmin <= d <= dmax:
                ang = angle(prev, center, a)
                base.append((i, d, ang))
        if base:
            break
        dmin = max(0.0, dmin - delta)
        dmax += delta

    if not base:
        raise RuntimeError(f"Nenhum candidato encontrado para {label} mesmo após expandir janela.")

    if len(base) == 1:
        return base[0]

    # REF1
    prev_list = base
    prev_count = len(prev_list)
    ref1 = [(i, d, ang) for (i, d, ang) in prev_list if ang1_min <= ang <= ang1_max]
    if len(ref1) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref1) == 1:
        return ref1[0]

    # REF2
    prev_list = ref1
    prev_count = len(prev_list)
    ref2 = [(i, d, ang) for (i, d, ang) in prev_list if ang2_min <= ang <= ang2_max]
    if len(ref2) == 0 and prev_count >= 2:
        ref3 = sorted(prev_list, key=lambda t: abs(t[2] - alvo_angulo))
        return ref3[0]
    if len(ref2) == 1:
        return ref2[0]

    # REF3 em ref2
    ref3 = sorted(ref2, key=lambda t: abs(t[2] - alvo_angulo))
    return ref3[0]

# ===========================
# Janela dinâmica p/ hidrogênios
# ===========================

def escolher_vizinhos_dinamico(atoms, locked, idx_ref,
                               target_count,
                               dmin_init, dmax_init,
                               delta=0.005, max_iter=200,
                               label="H?"):
    """
    Janela dinâmica [dmin,dmax] para encontrar target_count vizinhos.
    > target_count: estreita (dmin += delta, dmax -= delta)
    < target_count: alarga (dmin -= delta, dmax += delta).
    Se não convergir, pega os target_count mais próximos.
    """
    ref = atoms[idx_ref]
    dmin = dmin_init
    dmax = dmax_init

    for _ in range(max_iter):
        cand = []
        for i, a in enumerate(atoms):
            if i in locked or i == idx_ref:
                continue
            d = dist(ref, a)
            if dmin <= d <= dmax:
                cand.append((i, d))
        if len(cand) == target_count:
            return [i for (i, _) in cand], dmin, dmax
        elif len(cand) > target_count:
            dmin += delta
            dmax -= delta
            if dmin >= dmax:
                break
        else:  # len(cand) < target_count
            dmin = max(0.0, dmin - delta)
            dmax += delta

    # fallback
    cand_all = []
    for i, a in enumerate(atoms):
        if i in locked or i == idx_ref:
            continue
        d = dist(ref, a)
        cand_all.append((i, d))
    cand_all.sort(key=lambda t: t[1])
    chosen = cand_all[:target_count]
    return [i for (i, _) in chosen], dmin, dmax

def atribuir_coord_alvos(atoms, locked, idx_ref, alvo_idxs,
                         dmin_init, dmax_init,
                         target_count,
                         delta=0.005, max_iter=200,
                         label="H?"):
    """
    Usa janela dinâmica para achar target_count candidatos ao redor de idx_ref,
    e distribui coordenadas entre os átomos alvo.

    - Primeiro fixa alvos que já têm coord igual a alguma candidata (sem swap).
    - Depois faz swap para os demais alvos com candidatos restantes.
    - Todos os alvos entram em locked ao final.
    """
    ref = atoms[idx_ref]
    cand_idxs, dmin_final, dmax_final = escolher_vizinhos_dinamico(
        atoms, locked, idx_ref, target_count,
        dmin_init, dmax_init, delta, max_iter, label
    )
    cand_coords = {i: (atoms[i]['x'], atoms[i]['y'], atoms[i]['z']) for i in cand_idxs}
    used_cands = set()

    detalhes = {
        'ref': ref,
        'ref_idx': idx_ref,
        'janela_final': (dmin_final, dmax_final),
        'mapeamentos': []
    }

    # 1) Alvos já coincidindo com algum candidato
    for idx_alvo in alvo_idxs:
        alvo = atoms[idx_alvo]
        match = None
        for i_cand in cand_idxs:
            if i_cand in used_cands:
                continue
            cx, cy, cz = cand_coords[i_cand]
            if (abs(alvo['x'] - cx) < 1e-3 and
                abs(alvo['y'] - cy) < 1e-3 and
                abs(alvo['z'] - cz) < 1e-3):
                match = i_cand
                break
        if match is not None:
            used_cands.add(match)
            locked.add(idx_alvo)
            d = dist(ref, alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': match,
                'cand': atoms[match],
                'dist_ref_alvo': d,
                'swap_feito': False
            })

    # 2) Alvos restantes com swap
    for idx_alvo in alvo_idxs:
        if idx_alvo in locked:
            continue
        alvo = atoms[idx_alvo]
        cand_rest = [i for i in cand_idxs if i not in used_cands]
        if not cand_rest:
            locked.add(idx_alvo)
            detalhes['mapeamentos'].append({
                'alvo_idx': idx_alvo,
                'alvo': alvo,
                'cand_idx': None,
                'cand': None,
                'dist_ref_alvo': dist(ref, alvo),
                'swap_feito': False
            })
            continue

        cand_rest.sort(key=lambda i: dist(ref, atoms[i]))
        i_cand = cand_rest[0]
        used_cands.add(i_cand)
        cand_atom = atoms[i_cand]

        d_before = dist(ref, alvo)
        if not same_coords(alvo, cand_atom):
            swap_coords(alvo, cand_atom)
            swap_feito = True
        else:
            swap_feito = False

        locked.add(idx_alvo)
        d_after = dist(ref, alvo)
        detalhes['mapeamentos'].append({
            'alvo_idx': idx_alvo,
            'alvo': alvo,
            'cand_idx': i_cand,
            'cand': cand_atom,
            'dist_ref_alvo_antes': d_before,
            'dist_ref_alvo_depois': d_after,
            'swap_feito': swap_feito
        })

    return detalhes

# ===========================
# Pipeline principal ARG 582
# ===========================

def reconstruir_arg582_sidechain(pdb_filename,
                                 saida_filename="backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581_ARG582.pdb",
                                 resname="ARG", resseq=582, chain="B"):
    with open(pdb_filename, 'r') as f:
        lines = f.read().splitlines()

    atoms, atom_line_idx = parse_pdb_lines(lines)
    print(f"PDB carregado: {pdb_filename}")
    print(f"Total de átomos lidos: {len(atoms)}")

    # locked inicial: N, CA, C, HN, HA, O, CB
    locked = set()
    base_locked_names = {"N", "CA", "C", "HN", "HA", "O", "CB"}
    for i, a in enumerate(atoms):
        if a['name'] in base_locked_names:
            locked.add(i)

    # Bloqueia todo ILE 576, THR 577, LEU 578, TYR 579, CYS 580, LYS 581
    for (rname, rseq) in [("ILE", 576),
                          ("THR", 577),
                          ("LEU", 578),
                          ("TYR", 579),
                          ("CYS", 580),
                          ("LYS", 581)]:
        idxs_res = find_residue_atoms(atoms, resname=rname, resseq=rseq, chain=chain)
        for i in idxs_res:
            locked.add(i)

    print(f"Átomos inicializados como imutáveis (locked): {len(locked)}")

    # Resíduo ARG 582
    res_idxs = find_residue_atoms(atoms, resname=resname, resseq=resseq, chain=chain)
    if not res_idxs:
        raise RuntimeError(f"Resíduo {resname} {chain} {resseq} não encontrado.")

    print(f"Reconstruindo cadeia lateral de {resname} {chain} {resseq}")
    name2idx = build_name_index(atoms, res_idxs)

    # Índices importantes
    idx_CA   = name2idx.get("CA")
    idx_CB   = name2idx.get("CB")
    idx_HB1  = name2idx.get("HB1")
    idx_HB2  = name2idx.get("HB2")
    idx_CG   = name2idx.get("CG")
    idx_HG1  = name2idx.get("HG1")
    idx_HG2  = name2idx.get("HG2")
    idx_CD   = name2idx.get("CD")
    idx_HD1  = name2idx.get("HD1")
    idx_HD2  = name2idx.get("HD2")
    idx_NE   = name2idx.get("NE")
    idx_HE   = name2idx.get("HE")
    idx_CZ   = name2idx.get("CZ")
    idx_NH1  = name2idx.get("NH1")
    idx_1HH1 = name2idx.get("1HH1")
    idx_2HH1 = name2idx.get("2HH1")
    idx_NH2  = name2idx.get("NH2")
    idx_1HH2 = name2idx.get("1HH2")
    idx_2HH2 = name2idx.get("2HH2")

    obrigatorios = [
        "CA", "CB", "HB1", "HB2", "CG", "HG1", "HG2",
        "CD", "HD1", "HD2", "NE", "HE", "CZ",
        "NH1", "1HH1", "2HH1", "NH2", "1HH2", "2HH2"
    ]
    for nm in obrigatorios:
        if nm not in name2idx:
            print(f"AVISO: átomo {nm} não encontrado em {resname} {chain} {resseq}.")

    # =======================
    # 1) CB–HB1/HB2
    # =======================
    print("\n===== ETAPA 1: CB–HB1/HB2 =====")
    alvo_hb = [idx for idx in (idx_HB1, idx_HB2) if idx is not None]
    if alvo_hb and idx_CB is not None:
        det_hb = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CB,
            alvo_idxs=alvo_hb,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hb),
            delta=0.005, max_iter=200,
            label="HB"
        )
        print(f"Janela final CB–HB1/HB2 usada: {det_hb['janela_final'][0]:.3f} – {det_hb['janela_final'][1]:.3f} Å")
        for m in det_hb['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HB] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f" com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HB1/HB2 não encontrados ou CB ausente, pulando etapa.")

    # =======================
    # 2) CB–CG (pesado)
    # =======================
    print("\n===== ETAPA 2: CB–CG =====")
    if idx_CG is not None and idx_CB is not None and idx_CA is not None:
        cand_idx, d_cg, ang_cg = escolher_peso_arg_sp3(
            atoms, locked,
            idx_center=idx_CB, idx_prev=idx_CA,
            dmin_init=1.40, dmax_init=1.60,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=127.0,
            ang2_min=103.0, ang2_max=124.0,
            alvo_angulo=113.6,
            label="CG"
        )
        cand = atoms[cand_idx]
        print(f"CG candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CB–candidato = {d_cg:.3f} Å")
        print(f"  Ângulo CA–CB–candidato = {ang_cg:.3f} °")

        at_cg = atoms[idx_CG]
        if not same_coords(at_cg, cand):
            print("  -> Fazendo swap de coordenadas entre CG e candidato.")
            swap_coords(at_cg, cand)
        else:
            print("  -> CG já está usando essa coordenada (nenhum swap).")
        locked.add(idx_CG)
    else:
        print("  -> CA/CB/CG ausente(s), pulando etapa CB–CG.")

    # =======================
    # 3) CG–HG1/HG2
    # =======================
    print("\n===== ETAPA 3: CG–HG1/HG2 =====")
    alvo_hg = [idx for idx in (idx_HG1, idx_HG2) if idx is not None]
    if alvo_hg and idx_CG is not None:
        det_hg = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CG,
            alvo_idxs=alvo_hg,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hg),
            delta=0.005, max_iter=200,
            label="HG"
        )
        print(f"Janela final CG–HG1/HG2 usada: {det_hg['janela_final'][0]:.3f} – {det_hg['janela_final'][1]:.3f} Å")
        for m in det_hg['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HG] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HG1/HG2 não encontrados ou CG ausente, pulando etapa.")

    # =======================
    # 4) CG–CD (pesado)
    # =======================
    print("\n===== ETAPA 4: CG–CD =====")
    if idx_CD is not None and idx_CG is not None and idx_CB is not None:
        cand_idx, d_cd, ang_cd = escolher_peso_arg_sp3(
            atoms, locked,
            idx_center=idx_CG, idx_prev=idx_CB,
            dmin_init=1.40, dmax_init=1.60,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=127.0,
            ang2_min=103.0, ang2_max=124.0,
            alvo_angulo=113.6,
            label="CD"
        )
        cand = atoms[cand_idx]
        print(f"CD candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CG–candidato = {d_cd:.3f} Å")
        print(f"  Ângulo CB–CG–candidato = {ang_cd:.3f} °")

        at_cd = atoms[idx_CD]
        if not same_coords(at_cd, cand):
            print("  -> Fazendo swap de coordenadas entre CD e candidato.")
            swap_coords(at_cd, cand)
        else:
            print("  -> CD já está usando essa coordenada (nenhum swap).")
        locked.add(idx_CD)
    else:
        print("  -> CB/CG/CD ausente(s), pulando etapa CG–CD.")

    # =======================
    # 5) CD–HD1/HD2
    # =======================
    print("\n===== ETAPA 5: CD–HD1/HD2 =====")
    alvo_hd = [idx for idx in (idx_HD1, idx_HD2) if idx is not None]
    if alvo_hd and idx_CD is not None:
        det_hd = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_CD,
            alvo_idxs=alvo_hd,
            dmin_init=0.995, dmax_init=1.115,
            target_count=len(alvo_hd),
            delta=0.005, max_iter=200,
            label="HD"
        )
        print(f"Janela final CD–HD1/HD2 usada: {det_hd['janela_final'][0]:.3f} – {det_hd['janela_final'][1]:.3f} Å")
        for m in det_hd['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HD] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HD] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HD] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HD1/HD2 não encontrados ou CD ausente, pulando etapa.")

    # =======================
    # 6) CD–NE (pesado)
    # =======================
    print("\n===== ETAPA 6: CD–NE =====")
    if idx_NE is not None and idx_CD is not None and idx_CG is not None:
        cand_idx, d_ne, ang_ne = escolher_peso_arg_ne(
            atoms, locked,
            idx_center=idx_CD, idx_prev=idx_CG,
            dmin_init=1.40, dmax_init=1.60,
            delta=0.005, max_iter=200,
            ang1_min=103.0, ang1_max=133.0,
            ang2_min=108.0, ang2_max=128.0,
            alvo_angulo=118.0,
            label="NE"
        )
        cand = atoms[cand_idx]
        print(f"NE candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CD–candidato = {d_ne:.3f} Å")
        print(f"  Ângulo CG–CD–candidato = {ang_ne:.3f} °")

        at_ne = atoms[idx_NE]
        if not same_coords(at_ne, cand):
            print("  -> Fazendo swap de coordenadas entre NE e candidato.")
            swap_coords(at_ne, cand)
        else:
            print("  -> NE já está usando essa coordenada (nenhum swap).")
        locked.add(idx_NE)
    else:
        print("  -> CG/CD/NE ausente(s), pulando etapa CD–NE.")

    # =======================
    # 7) NE–HE
    # =======================
    print("\n===== ETAPA 7: NE–HE =====")
    if idx_HE is not None and idx_NE is not None:
        det_he = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_NE,
            alvo_idxs=[idx_HE],
            dmin_init=0.900, dmax_init=1.100,
            target_count=1,
            delta=0.005, max_iter=200,
            label="HE"
        )
        print(f"Janela final NE–HE usada: {det_he['janela_final'][0]:.3f} – {det_he['janela_final'][1]:.3f} Å")
        for m in det_he['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HE] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HE] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HE] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> HE não encontrado ou NE ausente, pulando etapa.")

    # =======================
    # 8) NE–CZ (pesado planar)
    # =======================
    print("\n===== ETAPA 8: NE–CZ =====")
    if idx_CZ is not None and idx_NE is not None and idx_CD is not None:
        cand_idx, d_cz, ang_cz = escolher_peso_arg_planar(
            atoms, locked,
            idx_center=idx_NE, idx_prev=idx_CD,
            dmin_init=1.25, dmax_init=1.45,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=130.0,
            ang2_min=114.0, ang2_max=126.0,
            alvo_angulo=120.0,
            label="CZ"
        )
        cand = atoms[cand_idx]
        print(f"CZ candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância NE–candidato = {d_cz:.3f} Å")
        print(f"  Ângulo CD–NE–candidato = {ang_cz:.3f} °")

        at_cz = atoms[idx_CZ]
        if not same_coords(at_cz, cand):
            print("  -> Fazendo swap de coordenadas entre CZ e candidato.")
            swap_coords(at_cz, cand)
        else:
            print("  -> CZ já está usando essa coordenada (nenhum swap).")
        locked.add(idx_CZ)
    else:
        print("  -> CD/NE/CZ ausente(s), pulando etapa NE–CZ.")

    # =======================
    # 9) CZ–NH1 (pesado planar)
    # =======================
    print("\n===== ETAPA 9: CZ–NH1 =====")
    if idx_NH1 is not None and idx_CZ is not None and idx_NE is not None:
        cand_idx, d_nh1, ang_nh1 = escolher_peso_arg_planar(
            atoms, locked,
            idx_center=idx_CZ, idx_prev=idx_NE,
            dmin_init=1.25, dmax_init=1.45,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=130.0,
            ang2_min=114.0, ang2_max=126.0,
            alvo_angulo=120.0,
            label="NH1"
        )
        cand = atoms[cand_idx]
        print(f"NH1 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CZ–candidato = {d_nh1:.3f} Å")
        print(f"  Ângulo NE–CZ–candidato = {ang_nh1:.3f} °")

        at_nh1 = atoms[idx_NH1]
        if not same_coords(at_nh1, cand):
            print("  -> Fazendo swap de coordenadas entre NH1 e candidato.")
            swap_coords(at_nh1, cand)
        else:
            print("  -> NH1 já está usando essa coordenada (nenhum swap).")
        locked.add(idx_NH1)
    else:
        print("  -> NE/CZ/NH1 ausente(s), pulando etapa CZ–NH1.")

    # =======================
    # 10) NH1–1HH1/2HH1
    # =======================
    print("\n===== ETAPA 10: NH1–1HH1/2HH1 =====")
    alvo_hh1 = [idx for idx in (idx_1HH1, idx_2HH1) if idx is not None]
    if alvo_hh1 and idx_NH1 is not None:
        det_hh1 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_NH1,
            alvo_idxs=alvo_hh1,
            dmin_init=0.900, dmax_init=1.100,
            target_count=len(alvo_hh1),
            delta=0.005, max_iter=200,
            label="HH1"
        )
        print(f"Janela final NH1–1HH1/2HH1 usada: {det_hh1['janela_final'][0]:.3f} – {det_hh1['janela_final'][1]:.3f} Å")
        for m in det_hh1['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HH1] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HH1] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HH1] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> 1HH1/2HH1 não encontrados ou NH1 ausente, pulando etapa.")

    # =======================
    # 11) CZ–NH2 (pesado planar)
    # =======================
    print("\n===== ETAPA 11: CZ–NH2 =====")
    if idx_NH2 is not None and idx_CZ is not None and idx_NE is not None:
        cand_idx, d_nh2, ang_nh2 = escolher_peso_arg_planar(
            atoms, locked,
            idx_center=idx_CZ, idx_prev=idx_NE,
            dmin_init=1.25, dmax_init=1.45,
            delta=0.005, max_iter=200,
            ang1_min=100.0, ang1_max=130.0,
            ang2_min=114.0, ang2_max=126.0,
            alvo_angulo=120.0,
            label="NH2"
        )
        cand = atoms[cand_idx]
        print(f"NH2 candidato: serial={cand['serial']} {cand['name']} {cand['resname']} {cand['resseq']}")
        print(f"  Distância CZ–candidato = {d_nh2:.3f} Å")
        print(f"  Ângulo NE–CZ–candidato = {ang_nh2:.3f} °")

        at_nh2 = atoms[idx_NH2]
        if not same_coords(at_nh2, cand):
            print("  -> Fazendo swap de coordenadas entre NH2 e candidato.")
            swap_coords(at_nh2, cand)
        else:
            print("  -> NH2 já está usando essa coordenada (nenhum swap).")
        locked.add(idx_NH2)
    else:
        print("  -> NE/CZ/NH2 ausente(s), pulando etapa CZ–NH2.")

    # =======================
    # 12) NH2–1HH2/2HH2
    # =======================
    print("\n===== ETAPA 12: NH2–1HH2/2HH2 =====")
    alvo_hh2 = [idx for idx in (idx_1HH2, idx_2HH2) if idx is not None]
    if alvo_hh2 and idx_NH2 is not None:
        det_hh2 = atribuir_coord_alvos(
            atoms, locked,
            idx_ref=idx_NH2,
            alvo_idxs=alvo_hh2,
            dmin_init=0.900, dmax_init=1.100,
            target_count=len(alvo_hh2),
            delta=0.005, max_iter=200,
            label="HH2"
        )
        print(f"Janela final NH2–1HH2/2HH2 usada: {det_hh2['janela_final'][0]:.3f} – {det_hh2['janela_final'][1]:.3f} Å")
        for m in det_hh2['mapeamentos']:
            alvo = atoms[m['alvo_idx']]
            if m.get('cand_idx') is None:
                print(f"  [HH2] alvo {alvo['name']}({alvo['serial']}) sem candidato explícito, apenas locked.")
                continue
            cand = atoms[m['cand_idx']]
            if m['swap_feito']:
                print(f"  [HH2] alvo {alvo['name']}({alvo['serial']}) "
                      f"<-> candidato {cand['name']}({cand['serial']}): "
                      f"swap. dist_ref_antes={m['dist_ref_alvo_antes']:.3f} Å, "
                      f"dist_ref_depois={m['dist_ref_alvo_depois']:.3f} Å")
            else:
                print(f"  [HH2] alvo {alvo['name']}({alvo['serial']}) já coincidia "
                      f"com candidato {cand['name']}({cand['serial']}), sem swap. "
                      f"dist_ref={m.get('dist_ref_alvo', m.get('dist_ref_alvo_antes', 0.0)):.3f} Å")
    else:
        print("  -> 1HH2/2HH2 não encontrados ou NH2 ausente, pulando etapa.")

    # ===========================
    # Escrita do novo PDB
    # ===========================
    lines_out = update_pdb_lines(lines, atoms)
    with open(saida_filename, 'w') as f:
        f.write("\n".join(lines_out) + "\n")
    print(f"\nNovo PDB escrito em: {saida_filename}")

    return saida_filename

# ===========================
# Bloco Colab
# ===========================

if __name__ == "__main__":
    try:
        from google.colab import files  # type: ignore
        print("Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581.pdb):")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("Nenhum arquivo foi carregado.")
        pdb_in = list(uploaded.keys())[0]
        saida = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581_ARG582.pdb"
        reconstruir_arg582_sidechain(pdb_in, saida_filename=saida,
                                     resname="ARG", resseq=582, chain="B")
        files.download(saida)
    except ImportError:
        # Uso local fora do Colab
        pdb_in = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581.pdb"
        saida = "backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581_ARG582.pdb"
        reconstruir_arg582_sidechain(pdb_in, saida_filename=saida,
                                     resname="ARG", resseq=582, chain="B")


Selecione o arquivo PDB de entrada (ex.: backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581.pdb):


Saving backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581.pdb to backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581 (2).pdb
PDB carregado: backbone_rebuilt_with_HNO_CB_TYR579_CYS580_LYS581 (2).pdb
Total de átomos lidos: 137
Átomos inicializados como imutáveis (locked): 118
Reconstruindo cadeia lateral de ARG B 582

===== ETAPA 1: CB–HB1/HB2 =====
Janela final CB–HB1/HB2 usada: 0.995 – 1.115 Å
  [HB] alvo HB1(8752) <-> candidato HG1(8755): swap. dist_ref_antes=2.233 Å, dist_ref_depois=1.109 Å
  [HB] alvo HB2(8753) <-> candidato CG(8754): swap. dist_ref_antes=2.139 Å, dist_ref_depois=1.112 Å

===== ETAPA 2: CB–CG =====
CG candidato: serial=8756 HG2 ARG 582
  Distância CB–candidato = 1.526 Å
  Ângulo CA–CB–candidato = 110.972 °
  -> Fazendo swap de coordenadas entre CG e candidato.

===== ETAPA 3: CG–HG1/HG2 =====
Janela final CG–HG1/HG2 usada: 0.990 – 1.120 Å
  [HG] alvo HG1(8755) já coincidia com candidato HG1(8755), sem swap. dist_ref=1.117 Å
  [HG] alvo HG2(8756) já coincidia com candid

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>